# PART A and B


In [23]:
import numpy as np
import pandas as pd


In [2]:
np.random.seed(1)


In [3]:
# data=pd.read_csv('toy_dataset_train.csv',header=None)
# X_test=pd.read_csv('toy_dataset_test.csv',header=None).iloc[:,1:].to_numpy()/255.
# y_test=pd.read_csv('toy_dataset_test_labels.csv',header=None)
# y_test=pd.get_dummies(y_test,columns=y_test.columns).to_numpy()
# X,y=data.iloc[:,1:].to_numpy()/255.,pd.get_dummies(data.iloc[:,0],columns=data.columns[0]).to_numpy()
# X.shape,y.shape,X_test.shape,y_test.shape

((3000, 200), (3000, 2), (300, 200), (300, 2))

In [3]:
data=pd.read_csv('train_data_shuffled.csv',header=None)
test=pd.read_csv('public_test.csv',header=None)
X_test,y_test=test.iloc[:,:-1].to_numpy()/255.,pd.get_dummies(test.iloc[:,-1:],columns=test.columns[-1:]).to_numpy()
X,y=data.iloc[:,:-1].to_numpy()/255.,pd.get_dummies(data.iloc[:,-1:],columns=data.columns[-1:]).to_numpy()
X.shape,y.shape,X_test.shape,y_test.shape


((78200, 1024), (78200, 46), (4600, 1024), (4600, 46))

In [4]:
def softmax(a):
    m=np.max(a,axis=1).reshape(-1,1)
    #gamma=1e-15
    a=np.exp(a-m)
    #return a/(a.sum(axis=a.ndim-1).reshape(-1,1))
    return np.nan_to_num(a/(a.sum(axis=a.ndim-1).reshape(-1,1)),False)
def relu(a):
    return np.where(a>0,a,0)
def drelu(a):
    return np.where(a>0,1,0)
def tanh(a):
    return np.tanh(a)  
def dtanh(a):
    return 1-np.square(a)
def sigmoid(a):
    return 1/(1+np.exp(-a))
    #return expit(a)
def dsig(a):
    #return sigmoid(a)*(1-sigmoid(a))
    return a*(1-a)
    #return (1-expit(a))*expit(a)
def CE_loss(y_true,y_preds):
    gamma=1e-15
    return -np.sum(np.multiply(y_true,np.log(np.clip(y_preds,gamma,1-gamma))))/y_true.shape[0]

def mse(y_true,y_preds):  
    return np.sum(np.square(y_true.reshape(-1,1)-y_preds.reshape(-1,1)))/y_preds.shape[0]


def acc(y_true,y_preds):
    y_preds=np.argmax(y_preds,axis=1).squeeze()
    y_true=np.argmax(y_true,axis=1).squeeze()
    return np.count_nonzero(y_true-y_preds==0)/len(y_true)

In [9]:
class ANN:
    hidden_units=daf=af=l=W=Z=0
    
    def __init__(self,hidden_units,af,num_features,loss):
        
        self.hidden_units=hidden_units
        self.l=loss
        if(af==0):
            self.af=sigmoid
            self.daf=dsig
        elif(af==1):
            self.af=tanh
            self.daf=dtanh
        else:
            self.af=relu
            self.daf=drelu
        if(loss==0):
            self.loss=CE_loss
        else:
            self.loss=mse 
        self.W=[]
        self.b=[]
        temp=np.float32(np.random.normal(0,1,(num_features+1,hidden_units[0])))*(np.sqrt(2/(num_features+1+hidden_units[0])))
        self.W.append(temp[1:,:])
        self.b.append(temp[0])
        for i in range(1,len(hidden_units)):
            temp=np.float32(np.random.normal(0,1,(hidden_units[i-1]+1,hidden_units[i])))*(np.sqrt(2/(hidden_units[i-1]+1+hidden_units[i])))
            self.W.append(temp[1:,:])
            self.b.append(temp[0])
        self.Z=[]    
    def fp(self,X):
        self.Z=[]
        
        self.Z.append(X)
        
#         for i in range(len(self.W)):
#             self.Z.append(self.af(np.matmul(self.Z[i],self.W[i])+self.b[i]))
            
#         if(self.l==0):
#             return softmax(self.Z[-1]) 
        for i in range(len(self.W)-1):
            self.Z.append(self.af(np.matmul(self.Z[i],self.W[i])+self.b[i]))
            
        if(self.l==0):
            #print(self.Z[-1].shape,self.W[-1].shape,self.b[-1].shape)
            self.Z.append(np.matmul(self.Z[-1],self.W[-1])+self.b[-1])
            # In case of Cross Entropy loss, softmax is the final activation
            return softmax(self.Z[-1])
            
        else:
            self.Z.append(self.af(np.matmul(self.Z[-1],self.W[-1])+self.b[-1]))
        return self.Z[-1]
    def bp(self,X,y,lr):
        yh=self.fp(X)
        g=yh-y
        # Note:  In case of MSE:Y_h=Z_l ---> dL/dZ_l=Y_h-Y
        #        In case of CE Loss:Y_h=Softmax(Z_l) --->dL/dZ_l= (dL/dY_h)*(dY_h/dZ_l)= Y_h-Y
        # Since its same in both the cases, no need for different functions
        
        #print(g)
        dummy=np.ones(yh.shape[0])
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            
            if(i<len(self.W)-1 or self.l==1):
                # if loss== mse (l=1) ---> we multiply with deriv of af always,
                # but if loss==CE ---> last layer has softmax activation, whose derivative is already incorporated in yh-y 
                #so no need
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #print(dw[0])
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
            
            g=np.dot(g,self.W[i].T)
            self.W[i]-=dw*lr
            self.b[i]-=db*lr
#     def bp(self,X,y,lr):
#         yh=self.fp(X)
#         g=yh-y
#         #print(g)
#         dummy=np.ones(yh.shape[0])
#         for i in reversed(range(len(self.W))):
#             deriv_af=self.daf(self.Z[i+1])
#             g=np.multiply(deriv_af,g)
#             dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
#             #print(dw[0])
#             #db=np.matmul(dummy,g)/yh.shape[0]
#             db=np.sum(g,axis=0)/yh.shape[0]
            
#             g=np.dot(g,self.W[i].T)
#             self.W[i]-=dw*lr
#             self.b[i]-=db*lr
    def bp_mom(self,X,y,lr,wt,bt,gamma=.9):
        yh=self.fp(X)
        g=yh-y
        #print(g)
        #print(gamma)
        dummy=np.ones(yh.shape[0])
        
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            if(i<len(self.W)-1 or self.l==1):
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
            g=np.dot(g,self.W[i].T)
            #print(gamma*wt[i])
            change=gamma*wt[i]+dw*lr
            self.W[i]-=change
            wt[i]=change
            #print(wt[i])
            change=gamma*bt[i]+db*lr
            self.b[i]-=change
            bt[i]=change
    
    def bp_rmsprop(self,X,y,lr,wt,bt,gamma=.9):
        yh=self.fp(X)
        g=yh-y
        #print(g)
        #print(gamma)
        dummy=np.ones(yh.shape[0])
        e=1e-8
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            
            if(i<len(self.W)-1 or self.l==1):
                
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
            g=np.dot(g,self.W[i].T)
            #print(gamma*wt[i])
            temp=gamma*wt[i]+0.1*(dw**2)
            self.W[i]-=lr*dw/(np.sqrt(temp+e))
            wt[i]=temp
            #print(wt[i])
            temp=gamma*bt[i]+.1*(db**2)
            self.b[i]-=lr*db/(np.sqrt(temp+e))
            bt[i]=temp
    def bp_adam(self,X,y,lr,vwt,vbt,mwt,mbt,b1=.9,b2=.999):
        yh=self.fp(X)
        g=yh-y
        #print(g)
        #print(gamma)
        dummy=np.ones(yh.shape[0])
        e=1e-8
        b1t=1
        b2t=1
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            
            if(i<len(self.W)-1 or self.l==1):
                
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
            g=np.dot(g,self.W[i].T)
            #print(gamma*wt[i])
            vwt[i]=b2*vwt[i]+(1-b2)*(dw**2)
            vbt[i]=b2*vbt[i]+(1-b2)*(db**2)
            mwt[i]=b1*mwt[i]+(1-b1)*(dw)
            mbt[i]=b1*mbt[i]+(1-b1)*(db)
            #print(mbt[i])
            b1t*=b1
            b2t*=b2
            self.W[i]-=lr*(mwt[i]/(1-b1t))/(np.sqrt(vwt[i]/(1-b2t))+e)
            self.b[i]-=lr*(mbt[i]/(1-b1t))/(np.sqrt(vbt[i]/(1-b2t))+e)
            
    def bp_nag(self,X,y,lr,wt,bt,gamma=.9):
        yh=self.fp(X)
        g=yh-y
        #print(g)
        #print(gamma)
        dummy=np.ones(yh.shape[0])
        e=1e-8
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            
            if(i<len(self.W)-1 or self.l==1):
                
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
            
            g=np.dot(g,self.W[i].T)
            #print(gamma*wt[i])
            temp=gamma*wt[i]+lr*(dw**2)
            self.W[i]-=temp
            wt[i]=temp
            #print(wt[i])
            temp=gamma*bt[i]+.1*(db**2)
            self.b[i]-=temp
            bt[i]=temp
    def fit(self,bs,epochs,X,y,lr,adaptive=False):
        for i in range(len(self.W)):
            self.W[i]=np.float64(self.W[i])
            self.b[i]=np.float64(self.b[i])
        l=[]    
        for i in range(epochs):
            if(adaptive):
                lr/=(i+1)**.5
            for j in range(X.shape[0]//bs):
#                 if(j==5 and i==0):
#                     for k in range(len(a.W)):
#                         temp=np.concatenate((a.b[k].reshape(1,-1),a.W[k]),axis=0)
#                         np.save('essentials/part_a_and_b/multiclass_dataset/tc_3/w_'+str(k+1)+'_iter.npy',temp)
#                         print(np.max(np.load('essentials/part_a_and_b/multiclass_dataset/tc_3/ac_w_'+str(k+1)+'_iter.npy')-temp))
                    
                self.bp(X[j*bs:(j+1)*bs,:],y[j*bs:(j+1)*bs,:],lr)
            y_pred=self.fp(X_test)
            l.append(self.loss(y,self.fp(X)))
            print(i,l[-1],acc(y,self.fp(X)))
            #print(i,self.loss(y_test,y_pred),acc(y_test,y_pred))
        return l    
    def fit_mom(self,bs,epochs,X,y,lr,adaptive=False):
        l=[]
        wt=[0]*len(self.W)
        bt=[0]*len(self.W)
        for i in range(epochs):
            if(adaptive):
                lr/=(i+1)**.5
            for j in range(X.shape[0]//bs):
                self.bp_mom(X[j*bs:(j+1)*bs,:],y[j*bs:(j+1)*bs,:],lr,wt,bt)
            y_pred=self.fp(X_test)
            l.append(self.loss(y,self.fp(X)))
            print(i,l[-1],acc(y,self.fp(X)))
            #print(i,self.loss(y_test,y_pred),acc(y_test,y_pred)) 
        return l
    def fit_adam(self,bs,epochs,X,y,lr,adaptive=False):
        mwt=[0]*len(self.W)
        mbt=[0]*len(self.W)
        vwt=[0]*len(self.W)
        vbt=[0]*len(self.W)
        l=[]
        for i in range(epochs):
            if(adaptive):
                lr/=(i+1)**.5
            for j in range(X.shape[0]//bs):
                self.bp_adam(X[j*bs:(j+1)*bs,:],y[j*bs:(j+1)*bs,:],lr,vwt,vbt,mwt,mbt)
            y_pred=self.fp(X_test) 
            l.append(self.loss(y,self.fp(X)))
            print(i,l[-1],acc(y,self.fp(X)))
            #print(i,self.loss(y_test,y_pred),acc(y_test,y_pred))    
        return l
    def fit_rmsprop(self,bs,epochs,X,y,lr,adaptive=False):
        wt=[0]*len(self.W)
        bt=[0]*len(self.W)
        l=[]
        for i in range(epochs):
            if(adaptive):
                lr/=(i+1)**.5
            for j in range(X.shape[0]//bs):
                self.bp_rmsprop(X[j*bs:(j+1)*bs,:],y[j*bs:(j+1)*bs,:],lr,wt,bt)
            y_pred=self.fp(X_test)
            l.append(self.loss(y,self.fp(X)))
            print(i,l[-1],acc(y,self.fp(X)))
            #print(i,self.loss(y_test,y_pred),acc(y_test,y_pred))             
        return l    
    def pred(X):
        return fp(X)
        
            

In [ ]:
a=ANN([512,256,128,46],2,1024,1)

In [ ]:
a.fit(1700,5,X,y,.01)

In [ ]:
for i in range(len(a.W)):
    temp=np.concatenate((a.b[i].reshape(1,-1),a.W[i]),axis=0)
    np.save('essentials/part_a_and_b/multiclass_dataset/tc_3/w_'+str(i+1)+'.npy',temp)
    print(np.max((np.load('essentials/part_a_and_b/multiclass_dataset/tc_3/ac_w_'+str(i+1)+'.npy')-temp)))
    #print(np.max((np.load('essentials/part_a_and_b/multiclass_dataset/tc_3/ac_w_'+str(i+1)+'.npy')-temp)/np.load('essentials/part_a_and_b/multiclass_dataset/tc_3/ac_w_'+str(i+1)+'.npy')))

In [ ]:
a=ANN([100,50,20,10,2],2,200,0)
#a.fp(X)
a.W[0].dtype

In [ ]:
a.fit(100,5,X,y,.001)
a.W[0].dtype

In [ ]:
for i in range(len(a.W)):
    temp=np.concatenate((a.b[i].reshape(1,-1),a.W[i]),axis=0)
    np.save('essentials/part_a_and_b/toy_dataset/tc_3/w_'+str(i+1)+'.npy',temp)
    print(np.max((np.load('essentials/part_a_and_b/toy_dataset/tc_3/ac_w_'+str(i+1)+'.npy')-temp)))
    #print(np.max((np.load('essentials/part_a_and_b/toy_dataset/tc_3/ac_w_'+str(i+1)+'.npy')-temp)/np.load('essentials/part_a_and_b/toy_dataset/tc_3/ac_w_'+str(i+1)+'.npy')))

# PART C

In [1]:
import numpy as np
import pandas as pd
#from scipy.special import expit
import matplotlib
from matplotlib import pyplot as plt
matplotlib.use('Agg')
from time import time

In [2]:
np.random.seed(146)


data=pd.read_csv('data/train_data_shuffled.csv',header=None)
test=pd.read_csv('data/public_test.csv',header=None)
X_test,y_test=test.iloc[:,:-1].to_numpy()/255.,pd.get_dummies(test.iloc[:,-1:],columns=test.columns[-1:]).to_numpy()
X,y=data.iloc[:,:-1].to_numpy()/255.,pd.get_dummies(data.iloc[:,-1:],columns=data.columns[-1:]).to_numpy()
X.shape,y.shape,X_test.shape,y_test.shape


((78200, 1024), (78200, 46), (4600, 1024), (4600, 46))

In [3]:
def softmax(a):
    m=np.max(a,axis=1).reshape(-1,1)
    #gamma=1e-15
    a=np.exp(a-m)
    #return a/(a.sum(axis=a.ndim-1).reshape(-1,1))
    return np.nan_to_num(a/(a.sum(axis=a.ndim-1).reshape(-1,1)),False)
def relu(a):
    return np.where(a>0,a,0)
def drelu(a):
    return np.where(a>0,1,0)
def tanh(a):
    return np.tanh(a)  
def dtanh(a):
    return 1-np.square(a)
def sigmoid(a):
    return 1/(1+np.exp(-a))
    #return expit(a)
def dsig(a):
    #return sigmoid(a)*(1-sigmoid(a))
    return a*(1-a)
    #return (1-expit(a))*expit(a)
def CE_loss(y_true,y_preds):
    gamma=1e-15
    return -np.sum(np.multiply(y_true,np.log(np.clip(y_preds,gamma,1-gamma))))/y_true.shape[0]

def mse(y_true,y_preds):  
    return np.sum(np.square(y_true.reshape(-1,1)-y_preds.reshape(-1,1)))/y_preds.shape[0]


def acc(y_true,y_preds):
    y_preds=np.argmax(y_preds,axis=1).squeeze()
    y_true=np.argmax(y_true,axis=1).squeeze()
    return np.count_nonzero(y_true-y_preds==0)/len(y_true)

In [4]:
class ANN:
    hidden_units=daf=af=l=W=Z=0
    
    def __init__(self,hidden_units,af,num_features,loss):
        
        self.hidden_units=hidden_units
        self.l=loss
        if(af==0):
            self.af=sigmoid
            self.daf=dsig
        elif(af==1):
            self.af=tanh
            self.daf=dtanh
        else:
            self.af=relu
            self.daf=drelu
        if(loss==0):
            self.loss=CE_loss
        else:
            self.loss=mse 
        self.W=[]
        self.b=[]
        temp=np.float32(np.random.normal(0,1,(num_features+1,hidden_units[0])))*(np.sqrt(2/(num_features+1+hidden_units[0])))
        self.W.append(temp[1:,:])
        self.b.append(temp[0])
        for i in range(1,len(hidden_units)):
            temp=np.float32(np.random.normal(0,1,(hidden_units[i-1]+1,hidden_units[i])))*(np.sqrt(2/(hidden_units[i-1]+1+hidden_units[i])))
            self.W.append(temp[1:,:])
            self.b.append(temp[0])
        self.Z=[]    
    def fp(self,X):
        self.Z=[]
        
        self.Z.append(X)
        
#         for i in range(len(self.W)):
#             self.Z.append(self.af(np.matmul(self.Z[i],self.W[i])+self.b[i]))
            
#         if(self.l==0):
#             return softmax(self.Z[-1]) 
        for i in range(len(self.W)-1):
            self.Z.append(self.af(np.matmul(self.Z[i],self.W[i])+self.b[i]))
            
        if(self.l==0):
            #print(self.Z[-1].shape,self.W[-1].shape,self.b[-1].shape)
            self.Z.append(np.matmul(self.Z[-1],self.W[-1])+self.b[-1])
            return softmax(self.Z[-1])
            
        else:
            self.Z.append(self.af(np.matmul(self.Z[-1],self.W[-1])+self.b[-1]))
        return self.Z[-1]
    def bp(self,X,y,lr):
        yh=self.fp(X)
        g=yh-y
        #print(g)
        dummy=np.ones(yh.shape[0])
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            
            if(i<len(self.W)-1 or self.l==1):
                
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #print(dw[0])
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
            
            g=np.dot(g,self.W[i].T)
            self.W[i]-=dw*lr
            self.b[i]-=db*lr
#     def bp(self,X,y,lr):
#         yh=self.fp(X)
#         g=yh-y
#         #print(g)
#         dummy=np.ones(yh.shape[0])
#         for i in reversed(range(len(self.W))):
#             deriv_af=self.daf(self.Z[i+1])
#             g=np.multiply(deriv_af,g)
#             dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
#             #print(dw[0])
#             #db=np.matmul(dummy,g)/yh.shape[0]
#             db=np.sum(g,axis=0)/yh.shape[0]
            
#             g=np.dot(g,self.W[i].T)
#             self.W[i]-=dw*lr
#             self.b[i]-=db*lr
    def bp_mom(self,X,y,lr,wt,bt,gamma=.9):
        yh=self.fp(X)
        g=yh-y
        #print(g)
        #print(gamma)
        dummy=np.ones(yh.shape[0])
        
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            if(i<len(self.W)-1 or self.l==1):
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
            g=np.dot(g,self.W[i].T)
            #print(gamma*wt[i])
            change=gamma*wt[i]+dw*lr
            self.W[i]-=change
            wt[i]=change
            #print(wt[i])
            change=gamma*bt[i]+db*lr
            self.b[i]-=change
            bt[i]=change
            
    
    def bp_rmsprop(self,X,y,lr,wt,bt,gamma=.9):
        
        yh=self.fp(X)
        g=yh-y
        
        #print(g)
        #print(gamma)
        dummy=np.ones(yh.shape[0])
        e=1e-8
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            
            if(i<len(self.W)-1 or self.l==1):
                
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
            g=np.dot(g,self.W[i].T)
            #print(gamma*wt[i])
            temp=gamma*wt[i]+0.1*(dw**2)
            self.W[i]-=lr*dw/(np.sqrt(temp+e))
            wt[i]=temp
            #print(wt[i])
            temp=gamma*bt[i]+.1*(db**2)
            self.b[i]-=lr*db/(np.sqrt(temp+e))
            bt[i]=temp
    def bp_adam(self,X,y,lr,vwt,vbt,mwt,mbt,b1=.9,b2=.999):
        yh=self.fp(X)
        g=yh-y
        #print(g)
        #print(gamma)
        dummy=np.ones(yh.shape[0])
        e=1e-8
        b1t=1
        b2t=1
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            
            if(i<len(self.W)-1 or self.l==1):
                
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
            g=np.dot(g,self.W[i].T)
            #print(gamma*wt[i])
            vwt[i]=b2*vwt[i]+(1-b2)*(dw**2)
            vbt[i]=b2*vbt[i]+(1-b2)*(db**2)
            mwt[i]=b1*mwt[i]+(1-b1)*(dw)
            mbt[i]=b1*mbt[i]+(1-b1)*(db)
            #print(mbt[i])
            b1t*=b1
            b2t*=b2
            self.W[i]-=lr*(mwt[i]/(1-b1t))/(np.sqrt(vwt[i]/(1-b2t)+e))
            self.b[i]-=lr*(mbt[i]/(1-b1t))/(np.sqrt(vbt[i]/(1-b2t)+e))
            
            
    def bp_nadam(self,X,y,lr,vwt,vbt,mwt,mbt,b1=.9,b2=.999):
        yh=self.fp(X)
        g=yh-y
        #print(g)
        #print(gamma)
        dummy=np.ones(yh.shape[0])
        e=1e-8
        b1t=1
        b2t=1
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            
            if(i<len(self.W)-1 or self.l==1):
                
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
            g=np.dot(g,self.W[i].T)
            #print(gamma*wt[i])
            vwt[i]=b2*vwt[i]+(1-b2)*(dw**2)
            vbt[i]=b2*vbt[i]+(1-b2)*(db**2)
            mwt[i]=b1*mwt[i]+(1-b1)*(dw)
            mbt[i]=b1*mbt[i]+(1-b1)*(db)
            #print(mbt[i])
            b1t*=b1
            b2t*=b2
            self.W[i]-=lr*(b1*mwt[i]/(1-b1t)+(1-b1)*dw/(1-b1t))/(np.sqrt(vwt[i]/(1-b2t)+e))
            self.b[i]-=lr*(b1*mbt[i]/(1-b1t)+(1-b1)*db/(1-b1t))/(np.sqrt(vbt[i]/(1-b2t)+e))
    
    def bp_nag(self,X,y,lr,wt,bt,gamma=.9):
        W_temp=self.W
        b_temp=self.b
        for i in range(len(self.W)):
            self.W[i]-=gamma*wt[i]
            self.b[i]-=gamma*bt[i]
            
        yh=self.fp(X)
        g=yh-y
        
        #print(g)
        #print(gamma)
        
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            
            if(i<len(self.W)-1 or self.l==1):
                
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
               
            g=np.dot(g,self.W[i].T)
           
            temp=gamma*wt[i]+lr*dw
            
            W_temp[i]-=temp
            wt[i]=temp
            #print(wt[i])
            temp=gamma*bt[i]+lr*db
            b_temp[i]-=temp
            bt[i]=temp
        self.W=W_temp
        self.b=b_temp
    
    def fit(self,bs,epochs,X,y,lr,adaptive=False):
        for i in range(len(self.W)):
            self.W[i]=np.float64(self.W[i])
            self.b[i]=np.float64(self.b[i])
        l=[]    
        for i in range(epochs):
            if(adaptive):
                lr/=(i+1)**.5
            for j in range(X.shape[0]//bs):
#                 if(j==5 and i==0):
#                     for k in range(len(a.W)):
#                         temp=np.concatenate((a.b[k].reshape(1,-1),a.W[k]),axis=0)
#                         np.save('essentials/part_a_and_b/multiclass_dataset/tc_3/w_'+str(k+1)+'_iter.npy',temp)
#                         print(np.max(np.load('essentials/part_a_and_b/multiclass_dataset/tc_3/ac_w_'+str(k+1)+'_iter.npy')-temp))
                    
                self.bp(X[j*bs:(j+1)*bs,:],y[j*bs:(j+1)*bs,:],lr)
            y_pred=self.fp(X_test)
            l.append((self.loss(y,self.fp(X)),acc(y,self.fp(X))))
            print(i,l[-1])
            print(i,self.loss(y_test,y_pred),acc(y_test,y_pred))
        return l    
    def fit_mom(self,bs,epochs,X,y,lr,adaptive=False):
        l=[]
        wt=[0]*len(self.W)
        bt=[0]*len(self.W)
        for i in range(epochs):
            if(adaptive):
                lr/=(i+1)**.5
            for j in range(X.shape[0]//bs):
                self.bp_mom(X[j*bs:(j+1)*bs,:],y[j*bs:(j+1)*bs,:],lr,wt,bt)
            y_pred=self.fp(X_test)
            l.append((self.loss(y,self.fp(X)),acc(y,self.fp(X))))
            print(i,l[-1])
            print(i,self.loss(y_test,y_pred),acc(y_test,y_pred))
        return l
    def fit_nag(self,bs,epochs,X,y,lr,adaptive=False):
        l=[]
        wt=[0]*len(self.W)
        bt=[0]*len(self.W)
        for i in range(epochs):
            if(adaptive):
                lr/=(i+1)**.5
            for j in range(X.shape[0]//bs):
                self.bp_nag(X[j*bs:(j+1)*bs,:],y[j*bs:(j+1)*bs,:],lr,wt,bt)
            y_pred=self.fp(X_test)
            l.append((self.loss(y,self.fp(X)),acc(y,self.fp(X))))
            print(i,l[-1])
            print(i,self.loss(y_test,y_pred),acc(y_test,y_pred))
        return l
    def fit_adam(self,bs,epochs,X,y,lr,adaptive=False):
        mwt=[0]*len(self.W)
        mbt=[0]*len(self.W)
        vwt=[0]*len(self.W)
        vbt=[0]*len(self.W)
        l=[]
        for i in range(epochs):
            if(adaptive):
                lr/=(i+1)**.5
            for j in range(X.shape[0]//bs):
                self.bp_adam(X[j*bs:(j+1)*bs,:],y[j*bs:(j+1)*bs,:],lr,vwt,vbt,mwt,mbt)
            y_pred=self.fp(X_test)
            l.append((self.loss(y,self.fp(X)),acc(y,self.fp(X))))
            print(i,l[-1])
            print(i,self.loss(y_test,y_pred),acc(y_test,y_pred))    
        return l
    def fit_nadam(self,bs,epochs,X,y,lr,adaptive=False):
        mwt=[0]*len(self.W)
        mbt=[0]*len(self.W)
        vwt=[0]*len(self.W)
        vbt=[0]*len(self.W)
        l=[]
        for i in range(epochs):
            if(adaptive):
                lr/=(i+1)**.5
            for j in range(X.shape[0]//bs):
                self.bp_nadam(X[j*bs:(j+1)*bs,:],y[j*bs:(j+1)*bs,:],lr,vwt,vbt,mwt,mbt)
            y_pred=self.fp(X_test)
            l.append((self.loss(y,self.fp(X)),acc(y,self.fp(X))))
            print(i,l[-1])
            print(i,self.loss(y_test,y_pred),acc(y_test,y_pred))    
        return l
    def fit_rmsprop(self,bs,epochs,X,y,lr,adaptive=False):
        wt=[0]*len(self.W)
        bt=[0]*len(self.W)
        l=[]
        for i in range(epochs):
            if(adaptive):
                lr/=(i+1)**.5
            for j in range(X.shape[0]//bs):
                self.bp_rmsprop(X[j*bs:(j+1)*bs,:],y[j*bs:(j+1)*bs,:],lr,wt,bt)
            y_pred=self.fp(X_test)
            l.append((self.loss(y,self.fp(X)),acc(y,self.fp(X))))
            print(i,l[-1])
            print(i,self.loss(y_test,y_pred),acc(y_test,y_pred))             
        return l    
    def pred(self,X):
        return self.fp(X)
        

# Gradient Decent

In [ ]:
#figuring out batch size
plt.clf()
best_metrics=[]
times=[]
for bs in [5,10,25,50,100,150,500,1000]:
    t=time()
    a=ANN([256,46],2,1024,0)
    losses=a.fit(bs,8,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(bs))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
#plt.savefig('best_batchsize_arc1.png')   

In [ ]:
print(*times)
print(best_metrics)

In [ ]:
#figuring out batch size
plt.clf()
best_metrics=[]
times=[]
for bs in [5,10,25,50,100,150,500,1000]:
    t=time()
    a=ANN([512,256,128,64,46],2,1024,0)
    losses=a.fit(bs,8,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(bs))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
#plt.savefig('best_batchsize_arc2.png') 

In [ ]:
print(*times)
print(best_metrics)

In [ ]:
# 342.3061590194702
# [(0.06582774118697778, 0.9819948849104859)], bs=50, 20 epochs
# 246.69421291351318
# [(0.18625646011060976, 0.9508312020460358)] bs=100 epoch=20
# 436.7336814403534
# [(0.02960007737225836, 0.9919437340153453)] bs=25 epoch=15
# 298.78515434265137
# [(0.061727844479410494, 0.9822634271099744)] bs=25 epoch=12

In [ ]:
#figuring out activation function
plt.clf()
best_metrics=[]
times=[]
for af in [('sigmoid',0),('tanh',1),('relu',2)]:
    t=time()
    a=ANN([512,256,128,64,46],af[1],1024,0)
    losses=a.fit(100,5,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=af[0])
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_af_arc2.png') 

In [ ]:
print(*times)
print(best_metrics)

In [ ]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [1e-4,1e-3,1e-2,.1,1]:
    t=time()
    a=ANN([256,46],2,1024,0)
    losses=a.fit(100,10,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr_arc1.png')


In [ ]:
print(*times)
print(best_metrics)

In [ ]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [1e-4,1e-3,1e-2,.1,1]:
    t=time()
    a=ANN([512,256,128,64,46],2,1024,0)
    losses=a.fit(100,10,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr_arc2.png') 

In [ ]:
print(*times)
print(best_metrics)

In [ ]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [5e-2,.07,.1,.15,.2,.3,.4,.5,.7]:
    t=time()
    a=ANN([256,46],2,1024,0)
    losses=a.fit(100,10,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr1_arc1.png') 

In [ ]:
print(*times)
print(best_metrics)

In [ ]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [5e-2,.07,.1,.15,.2,.3,.4,.5,.7]:
    t=time()
    a=ANN([512,256,128,64,46],2,1024,0)
    losses=a.fit(100,10,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr1_arc2.png') 

In [ ]:
print(*times)
print(best_metrics)

# MOMENTUM

In [26]:
#figuring out batch size
plt.clf()
best_metrics=[]
times=[]
for bs in [5,10,25,50,100,150,500,1000]:
    t=time()
    a=ANN([256,46],2,1024,0)
    losses=a.fit_mom(bs,10,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(bs))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_batchsize_mom_arc1.png')    

0 (0.703111713095217, 0.814923273657289)
0 0.7442710984439754 0.8023913043478261
1 (0.43189943249061347, 0.8867774936061381)
1 0.5042542749871772 0.8658695652173913
2 (0.31839017085575333, 0.9173401534526854)
2 0.41316990254030916 0.8910869565217391
3 (0.2517303522078707, 0.9348209718670076)
3 0.36316266331038266 0.9010869565217391
4 (0.2083904523452574, 0.9455882352941176)
4 0.33563160081379095 0.9082608695652173
5 (0.17854783216064216, 0.9531457800511509)
5 0.31847236087659125 0.9128260869565218
6 (0.1549254728566791, 0.9593606138107417)
6 0.30641624388092975 0.9165217391304348
7 (0.1363063889962506, 0.9642966751918158)
7 0.2971213907360447 0.9191304347826087
8 (0.11986398822461922, 0.9683375959079283)
8 0.2888560498968966 0.9219565217391305
9 (0.10531913195925711, 0.9724424552429668)
9 0.28218378222924506 0.926304347826087



In [28]:
print(*times)
print(best_metrics)

98.87185025215149
[(0.10531913195925711, 0.9724424552429668)]


In [14]:
#figuring out batch size
plt.clf()
best_metrics=[]
times=[]
for bs in [5,10,25,50,100,150,500,1000]:
    t=time()
    a=ANN([512,256,128,64,46],2,1024,0)
    losses=a.fit_mom(bs,10,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(bs))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_batchsize_mom_arc2.png') 

0 (0.8307514997935778, 0.7684271099744245)
0 0.8622091040763213 0.7613043478260869
1 (0.44948504530161426, 0.8700127877237852)
1 0.5227694773535877 0.85
2 (0.3084963261162603, 0.9091943734015345)
2 0.4140709889274521 0.8817391304347826
3 (0.221591067000425, 0.9343989769820972)
3 0.3529281014905998 0.8967391304347826
4 (0.18748333279561027, 0.9416879795396419)
4 0.33552008140656175 0.9026086956521739
5 (0.14301938538831388, 0.9554092071611253)
5 0.3020154002633157 0.9132608695652173
6 (0.1468000690294204, 0.9532225063938619)
6 0.3231981299660416 0.9117391304347826
7 (0.13014833761879022, 0.9579795396419437)
7 0.3318332892423893 0.9117391304347826
8 (0.08829692459612515, 0.9714066496163682)
8 0.30071235866390833 0.9206521739130434
9 (0.06176018110688603, 0.9800895140664961)
9 0.2894690625903368 0.9267391304347826

0 (1.1377225403825943, 0.6876854219948849)
0 1.1509441120397232 0.6867391304347826
1 (0.7016512398310195, 0.8007800511508951)
1 0.7395311727457375 0.7917391304347826
2 (0.46105

In [17]:
print(*times)
print(best_metrics)

99.93885087966919 63.98290205001831
[(0.10682739068660597, 0.9720076726342711), (0.1877940438856365, 0.9532736572890026)]


In [46]:
#figuring out activation function
plt.clf()
best_metrics=[]
times=[]
for af in [('sigmoid',0),('tanh',1),('relu',2)]:
    t=time()
    a=ANN([512,256,128,64,46],af[1],1024,0)
    losses=a.fit_mom(100,5,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=af[0])
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_af_mom_arc2.png') 


0 (3.831245816409815, 0.021739130434782608)
0 3.8312787282985705 0.021739130434782608
1 (3.828992319477445, 0.021739130434782608)
1 3.829047224861386 0.021739130434782608
2 (3.8261239112810244, 0.026956521739130435)
2 3.8262153339056035 0.025652173913043478
3 (3.8210587699604637, 0.04352941176470588)
3 3.821235688492051 0.044130434782608696
4 (3.804293467257725, 0.06584398976982098)
4 3.80484614000601 0.0632608695652174

0 (0.8871761844149905, 0.7749488491048593)
0 0.9142638873342163 0.7676086956521739
1 (0.4998826578644193, 0.8726982097186701)
1 0.5523786177811885 0.8489130434782609
2 (0.3343413025685625, 0.9153196930946291)
2 0.40648082720038625 0.8897826086956522
3 (0.2506279252907179, 0.9354475703324808)
3 0.34134543783481985 0.9054347826086957
4 (0.188875560277995, 0.9521739130434783)
4 0.2960587210244129 0.9193478260869565

0 (0.6257073135126576, 0.821611253196931)
0 0.6739249944898758 0.8184782608695652
1 (0.32055093053183054, 0.9068925831202046)
1 0.4135088247374831 0.881956521

In [47]:
print(*times)
print(best_metrics)

93.27585625648499 97.29585123062134 81.47588014602661
[(3.804293467257725, 0.06584398976982098), (0.188875560277995, 0.9521739130434783), (0.15479302837368872, 0.9504347826086956)]


In [48]:
#figuring out activation function
plt.clf()
best_metrics=[]
times=[]
for af in [('sigmoid',0),('tanh',1),('relu',2)]:
    t=time()
    a=ANN([256,46],af[1],1024,0)
    losses=a.fit_mom(100,5,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=af[0])
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_af_mom_arc1.png') 


0 (1.6636151385733948, 0.6163938618925832)
0 1.6554553400596457 0.6156521739130435
1 (1.2960257507151987, 0.6750895140664962)
1 1.2921549612925112 0.6797826086956522
2 (1.155399232808624, 0.7042966751918158)
2 1.1588814378509151 0.7067391304347826
3 (1.0682304111942726, 0.7241687979539642)
3 1.0782501982775443 0.7247826086956521
4 (1.002247871743516, 0.7388874680306905)
4 1.017674327437366 0.7354347826086957

0 (1.0265693191250564, 0.7331457800511509)
0 1.040827035338164 0.7376086956521739
1 (0.8267634120179892, 0.7845140664961637)
1 0.8600905946613566 0.7769565217391304
2 (0.6800460286535398, 0.8260102301790281)
2 0.7265659268095965 0.8067391304347826
3 (0.5671470244491539, 0.858158567774936)
3 0.6247760770972628 0.8360869565217391
4 (0.48178846645491036, 0.8809590792838875)
4 0.5492666956608298 0.8578260869565217

0 (0.9741126051662078, 0.7401278772378517)
0 0.9992920098274485 0.7334782608695652
1 (0.706394473861463, 0.8133759590792838)
1 0.7515622230925292 0.7965217391304348
2 (0.53

In [49]:
print(*times)
print(best_metrics)

37.17193961143494 37.695943117141724 32.671950340270996
[(1.002247871743516, 0.7388874680306905), (0.48178846645491036, 0.8809590792838875), (0.36048668554300783, 0.9080690537084399)]


In [59]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [1e-4,1e-3,1e-2,.1,1]:
    t=time()
    a=ANN([256,46],2,1024,0)
    losses=a.fit_mom(100,7,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr_mom_arc1.png') 

In [52]:
print(*times)
print(best_metrics)

48.479918479919434 45.92392826080322 48.64792776107788 44.95193099975586 45.57193040847778
[(2.3744167174807624, 0.49404092071611255), (1.03576554771077, 0.7315601023017902), (0.2643730371360236, 0.9328132992327366), (0.22836529371940992, 0.9373145780051151), (3.8555128499864417, 0.02176470588235294)]


In [60]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [1e-4,1e-3,1e-2,.1,1]:
    t=time()
    a=ANN([512,256,128,64,46],2,1024,0)
    losses=a.fit_mom(100,7,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr_mom_arc2.png') 

0 (3.8096758343887496, 0.03524296675191816)
0 3.808893876854413 0.030434782608695653
1 (3.7646437561615063, 0.05516624040920716)
1 3.764039586461608 0.050869565217391305
2 (3.7021612343071397, 0.08588235294117647)
2 3.7015211307311295 0.08369565217391305
3 (3.606335123064112, 0.1373017902813299)
3 3.6058243791955427 0.1323913043478261
4 (3.463723377927448, 0.1889002557544757)
4 3.463517053305036 0.19195652173913044
5 (3.254226672015963, 0.24881074168797954)
5 3.2543756858320827 0.24717391304347827
6 (2.9553359575704756, 0.31516624040920715)
6 2.9551629496588863 0.3154347826086957

0 (2.2733449512447423, 0.42164961636828646)
0 2.256400277036458 0.43021739130434783
1 (1.4088435736021219, 0.6244757033248082)
1 1.410726562221631 0.6204347826086957
2 (1.153913620372559, 0.6909718670076727)
2 1.1647175278813475 0.683695652173913
3 (0.9861938306070243, 0.7341304347826086)
3 1.0050860980378191 0.7260869565217392
4 (0.8475872359230491, 0.7698721227621483)
4 0.8732815023526872 0.7621739130434783

In [61]:
print(*times)
print(best_metrics)

110.47183132171631 113.0998318195343 114.50782465934753 115.9518256187439 115.52459669113159
[(2.9553359575704756, 0.31516624040920715), (0.635874491747902, 0.825997442455243), (0.1076983477087977, 0.9641432225063938), (0.3710570570968945, 0.9124424552429667), (3.8556164238344683, 0.021739130434782608)]


In [62]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [7e-3,1e-2,2e-2,4e-2]:
    t=time()
    a=ANN([512,256,128,64,46],2,1024,0)
    losses=a.fit_mom(100,7,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr1_mom_arc2.png') 

0 (0.7859398704447106, 0.7834910485933504)
0 0.8276567622580836 0.7676086956521739
1 (0.4090405198688935, 0.8840664961636828)
1 0.4730769484516586 0.8673913043478261
2 (0.2913138539947819, 0.9148081841432225)
2 0.3904651088569843 0.8904347826086957
3 (0.22271064750153782, 0.9339386189258312)
3 0.35484310334184854 0.8982608695652174
4 (0.18073447750911828, 0.9456393861892584)
4 0.33942114091680153 0.9026086956521739
5 (0.13580919926100596, 0.9577749360613811)
5 0.3123901879904835 0.9141304347826087
6 (0.1231882703859728, 0.960613810741688)
6 0.325827248272183 0.9128260869565218

0 (0.5912126266593093, 0.8340920716112532)
0 0.6415927483840842 0.8184782608695652
1 (0.3409473683181891, 0.8990920716112532)
1 0.4379916898024736 0.8719565217391304
2 (0.24617118810345112, 0.9257416879795396)
2 0.3758591840800076 0.8876086956521739
3 (0.1872005476663068, 0.9416496163682865)
3 0.33738753982938047 0.9034782608695652
4 (0.15677898844440546, 0.9495268542199489)
4 0.3269624287746522 0.90934782608695

In [63]:
print(*times)
print(best_metrics)

119.78380370140076 112.13856887817383 114.64382719993591 108.46434998512268
[(0.1231882703859728, 0.960613810741688), (0.09061251715379383, 0.9702685421994884), (0.13023704710655043, 0.9593478260869566), (0.1492914807807535, 0.9554219948849105)]


In [58]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [1e-2,3e-2,7e-2,.1,.11,.12,.14]:
    t=time()
    a=ANN([256,46],2,1024,0)
    losses=a.fit_mom(100,7,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr1_mom_arc1.png') 

0 (0.9669095909496254, 0.7422762148337596)
0 0.9885938079038199 0.7356521739130435
1 (0.6937300802134747, 0.8177109974424552)
1 0.7376885178685075 0.8069565217391305
2 (0.526582390377922, 0.8637468030690537)
2 0.5870958073618877 0.8460869565217392
3 (0.42369763853322207, 0.8916624040920716)
3 0.496302860473202 0.8665217391304347
4 (0.3558671250109262, 0.9094757033248082)
4 0.43934928565270154 0.8806521739130435
5 (0.30682888818721243, 0.9220460358056266)
5 0.39943499425715107 0.893695652173913
6 (0.2688946794990988, 0.9318670076726343)
6 0.3697446057205439 0.9019565217391304
7 (0.2381316884207255, 0.9396291560102302)
7 0.34739916108271707 0.9089130434782609
8 (0.21291928196090704, 0.9462915601023018)
8 0.3297783791218987 0.9130434782608695
9 (0.1920087048921828, 0.9518797953964194)
9 0.3160817359857038 0.9139130434782609

0 (0.5663846651600581, 0.8460613810741688)
0 0.625387153391489 0.8293478260869566
1 (0.3362775204207301, 0.9091687979539642)
1 0.42349173000181456 0.8858695652173914


In [57]:
print(*times)
print(best_metrics)

44.28393745422363 43.81192088127136 43.95993399620056 44.135934352874756 43.867934703826904 43.73193359375 43.47993731498718
[(0.2623428209878484, 0.9341304347826087), (0.12353589842539049, 0.9644373401534527), (0.11176657990835927, 0.9641943734015346), (0.21685401387251252, 0.9387340153452686), (0.22174802779651323, 0.9379539641943734), (0.253777272799641, 0.929923273657289), (0.4002061421680739, 0.9067391304347826)]


# NAG

In [70]:
#figuring out batch size
plt.clf()
best_metrics=[]
times=[]
for bs in [25,50,100,150,500,1000]:
    t=time()
    a=ANN([256,46],2,1024,0)
    losses=a.fit_nag(bs,10,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(bs))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_batchsize_nag_arc1.png')  

0 (0.8892528665544023, 0.7632480818414322)
0 0.9127648638652258 0.7545652173913043
1 (0.6018024515761946, 0.8432225063938619)
1 0.6463649964130466 0.8302173913043478
2 (0.4475256501831831, 0.8853452685421995)
2 0.5097464284064797 0.8660869565217392
3 (0.3566547508732969, 0.9094373401534527)
3 0.43518191516017746 0.8867391304347826
4 (0.2980284719299274, 0.9239258312020461)
4 0.39103608132730094 0.8971739130434783
5 (0.2556010075123842, 0.9345524296675192)
5 0.36105047525094425 0.9030434782608696
6 (0.22211388221328388, 0.9434398976982097)
6 0.3371896810344859 0.9067391304347826
7 (0.1953260262148944, 0.9509590792838875)
7 0.31906568478926994 0.9104347826086957
8 (0.17351929809688593, 0.957084398976982)
8 0.30554238509103815 0.9143478260869565
9 (0.15455477628335337, 0.9624424552429668)
9 0.2948053430656257 0.9184782608695652
10 (0.1387344431207014, 0.9668925831202047)
10 0.28639152723489786 0.9202173913043479
11 (0.12482192146471359, 0.9708312020460358)
11 0.2793571677097383 0.92108695

In [68]:
print(*times)
print(best_metrics)

205.81445813179016 118.77981734275818 72.29989051818848 59.13191294670105 37.940460205078125 34.49594569206238
[(0.09118178796073186, 0.9714961636828644), (0.0572566450892252, 0.9837979539641943), (0.11644426449839615, 0.9687851662404092), (0.1530439684552261, 0.9627493606138108), (0.4485570167023755, 0.886994884910486), (0.739027379982765, 0.8070204603580563)]


In [72]:
#figuring out batch size
plt.clf()
best_metrics=[]
times=[]
for bs in [25,50,100,150,500,1000]:
    t=time()
    a=ANN([512,256,128,64,46],2,1024,0)
    losses=a.fit_nag(bs,10,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(bs))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_batchsize_nag_arc2.png') 

0 (0.5402990262403794, 0.838618925831202)
0 0.5892725208662442 0.8297826086956521
1 (0.36519343856001074, 0.8941687979539642)
1 0.4751472642762232 0.8702173913043478
2 (0.3308998350095716, 0.9097570332480819)
2 0.4671499621611311 0.8832608695652174
3 (0.23150551012572762, 0.9332097186700767)
3 0.37140104570307697 0.9015217391304348
4 (0.2181437470752625, 0.9369820971867008)
4 0.3511570428919252 0.905
5 (0.23938742033455676, 0.9340281329923273)
5 0.41444225007032093 0.8963043478260869
6 (0.16866843183543362, 0.9527365728900256)
6 0.35154073536606567 0.918695652173913
7 (0.1823785558495994, 0.9483887468030691)
7 0.36896377705543826 0.9123913043478261
8 (0.19437695646770595, 0.9477621483375959)
8 0.42239052586249815 0.9032608695652173
9 (0.11869241961292115, 0.9659718670076727)
9 0.3341694586688771 0.923695652173913

0 (0.44193662158304553, 0.8660358056265984)
0 0.540248575146037 0.8441304347826087
1 (0.26768234964572973, 0.9187851662404092)
1 0.40191772586535746 0.8860869565217391
2 (0.2

In [73]:
print(*times)
print(best_metrics)

538.4816226959229 296.74824500083923 177.28170561790466 138.11179089546204 88.92386865615845 77.87987899780273
[(0.11869241961292115, 0.9659718670076727), (0.07931926784697643, 0.9751790281329923), (0.08237930956727905, 0.9723273657289002), (0.07107518448254102, 0.975997442455243), (0.1300982400828952, 0.9605626598465473), (0.2868496535411937, 0.9166240409207161)]


In [74]:
#figuring out batch size
plt.clf()
best_metrics=[]
times=[]
for bs in [150,250,350,400]:
    t=time()
    a=ANN([512,256,128,64,46],2,1024,0)
    losses=a.fit_nag(bs,10,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(bs))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_batchsize1_nag_arc2.png') 

0 (0.5409029592342997, 0.8419948849104859)
0 0.5887375133672926 0.8269565217391305
1 (0.33149603831303576, 0.8986445012787724)
1 0.437382079535038 0.871304347826087
2 (0.20829739886179247, 0.9368414322250639)
2 0.3282696538512705 0.9041304347826087
3 (0.1809501462233266, 0.9432992327365729)
3 0.33489710803530415 0.9039130434782608
4 (0.14239853596963223, 0.9544245524296675)
4 0.31439163529673697 0.9128260869565218
5 (0.11601867581768495, 0.9626854219948849)
5 0.30813750078626295 0.9191304347826087
6 (0.08778814412833517, 0.9713682864450128)
6 0.2918371028931767 0.9221739130434783
7 (0.07310819040872599, 0.9757800511508952)
7 0.3165105121960034 0.9197826086956522
8 (0.061732141167816286, 0.9787084398976982)
8 0.31172797807374164 0.9278260869565217
9 (0.08708251333463617, 0.9715856777493607)
9 0.34887629549139915 0.9189130434782609

0 (0.8489255980289327, 0.7598081841432225)
0 0.8673329173197574 0.7571739130434783
1 (0.4234094871942923, 0.8769437340153453)
1 0.49799556864334005 0.8606521

In [75]:
print(*times)
print(best_metrics)

138.89579010009766 111.02382969856262 101.3718478679657 102.66088891029358
[(0.08708251333463617, 0.9715856777493607), (0.06725019268537831, 0.9779923273657289), (0.07212199615010918, 0.9786061381074169), (0.10920724360782022, 0.9659207161125319)]


In [76]:
#figuring out activation function
plt.clf()
best_metrics=[]
times=[]
for af in [('sigmoid',0),('tanh',1),('relu',2)]:
    t=time()
    a=ANN([512,256,128,64,46],af[1],1024,0)
    losses=a.fit_nag(250,5,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=af[0])
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_af_nag_arc2.png') 

0 (3.8306184440270785, 0.0220460358056266)
0 3.8306514592178105 0.021739130434782608
1 (3.8290706627783657, 0.02176470588235294)
1 3.8290968725808243 0.021739130434782608
2 (3.8272036348870926, 0.021918158567774935)
2 3.827222483772888 0.021739130434782608
3 (3.824557821175228, 0.023734015345268544)
3 3.824566724686806 0.02391304347826087
4 (3.819706397508451, 0.03014066496163683)
4 3.819696128246374 0.030869565217391304

0 (1.0432134549721797, 0.7348721227621483)
0 1.0559211253961345 0.7276086956521739
1 (0.6223455006971291, 0.8420588235294117)
1 0.6596419764252669 0.8304347826086956
2 (0.41129964141908637, 0.8966624040920717)
2 0.47109204019253964 0.8741304347826087
3 (0.29547845361215963, 0.9262020460358056)
3 0.37737828813123536 0.9004347826086957
4 (0.22806274225173115, 0.9432225063938618)
4 0.3299876729514856 0.9130434782608695

0 (0.7898920475251596, 0.7772506393861892)
0 0.820395410279014 0.7721739130434783
1 (0.40785915508688153, 0.8824040920716113)
1 0.4816494515387814 0.8636

In [77]:
print(*times)
print(best_metrics)

67.71188044548035 70.50542831420898 53.83092474937439
[(3.819706397508451, 0.03014066496163683), (0.22806274225173115, 0.9432225063938618), (0.19082921759357815, 0.9398081841432225)]


In [78]:
#figuring out activation function
plt.clf()
best_metrics=[]
times=[]
for af in [('sigmoid',0),('tanh',1),('relu',2)]:
    t=time()
    a=ANN([256,46],af[1],1024,0)
    losses=a.fit_nag(150,5,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=af[0])
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_af_nag_arc1.png') 

0 (1.5116203921522668, 0.6364833759590793)
0 1.5070115026050879 0.6363043478260869
1 (1.2111541327238933, 0.6907800511508951)
1 1.212044836870097 0.697391304347826
2 (1.0843158497825192, 0.7184015345268542)
2 1.0921991267117457 0.721304347826087
3 (0.9986463666760234, 0.7388746803069054)
3 1.0121919805329376 0.736304347826087
4 (0.9293019615798321, 0.7556905370843989)
4 0.947620612851895 0.7513043478260869

0 (0.9818770155188488, 0.743158567774936)
0 1.002534684354445 0.7386956521739131
1 (0.7631598664217837, 0.8013171355498722)
1 0.8081299514083614 0.7860869565217391
2 (0.605710248764647, 0.845306905370844)
2 0.6678781555513154 0.8247826086956521
3 (0.49487307287942056, 0.8764450127877238)
3 0.5700801302778675 0.8504347826086956
4 (0.4153934017507939, 0.8976598465473146)
4 0.5016873446785907 0.8691304347826087

0 (0.8901885539223628, 0.7617263427109975)
0 0.9156717245314214 0.7539130434782608
1 (0.5975909372033003, 0.8452685421994884)
1 0.651460166732781 0.831304347826087
2 (0.4387940

In [79]:
print(*times)
print(best_metrics)            

29.81995177268982 32.041951417922974 28.169959545135498
[(0.9293019615798321, 0.7556905370843989), (0.4153934017507939, 0.8976598465473146), (0.2908013762424991, 0.9264066496163683)]


In [80]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [1e-4,1e-3,1e-2,.1,1]:
    t=time()
    a=ANN([256,46],2,1024,0)
    losses=a.fit_nag(100,7,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr_nag_arc1.png') 

0 (3.4206826446822833, 0.1932992327365729)
0 3.426116250063099 0.19260869565217392
1 (2.9574556960117, 0.3560613810741688)
1 2.960214428702895 0.35630434782608694
2 (2.5517324252683355, 0.4507416879795396)
2 2.5521555175388206 0.4591304347826087
3 (2.240604625577449, 0.5089641943734016)
3 2.239733692702475 0.5089130434782608
4 (2.013619929190276, 0.5474296675191815)
4 2.0122070440456516 0.547608695652174
5 (1.847781584557551, 0.5745780051150895)
5 1.846312787683177 0.571304347826087
6 (1.7241177500151572, 0.5944245524296675)
6 1.7227726983719058 0.592391304347826

0 (1.4970284975619015, 0.6325703324808184)
0 1.4953631663329523 0.6341304347826087
1 (1.209382398825134, 0.6902173913043478)
1 1.2129260004211078 0.6895652173913044
2 (1.0922226085349407, 0.717391304347826)
2 1.1025961398681312 0.715
3 (1.0141132945810227, 0.7360485933503836)
3 1.0310711493655007 0.7341304347826086
4 (0.9504430155558317, 0.7521099744245524)
4 0.9729712553728607 0.7484782608695653
5 (0.8939609303855782, 0.7666

In [81]:
print(*times)
print(best_metrics)   

49.5749249458313 47.66492772102356 47.602927923202515 47.75493025779724 47.11392593383789
[(1.7241177500151572, 0.5944245524296675), (0.8419139520301482, 0.781074168797954), (0.16510895494649372, 0.9558567774936061), (0.6520767571130279, 0.8480434782608696), (3.8842744062667536, 0.021739130434782608)]


In [82]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [1e-4,1e-3,1e-2,.1,1]:
    t=time()
    a=ANN([512,256,128,64,46],2,1024,0)
    losses=a.fit_nag(100,7,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr_nag_arc2.png') 

0 (3.7475973856340654, 0.07237851662404092)
0 3.749028552885513 0.07326086956521739
1 (3.603754695594871, 0.1298849104859335)
1 3.606523684496087 0.12760869565217392
2 (3.337107723799012, 0.20717391304347826)
2 3.341058915561855 0.19956521739130434
3 (2.8592244174452013, 0.32864450127877237)
3 2.863365888810268 0.3241304347826087
4 (2.303240132168837, 0.4364450127877238)
4 2.3064188757412 0.435
5 (1.9285531762793071, 0.509539641943734)
5 1.9305610539932498 0.5097826086956522
6 (1.7020411017078092, 0.5593734015345269)
6 1.7047721152052835 0.5591304347826087

0 (1.4864652648398116, 0.594616368286445)
0 1.5009072015296687 0.5904347826086956
1 (1.0474118790893878, 0.7156265984654732)
1 1.073381075235962 0.7126086956521739
2 (0.7925746956527572, 0.7842838874680307)
2 0.8271159461275879 0.7795652173913044
3 (0.605791480848001, 0.8357289002557545)
3 0.6515617744377443 0.8178260869565217
4 (0.4884267816504994, 0.8676086956521739)
4 0.5493060646055815 0.8476086956521739
5 (0.40761375235329345, 

<ipython-input-66-f3ff4b4cd6ba>:233: RuntimeWarning: overflow encountered in subtract
  W_temp[i]-=temp
<ipython-input-66-f3ff4b4cd6ba>:237: RuntimeWarning: overflow encountered in subtract
  b_temp[i]-=temp
<ipython-input-66-f3ff4b4cd6ba>:209: RuntimeWarning: overflow encountered in subtract
  self.W[i]-=gamma*wt[i]
<ipython-input-66-f3ff4b4cd6ba>:42: RuntimeWarning: invalid value encountered in matmul
  self.Z.append(self.af(np.matmul(self.Z[i],self.W[i])+self.b[i]))
<ipython-input-66-f3ff4b4cd6ba>:46: RuntimeWarning: invalid value encountered in matmul
  self.Z.append(np.matmul(self.Z[-1],self.W[-1])+self.b[-1])


0 (34.53877639491091, 0.021739130434782608)
0 34.538776394910684 0.021739130434782608
1 (34.53877639491091, 0.021739130434782608)
1 34.538776394910684 0.021739130434782608
2 (34.53877639491091, 0.021739130434782608)
2 34.538776394910684 0.021739130434782608
3 (34.53877639491091, 0.021739130434782608)
3 34.538776394910684 0.021739130434782608
4 (34.53877639491091, 0.021739130434782608)
4 34.538776394910684 0.021739130434782608
5 (34.53877639491091, 0.021739130434782608)
5 34.538776394910684 0.021739130434782608
6 (34.53877639491091, 0.021739130434782608)
6 34.538776394910684 0.021739130434782608



In [83]:
print(*times)
print(best_metrics)   

121.68681693077087 124.17227458953857 130.03273057937622 124.95044612884521 130.4781723022461
[(1.7020411017078092, 0.5593734015345269), (0.34577532420290585, 0.9051278772378517), (0.08292424184450631, 0.9724296675191816), (3.8332382831047953, 0.021739130434782608), (34.53877639491091, 0.021739130434782608)]


In [84]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [6e-3,8e-3,1e-2,2e-2,4e-2]:
    t=time()
    a=ANN([256,46],2,1024,0)
    losses=a.fit_nag(150,7,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr_nag_arc1.png') 

0 (1.0509985193167064, 0.7208823529411764)
0 1.062355943670425 0.7154347826086956
1 (0.8116086824281655, 0.7876598465473146)
1 0.8458439770915799 0.7782608695652173
2 (0.6393836705223986, 0.8357289002557545)
2 0.6875008362522647 0.8239130434782609
3 (0.5203962286967911, 0.8670460358056266)
3 0.5801233058855958 0.8558695652173913
4 (0.4374273683870985, 0.889616368286445)
4 0.5080068999332066 0.8730434782608696
5 (0.3767021906616101, 0.9054987212276214)
5 0.4573074347265866 0.8821739130434783
6 (0.33060368492113046, 0.9171994884910486)
6 0.41988176519986486 0.8895652173913043

0 (0.9667410645283614, 0.7433248081841433)
0 0.9777547463734414 0.7369565217391304
1 (0.7038052003367068, 0.8150255754475704)
1 0.7359138013601052 0.8089130434782609
2 (0.5352042184945077, 0.8618286445012787)
2 0.5875803959787151 0.8452173913043478
3 (0.42828027777081457, 0.8906777493606138)
3 0.4966356102207889 0.8706521739130435
4 (0.35743285324235763, 0.9090792838874681)
4 0.43808887255645096 0.8834782608695653


In [85]:
print(*times)
print(best_metrics)  

43.22088861465454 43.29163694381714 42.625967025756836 43.1541063785553 41.59028196334839
[(0.33060368492113046, 0.9171994884910486), (0.26741029406286554, 0.9325063938618926), (0.2190328473302691, 0.9438235294117647), (0.13464646897733734, 0.9620204603580562), (0.11747498754473083, 0.9620076726342711)]


In [86]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [6e-3,8e-3,1e-2,2e-2,4e-2]:
    t=time()
    a=ANN([512,256,128,64,46],2,1024,0)
    losses=a.fit_nag(250,7,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr_nag_arc2.png') 

0 (1.0308708216326743, 0.7127365728900256)
0 1.0534354437020508 0.705
1 (0.5899607712132241, 0.8342710997442455)
1 0.6364674300635028 0.8217391304347826
2 (0.39930945507511184, 0.885306905370844)
2 0.4759237145987761 0.8678260869565217
3 (0.3429283559867467, 0.8951790281329923)
3 0.4544249792186756 0.8708695652173913
4 (0.24469796275189326, 0.92730179028133)
4 0.3752943022093357 0.8939130434782608
5 (0.18785984826308785, 0.9454475703324808)
5 0.34144852979951335 0.9052173913043479
6 (0.15748084801227463, 0.9534398976982097)
6 0.3254742905523667 0.9080434782608696

0 (0.8745885396051553, 0.7536445012787724)
0 0.8970759469805039 0.7484782608695653
1 (0.4640246735555552, 0.8668925831202046)
1 0.5295665618396951 0.8523913043478261
2 (0.30975482972198065, 0.9093734015345268)
2 0.40624361406107806 0.8815217391304347
3 (0.25307649423155193, 0.9237340153452686)
3 0.37040124509034533 0.8910869565217391
4 (0.21769493403929824, 0.9325319693094629)
4 0.35676122377812564 0.8973913043478261
5 (0.155

In [87]:
print(*times)
print(best_metrics)  

83.89712953567505 82.07650947570801 79.93774175643921 82.12377691268921 86.41462206840515
[(0.15748084801227463, 0.9534398976982097), (0.13374956732816845, 0.9575319693094629), (0.10781635232842064, 0.9667007672634271), (0.11393570974326171, 0.9624040920716113), (0.10577301480100233, 0.9662659846547315)]


# NADAM

In [89]:
#figuring out batch size
plt.clf()
best_metrics=[]
times=[]
for bs in [25,50,100,150,500,1000]:
    t=time()
    a=ANN([256,46],2,1024,0)
    losses=a.fit_nadam(bs,10,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(bs))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_batchsize_nadam_arc1.png')  

0 (0.7622132503483016, 0.7792199488491048)
0 0.8268055964909327 0.7708695652173913
1 (0.633571507703032, 0.8146547314578005)
1 0.7340588678483406 0.7917391304347826
2 (0.5726640675461543, 0.8322634271099745)
2 0.7000221814032557 0.7997826086956522
3 (0.522944474757352, 0.8451278772378517)
3 0.666489718902961 0.818695652173913
4 (0.5083629792874579, 0.8505882352941176)
4 0.6662001849094671 0.8254347826086956
5 (0.488202694803359, 0.8555626598465473)
5 0.6481979533036338 0.8260869565217391
6 (0.48054414598858675, 0.8571994884910485)
6 0.6589701655096067 0.8228260869565217
7 (0.4746851093218905, 0.8591304347826086)
7 0.6770885062102837 0.8271739130434783
8 (0.4614481363744848, 0.8638107416879796)
8 0.6801832335687042 0.8245652173913044
9 (0.4472369493445251, 0.8667519181585678)
9 0.6853913760127921 0.8232608695652174

0 (0.6816233418682469, 0.8023785166240409)
0 0.753269684123649 0.7871739130434783
1 (0.5705295483422922, 0.8332736572890026)
1 0.6639637556293706 0.8097826086956522
2 (0.504

In [90]:
print(*times)
print(best_metrics)  

725.5183036327362 338.3878335952759 141.14829349517822 96.77441310882568 49.73471474647522 38.00297546386719
[(0.4472369493445251, 0.8667519181585678), (0.3339285932147802, 0.9004219948849105), (0.3143671699916029, 0.905460358056266), (0.2169580148742381, 0.9324936061381074), (0.2779619220122635, 0.9200127877237851), (0.31520356074866485, 0.9107800511508951)]


In [98]:
#figuring out batch size
plt.clf()
best_metrics=[]
times=[]
for bs in [150,250,350,450]:
    t=time()
    a=ANN([256,46],0,1024,0)
    losses=a.fit_nadam(bs,10,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(bs))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_batchsize1_nadam_arc1.png')  

0 (0.5273550304449102, 0.8642071611253197)
0 0.5917640757724169 0.8454347826086956
1 (0.36027466854307955, 0.908695652173913)
1 0.4573821986910076 0.8797826086956522
2 (0.27589117574336713, 0.9299616368286445)
2 0.39383349739848583 0.8941304347826087
3 (0.22076814356430935, 0.9444373401534527)
3 0.35529517621647927 0.9045652173913044
4 (0.18178969456371577, 0.9545140664961637)
4 0.332114823362653 0.9113043478260869
5 (0.15203208725484244, 0.9629923273657289)
5 0.31769105191450175 0.9154347826086957
6 (0.13085119859885602, 0.9682352941176471)
6 0.3099577106365717 0.915
7 (0.11420562481916606, 0.9729156010230179)
7 0.30661168119609766 0.9152173913043479
8 (0.10123324299350596, 0.9758184143222507)
8 0.30661637648936085 0.9141304347826087
9 (0.09044528230730337, 0.9779411764705882)
9 0.30942839234964187 0.9147826086956522

0 (0.5493981480486714, 0.8582992327365729)
0 0.6008122256166595 0.8417391304347827
1 (0.3751610555050468, 0.9048721227621483)
1 0.45704975383155094 0.8747826086956522
2 

In [99]:
print(*times)
print(best_metrics) 

103.5038378238678 75.35188674926758 61.50730586051941 54.28791809082031
[(0.09044528230730337, 0.9779411764705882), (0.09482290860245471, 0.9808695652173913), (0.11026750844552347, 0.9780562659846548), (0.1320817814183061, 0.972851662404092)]


In [92]:
#figuring out batch size
plt.clf()
best_metrics=[]
times=[]
for bs in [25,50,100,150,500,1000]:
    t=time()
    a=ANN([512,256,128,64,46],2,1024,0)
    losses=a.fit_nadam(bs,5,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(bs))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_batchsize_nadam_arc2.png') 

0 (0.6993207490342821, 0.794309462915601)
0 0.7766548129470706 0.7810869565217391
1 (0.5424268551698372, 0.8401790281329923)
1 0.6732173073896538 0.8132608695652174
2 (0.45462138157491117, 0.865153452685422)
2 0.6079024655068885 0.836304347826087
3 (0.3895545266318499, 0.8870204603580563)
3 0.5761518492455961 0.8456521739130435
4 (0.3679501320733689, 0.892148337595908)
4 0.5848712245794467 0.8519565217391304

0 (0.6323673405250042, 0.8130562659846547)
0 0.7093433930080766 0.7928260869565218
1 (0.43113231761816107, 0.8695268542199488)
1 0.5500403447025572 0.8465217391304348
2 (0.34998318478134705, 0.8928900255754476)
2 0.5110107813615968 0.8619565217391304
3 (0.31392780734096865, 0.9030434782608696)
3 0.4866362991746399 0.8717391304347826
4 (0.31221141281311726, 0.9049744245524297)
4 0.5157052469968687 0.8665217391304347

0 (0.711853102923568, 0.7893606138107417)
0 0.790681204687983 0.7808695652173913
1 (0.4851881107095232, 0.855383631713555)
1 0.5956620033033576 0.8304347826086956
2 (0

In [93]:
print(*times)
print(best_metrics)  

831.5702362060547 317.18752241134644 153.61977672576904 114.01582717895508 58.60391163825989 46.251932859420776
[(0.3679501320733689, 0.892148337595908), (0.31221141281311726, 0.9049744245524297), (0.29370750442773225, 0.9113171355498721), (0.2617981469251226, 0.9183631713554987), (0.3625067312547533, 0.89153452685422), (0.3009267341933239, 0.9100639386189259)]


In [100]:
#figuring out batch size
plt.clf()
best_metrics=[]
times=[]
for bs in [150,250,350,450]:
    t=time()
    a=ANN([512,256,128,64,46],0,1024,0)
    losses=a.fit_nadam(bs,5,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(bs))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_batchsize1_nadam_arc2.png') 

0 (0.6880738307364977, 0.7963682864450128)
0 0.7807755448710127 0.7726086956521739
1 (0.4550611816852702, 0.8632480818414322)
1 0.5623097747552629 0.842391304347826
2 (0.37974944747121436, 0.885460358056266)
2 0.5178129075383251 0.8552173913043478
3 (0.32253925087949314, 0.9018542199488491)
3 0.4857069873735875 0.8706521739130435
4 (0.2798681175824529, 0.9140153452685422)
4 0.4675991780483964 0.8756521739130435

0 (0.7521528555075488, 0.775230179028133)
0 0.81151742605401 0.7578260869565218
1 (0.47993742759720254, 0.8563299232736573)
1 0.5913465350179248 0.827391304347826
2 (0.38201257947375516, 0.884846547314578)
2 0.5232218399491894 0.8523913043478261
3 (0.32843477242576763, 0.8991048593350384)
3 0.498101778766168 0.8610869565217392
4 (0.30857383893458384, 0.9043606138107417)
4 0.49742692851152065 0.8665217391304347

0 (0.9239545005695035, 0.7310102301790281)
0 0.9715276851047397 0.7160869565217391
1 (0.5816203721478038, 0.8287979539641944)
1 0.6548835890510927 0.8147826086956522
2 (

In [101]:
print(*times)
print(best_metrics) 

115.80381321907043 82.82387447357178 69.68789410591125 62.331907987594604
[(0.2798681175824529, 0.9140153452685422), (0.30857383893458384, 0.9043606138107417), (0.33202962664895475, 0.9006649616368286), (0.34203560565640634, 0.8976598465473146)]


In [94]:
#figuring out activation function
plt.clf()
best_metrics=[]
times=[]
for af in [('sigmoid',0),('tanh',1),('relu',2)]:
    t=time()
    a=ANN([512,256,128,64,46],af[1],1024,0)
    losses=a.fit_nadam(100,5,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=af[0])
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_af_nadam_arc2.png') 

0 (3.1995453370312728, 0.0808312020460358)
0 3.2003986326407783 0.08130434782608696
1 (0.6551497569898646, 0.8119437340153453)
1 0.729566011555629 0.7828260869565218
2 (0.3209350958665301, 0.9082992327365729)
2 0.437756457651325 0.8760869565217392
3 (0.22529331576192432, 0.9344245524296675)
3 0.3793844489153522 0.8906521739130435
4 (0.16727651801811175, 0.9506777493606138)
4 0.3553137644270358 0.9043478260869565

0 (2.764803585455344, 0.14370843989769821)
0 2.7830712396582795 0.1443478260869565
1 (2.0947908656373198, 0.30792838874680306)
1 2.133736730034326 0.29934782608695654
2 (1.788534971995919, 0.3676854219948849)
2 1.8576742202616527 0.36478260869565216
3 (1.6904190301000206, 0.4171739130434783)
3 1.7914080254847522 0.4091304347826087
4 (1.440089689331602, 0.5040664961636828)
4 1.549331718749769 0.4880434782608696

0 (0.6297117649805463, 0.8162787723785166)
0 0.7104941914194769 0.7967391304347826
1 (0.4325573354102026, 0.870537084398977)
1 0.5575884836629657 0.8410869565217391
2 (

In [95]:
print(*times)
print(best_metrics)  


162.3677535057068 170.95573925971985 153.0557713508606
[(0.16727651801811175, 0.9506777493606138), (1.440089689331602, 0.5040664961636828), (0.2654249439706307, 0.9178644501278772)]


In [96]:
#figuring out activation function
plt.clf()
best_metrics=[]
times=[]
for af in [('sigmoid',0),('tanh',1),('relu',2)]:
    t=time()
    a=ANN([256,46],af[1],1024,0)
    losses=a.fit_nadam(100,5,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=af[0])
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_af_nadam_arc1.png') 

0 (0.5331808537031673, 0.8616624040920716)
0 0.5984999810587708 0.8397826086956521
1 (0.36261273904030683, 0.906381074168798)
1 0.45505332666411336 0.8782608695652174
2 (0.2783394319507596, 0.928222506393862)
2 0.39182205947858023 0.8928260869565218
3 (0.22522867615714792, 0.9412787723785166)
3 0.355854048175032 0.9023913043478261
4 (0.18341753597990765, 0.95269820971867)
4 0.33114159818421535 0.9065217391304348

0 (0.6420216536370802, 0.8217263427109974)
0 0.7089424534417788 0.8078260869565217
1 (0.4720105373194459, 0.8674680306905371)
1 0.5666338937392855 0.8406521739130435
2 (0.39784698946823066, 0.8852685421994885)
2 0.515212955638595 0.8580434782608696
3 (0.34940844351089, 0.8983759590792839)
3 0.48814803600218404 0.8621739130434782
4 (0.31160340495710326, 0.9070716112531969)
4 0.4764475861279314 0.8665217391304347

0 (0.6411647604551968, 0.8161892583120205)
0 0.7010549372341397 0.7978260869565217
1 (0.48400154153247704, 0.8597442455242966)
1 0.5652862752826046 0.8391304347826087


In [97]:
print(*times)
print(best_metrics)  

64.01590251922607 65.60790205001831 61.04390549659729
[(0.18341753597990765, 0.95269820971867), (0.31160340495710326, 0.9070716112531969), (0.3512773926545463, 0.8941432225063939)]


In [102]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [1e-4,1e-3,1e-2,.1,1]:
    t=time()
    a=ANN([256,46],0,1024,0)
    losses=a.fit_nadam(150,7,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr_nadam_arc1.png') 

0 (2.910621833890904, 0.4863682864450128)
0 2.9065009786404308 0.48891304347826087
1 (2.560633131089382, 0.5343478260869565)
1 2.556064598301219 0.5323913043478261
2 (2.323152035294736, 0.5617263427109974)
2 2.3184520092698144 0.5595652173913044
3 (2.1433902187765996, 0.5811892583120205)
3 2.138596345076375 0.5791304347826087
4 (2.0004258047363113, 0.5971227621483376)
4 1.9955504210437973 0.5908695652173913
5 (1.8834279668414045, 0.610843989769821)
5 1.8785160704405257 0.6041304347826087
6 (1.785842949524149, 0.6231457800511508)
6 1.7809712644872684 0.6191304347826087

0 (1.2233366738647455, 0.6992966751918158)
0 1.2219482955419263 0.7019565217391305
1 (1.0509071959777607, 0.7343989769820972)
1 1.0583001385010231 0.7306521739130435
2 (0.9541377988035941, 0.7570971867007673)
2 0.9683678022785556 0.753695652173913
3 (0.8821827210683142, 0.7745780051150896)
3 0.9020513534116849 0.7704347826086957
4 (0.8225842405257703, 0.7900127877237851)
4 0.8473885503152908 0.7828260869565218
5 (0.77063

<ipython-input-4-9a4801ec2207>:16: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-a))


0 (2.2809855707250626, 0.3490153452685422)
0 2.26750979643925 0.35934782608695653
1 (1.9617073544757304, 0.43833759590792837)
1 1.9259703368886252 0.44804347826086954
2 (1.7998219477239783, 0.4922762148337596)
2 1.7990923572707083 0.4960869565217391
3 (1.6863004502372647, 0.5226086956521739)
3 1.6997718650331357 0.5232608695652174
4 (1.5540861145730251, 0.5560358056265985)
4 1.5673630425216432 0.5506521739130434
5 (1.5365977868853757, 0.5696547314578005)
5 1.5433693261893684 0.5693478260869566
6 (1.5181055644172043, 0.5697953964194373)
6 1.5517651105593664 0.5619565217391305

0 (6.026386546208456, 0.05592071611253197)
0 6.034201359852479 0.05478260869565217
1 (6.679176725432725, 0.07157289002557544)
1 6.674116419856551 0.07152173913043479
2 (6.209291505050971, 0.06506393861892583)
2 6.239348390494304 0.065
3 (5.58685334820596, 0.09759590792838875)
3 5.613581567823005 0.09717391304347826
4 (7.068567188814136, 0.06648337595907929)
4 7.083686280309709 0.0667391304347826
5 (7.2001504007909

In [103]:
print(*times)
print(best_metrics)  

68.65189480781555 68.86260151863098 67.46389770507812 67.31189799308777 89.71586561203003
[(1.785842949524149, 0.6231457800511508), (0.7241347944649428, 0.8156777493606138), (0.16461909740537867, 0.9580818414322251), (1.5181055644172043, 0.5697953964194373), (9.759133639699403, 0.11452685421994885)]


In [106]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [6e-3,8e-3,1e-2,2e-2,4e-2]:
    t=time()
    a=ANN([256,46],0,1024,0)
    losses=a.fit_nadam(150,7,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr1_nadam_arc1.png') 

0 (0.5428061093471547, 0.8631841432225064)
0 0.5962847134882752 0.846304347826087
1 (0.3688840347818986, 0.9103964194373402)
1 0.447602921889797 0.8832608695652174
2 (0.2822017378717448, 0.9330690537084398)
2 0.3789329575916901 0.8995652173913044
3 (0.22685744378144918, 0.9482352941176471)
3 0.337513236805319 0.9095652173913044
4 (0.18739368327714795, 0.9583759590792839)
4 0.3109362573125843 0.9158695652173913
5 (0.15742621431048937, 0.9658567774936061)
5 0.2933095176877753 0.9217391304347826
6 (0.13385196654145953, 0.9722634271099744)
6 0.2814349030064443 0.9247826086956522

0 (0.5168838735337161, 0.8679283887468031)
0 0.5691069625410332 0.8482608695652174
1 (0.346785408617731, 0.9141048593350384)
1 0.42991492632690237 0.8856521739130435
2 (0.260064883913055, 0.9376470588235294)
2 0.36450812436896085 0.9056521739130434
3 (0.20629079919758395, 0.9521355498721228)
3 0.32783751616626544 0.912608695652174
4 (0.1686576169579983, 0.9619437340153453)
4 0.30559083683726435 0.9152173913043479


In [107]:
print(*times)
print(best_metrics)  

68.90389585494995 67.77189493179321 68.2919111251831 67.90705609321594 68.02790021896362
[(0.13385196654145953, 0.9722634271099744), (0.11691649234015754, 0.9754475703324809), (0.13579712725915197, 0.9669693094629156), (0.4352874061637624, 0.8740920716112532), (0.9884072537579212, 0.7160869565217391)]


In [104]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [1e-4,1e-3,1e-2,.1,1]:
    t=time()
    a=ANN([512,256,128,64,46],0,1024,0)
    losses=a.fit_nadam(150,7,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr_nadam_arc2.png') 

0 (3.8217695915335193, 0.022621483375959078)
0 3.821674659042616 0.019782608695652175
1 (3.7981380083822023, 0.09667519181585678)
1 3.797737535884224 0.10173913043478261
2 (3.7468978835003455, 0.09969309462915601)
2 3.7460849162814527 0.09891304347826087
3 (3.669857746204064, 0.10548593350383632)
3 3.6689977014102895 0.10521739130434783
4 (3.583145279001694, 0.11428388746803068)
4 3.5823853598691326 0.11130434782608696
5 (3.4964593195265676, 0.12240409207161125)
5 3.495683628323738 0.12130434782608696
6 (3.4133977058600617, 0.13465473145780052)
6 3.4126074492319165 0.13217391304347825

0 (2.538583705508458, 0.3726086956521739)
0 2.5434757451420036 0.37521739130434784
1 (1.9320158792872835, 0.5204475703324808)
1 1.9397055699368806 0.5265217391304348
2 (1.5584941249162012, 0.609616368286445)
2 1.5712292138002917 0.6058695652173913
3 (1.2883559443447683, 0.6759718670076726)
3 1.3083282785610604 0.6739130434782609
4 (1.088044908217584, 0.7263171355498721)
4 1.1167476928211566 0.71673913043

<ipython-input-4-9a4801ec2207>:16: RuntimeWarning: overflow encountered in exp
  return 1/(1+np.exp(-a))


0 (4.003378877033226, 0.021739130434782608)
0 4.003378877033242 0.021739130434782608
1 (4.085362790403102, 0.021739130434782608)
1 4.085362790403096 0.021739130434782608
2 (4.1783507508437285, 0.021739130434782608)
2 4.178350750843751 0.021739130434782608
3 (4.267477401073073, 0.021739130434782608)
3 4.267477401073087 0.021739130434782608
4 (4.313806717375414, 0.021739130434782608)
4 4.31380671737543 0.021739130434782608
5 (4.336123596305221, 0.021739130434782608)
5 4.336123596305206 0.021739130434782608
6 (4.343491470053158, 0.021739130434782608)
6 4.34349147005318 0.021739130434782608



In [105]:
print(*times)
print(best_metrics)  

183.64297485351562 171.17574286460876 170.80774092674255 171.29174089431763 231.79192781448364
[(3.4133977058600617, 0.13465473145780052), (0.8190885454857658, 0.7926854219948849), (0.15093054513241877, 0.9553196930946292), (3.877738745792845, 0.021739130434782608), (4.343491470053158, 0.021739130434782608)]


In [108]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [6e-3,8e-3,1e-2,2e-2,4e-2]:
    t=time()
    a=ANN([512,256,128,64,46],0,1024,0)
    losses=a.fit_nadam(150,7,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr1_nadam_arc2.png') 

0 (2.589252913294305, 0.24773657289002557)
0 2.5898611081685714 0.24304347826086956
1 (0.7466549547180386, 0.7964961636828645)
1 0.8318013407038205 0.7726086956521739
2 (0.3955211269323563, 0.8936828644501279)
2 0.5333045862088469 0.8552173913043478
3 (0.2536035062624571, 0.9318797953964194)
3 0.41916913070070744 0.8817391304347826
4 (0.18596709472794637, 0.9491432225063938)
4 0.37993194288890275 0.8939130434782608
5 (0.15211507481730804, 0.9567263427109974)
5 0.3830339778403969 0.8965217391304348
6 (0.12948506197250045, 0.9619693094629156)
6 0.39556215733289707 0.8945652173913043

0 (2.0376107983915692, 0.4169309462915601)
0 2.0599640668181594 0.4132608695652174
1 (0.5116943593409996, 0.856611253196931)
1 0.6091786640351763 0.832391304347826
2 (0.27777478436476166, 0.9239769820971867)
2 0.4144707843487754 0.8884782608695653
3 (0.17833361170785744, 0.9512148337595908)
3 0.34567634478459597 0.9082608695652173
4 (0.13201284029119276, 0.962851662404092)
4 0.3274318493131972 0.909565217391

In [109]:
print(*times)
print(best_metrics)

175.36503219604492 174.34424448013306 173.9557363986969 176.57364225387573 173.73250102996826
[(0.12948506197250045, 0.9619693094629156), (0.08657385755024487, 0.9745524296675192), (0.104520176233498, 0.968312020460358), (3.8292609836825977, 0.021739130434782608), (3.8300816047922277, 0.021739130434782608)]


# ADAM

In [ ]:
#figuring out batch size
plt.clf()
best_metrics=[]
times=[]
for bs in [25,50,100,150,500,1000]:
    t=time()
    a=ANN([256,46],2,1024,0)
    losses=a.fit_adam(bs,6,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(bs))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_batchsize_adam_arc1.png')    

In [ ]:
print(*times)
print(best_metrics)

In [ ]:
#figuring out batch size
plt.clf()
best_metrics=[]
times=[]
for bs in [750,1000,1200,1500,2000,3000]:
    t=time()
    a=ANN([256,46],2,1024,0)
    losses=a.fit_adam(bs,6,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(bs))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_batchsize_adam1_arc1.png') 

In [ ]:
print(*times)
print(best_metrics)

In [ ]:
#figuring out batch size
plt.clf()
best_metrics=[]
times=[]
for bs in [25,50,100,150,500,1000]:
    t=time()
    a=ANN([512,256,128,64,46],2,1024,0)
    losses=a.fit_adam(bs,6,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(bs))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_batchsize_adam_arc2.png') 

In [ ]:
print(*times)
print(best_metrics)

In [ ]:
#figuring out batch size
plt.clf()
best_metrics=[]
times=[]
for bs in [750,1000,1200,1500,2000,3000]:
    t=time()
    a=ANN([512,256,128,64,46],2,1024,0)
    losses=a.fit_adam(bs,6,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(bs))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_batchsize_adam1_arc2.png') 

In [ ]:
print(*times)
print(best_metrics)

In [ ]:
#figuring out activation function
plt.clf()
best_metrics=[]
times=[]
for af in [('sigmoid',0),('tanh',1),('relu',2)]:
    t=time()
    a=ANN([512,256,128,64,46],af[1],1024,0)
    losses=a.fit_adam(500,10,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=af[0])
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_af_adam_arc2.png') 

In [ ]:
print(*times)
print(best_metrics)

In [ ]:
#figuring out activation function
plt.clf()
best_metrics=[]
times=[]
for af in [0,1,2]:
    t=time()
    a=ANN([256,46],af,1024,0)
    losses=a.fit_adam(500,5,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(af))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_af_adam_arc1.png') 

In [ ]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [1e-4,1e-3,1e-2,.1,1]:
    t=time()
    a=ANN([256,46],2,1024,0)
    losses=a.fit_adam(1000,10,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr_adam_arc1.png') 

In [ ]:
print(*times)
print(best_metrics)

In [ ]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [1e-4,1e-3,1e-2,.1]:
    t=time()
    a=ANN([512,256,128,64,46],2,1024,0)
    losses=a.fit_adam(750,10,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr_adam_arc2.png') 

In [ ]:
print(*times)
print(best_metrics)

In [ ]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [8e-3,1e-2,2e-2,4e-2]:
    t=time()
    a=ANN([256,46],2,1024,0)
    losses=a.fit_adam(1000,10,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr1_adam_arc1.png') 

In [ ]:
print(*times)
print(best_metrics)

In [ ]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [8e-3,1e-2,2e-2,4e-2]:
    t=time()
    a=ANN([512,256,128,64,46],2,1024,0)
    losses=a.fit_adam(750,10,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr1_adam_arc2.png')

In [ ]:
print(*times)
print(best_metrics)

# RMSPROP

In [ ]:
#figuring out batch size
plt.clf()
best_metrics=[]
times=[]
for bs in [10,25,50,100,150,500,1000]:
    t=time()
    a=ANN([256,46],2,1024,0)
    losses=a.fit_rmsprop(bs,6,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(bs))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_batchsize_rmsprop_arc1.png')  

In [ ]:
print(*times)
print(best_metrics)

In [ ]:
#figuring out batch size
plt.clf()
best_metrics=[]
times=[]
for bs in [150,250,400,500,600,750,850]:
    t=time()
    a=ANN([256,46],2,1024,0)
    losses=a.fit_rmsprop(bs,6,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(bs))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_batchsize_rmsprop1_arc1.png')

In [ ]:
print(*times)
print(best_metrics)

In [ ]:
#figuring out batch size
plt.clf()
best_metrics=[]
times=[]
for bs in [10,25,50,100,150,500,1000]:
    t=time()
    a=ANN([512,256,128,64,46],2,1024,0)
    losses=a.fit_rmsprop(bs,6,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(bs))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_batchsize_rmsprop_arc2.png') 

In [ ]:
print(*times)
print(best_metrics)

In [ ]:
#figuring out batch size
plt.clf()
best_metrics=[]
times=[]
for bs in [150,250,400,500,600,750,850]:
    t=time()
    a=ANN([512,256,128,64,46],2,1024,0)
    losses=a.fit_rmsprop(bs,6,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(bs))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_batchsize_rmsprop1_arc2.png') 

In [ ]:
print(*times)
print(best_metrics)

In [ ]:
#figuring out activation function
plt.clf()
best_metrics=[]
times=[]
for af in [('sigmoid',0),('tanh',1),('relu',2)]:
    t=time()
    a=ANN([512,256,128,64,46],af[1],1024,0)
    losses=a.fit_rmsprop(500,5,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=af[0])
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_af_rmsprop_arc2.png') 

In [ ]:
print(*times)
print(best_metrics)

In [ ]:
#figuring out activation function
plt.clf()
best_metrics=[]
times=[]
for af in [0,1,2]:
    t=time()
    a=ANN([256,46],af,1024,0)
    losses=a.fit_rmsprop(500,5,X,y,.01)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(af))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_af_rmsprop_arc1.png') 

In [ ]:
print(*times)
print(best_metrics)

In [ ]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [1e-4,1e-3,1e-2,.1,1]:
    t=time()
    a=ANN([256,46],0,1024,0)
    losses=a.fit_rmsprop(500,10,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr_rmsprop_arc1.png') 

In [ ]:
print(*times)
print(best_metrics)

In [ ]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [1e-4,1e-3,1e-2,.1,1]:
    t=time()
    a=ANN([512,256,128,64,46],0,1024,0)
    losses=a.fit_rmsprop(500,10,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr_rmsprop_arc2.png') 

In [ ]:
print(*times)
print(best_metrics)

In [ ]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [8e-3,1e-2,2e-2,4e-2]:
    t=time()
    a=ANN([256,46],0,1024,0)
    losses=a.fit_rmsprop(500,10,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr1_rmsprop_arc1.png') 

In [ ]:
print(*times)
print(best_metrics)

In [ ]:
#figuring out lr
plt.clf()
best_metrics=[]
times=[]
for lr in [8e-3,1e-2,2e-2,4e-2]:
    t=time()
    a=ANN([512,256,128,64,46],0,1024,0)
    losses=a.fit_rmsprop(500,10,X,y,lr)
    plt.plot(range(1,len(losses)+1),[a[0] for a in losses],label=str(lr))
    plt.ylabel('loss')
    plt.xlabel('epochs')
    best_metrics.append(losses[-1])
    times.append(time()-t)
    print()
plt.legend()
plt.savefig('best_lr1_rmsprop_arc2.png') 

In [ ]:
print(*times)
print(best_metrics)

# Final Model Selection

In [96]:
class ANN:
    hidden_units=daf=af=l=W=Z=0
    
    def __init__(self,hidden_units,af,num_features,loss):
        
        self.hidden_units=hidden_units
        self.l=loss
        if(af==0):
            self.af=sigmoid
            self.daf=dsig
        elif(af==1):
            self.af=tanh
            self.daf=dtanh
        else:
            self.af=relu
            self.daf=drelu
        if(loss==0):
            self.loss=CE_loss
        else:
            self.loss=mse 
        self.W=[]
        self.b=[]
        
        
        temp=np.float32(np.random.normal(0,1,(num_features+1,hidden_units[0])))*(np.sqrt(2/(num_features+1+hidden_units[0])))
        self.W.append(temp[1:,:])
        self.b.append(temp[0])
        for i in range(1,len(hidden_units)):
            temp=np.float32(np.random.normal(0,1,(hidden_units[i-1]+1,hidden_units[i])))*(np.sqrt(2/(hidden_units[i-1]+1+hidden_units[i])))
            self.W.append(temp[1:,:])
            self.b.append(temp[0])
        self.final_W=self.W
        self.final_b=self.b
        self.Z=[]    
    def fp(self,X):
        self.Z=[]
        
        self.Z.append(X)
        
#         for i in range(len(self.W)):
#             self.Z.append(self.af(np.matmul(self.Z[i],self.W[i])+self.b[i]))
            
#         if(self.l==0):
#             return softmax(self.Z[-1]) 
        for i in range(len(self.W)-1):
            self.Z.append(self.af(np.matmul(self.Z[i],self.W[i])+self.b[i]))
            
        if(self.l==0):
            #print(self.Z[-1].shape,self.W[-1].shape,self.b[-1].shape)
            self.Z.append(np.matmul(self.Z[-1],self.W[-1])+self.b[-1])
            return softmax(self.Z[-1])
            
        else:
            self.Z.append(self.af(np.matmul(self.Z[-1],self.W[-1])+self.b[-1]))
        return self.Z[-1]
    def bp(self,X,y,lr):
        yh=self.fp(X)
        g=yh-y
        #print(g)
        dummy=np.ones(yh.shape[0])
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            
            if(i<len(self.W)-1 or self.l==1):
                
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #print(dw[0])
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
            
            g=np.dot(g,self.W[i].T)
            self.W[i]-=dw*lr
            self.b[i]-=db*lr
#     def bp(self,X,y,lr):
#         yh=self.fp(X)
#         g=yh-y
#         #print(g)
#         dummy=np.ones(yh.shape[0])
#         for i in reversed(range(len(self.W))):
#             deriv_af=self.daf(self.Z[i+1])
#             g=np.multiply(deriv_af,g)
#             dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
#             #print(dw[0])
#             #db=np.matmul(dummy,g)/yh.shape[0]
#             db=np.sum(g,axis=0)/yh.shape[0]
            
#             g=np.dot(g,self.W[i].T)
#             self.W[i]-=dw*lr
#             self.b[i]-=db*lr
    def bp_mom(self,X,y,lr,wt,bt,gamma=.9):
        yh=self.fp(X)
        g=yh-y
        #print(g)
        #print(gamma)
        dummy=np.ones(yh.shape[0])
        
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            if(i<len(self.W)-1 or self.l==1):
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
            g=np.dot(g,self.W[i].T)
            #print(gamma*wt[i])
            change=gamma*wt[i]+dw*lr
            self.W[i]-=change
            wt[i]=change
            #print(wt[i])
            change=gamma*bt[i]+db*lr
            self.b[i]-=change
            bt[i]=change
            
    
    def bp_rmsprop(self,X,y,lr,wt,bt,gamma=.9):
        
        yh=self.fp(X)
        g=yh-y
        
        #print(g)
        #print(gamma)
        dummy=np.ones(yh.shape[0])
        e=1e-8
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            
            if(i<len(self.W)-1 or self.l==1):
                
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
            g=np.dot(g,self.W[i].T)
            #print(gamma*wt[i])
            temp=gamma*wt[i]+0.1*(dw**2)
            self.W[i]-=lr*dw/(np.sqrt(temp+e))
            wt[i]=temp
            #print(wt[i])
            temp=gamma*bt[i]+.1*(db**2)
            self.b[i]-=lr*db/(np.sqrt(temp+e))
            bt[i]=temp
    def bp_adam(self,X,y,lr,vwt,vbt,mwt,mbt,b1=.9,b2=.999):
        yh=self.fp(X)
        g=yh-y
        #print(g)
        #print(gamma)
        dummy=np.ones(yh.shape[0])
        e=1e-8
        b1t=1
        b2t=1
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            
            if(i<len(self.W)-1 or self.l==1):
                
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
            g=np.dot(g,self.W[i].T)
            #print(gamma*wt[i])
            vwt[i]=b2*vwt[i]+(1-b2)*(dw**2)
            vbt[i]=b2*vbt[i]+(1-b2)*(db**2)
            mwt[i]=b1*mwt[i]+(1-b1)*(dw)
            mbt[i]=b1*mbt[i]+(1-b1)*(db)
            #print(mbt[i])
            b1t*=b1
            b2t*=b2
            self.W[i]-=lr*(mwt[i]/(1-b1t))/(np.sqrt(vwt[i]/(1-b2t)+e))
            self.b[i]-=lr*(mbt[i]/(1-b1t))/(np.sqrt(vbt[i]/(1-b2t)+e))
            
            
    def bp_nadam(self,X,y,lr,vwt,vbt,mwt,mbt,b1=.9,b2=.999):
        yh=self.fp(X)
        g=yh-y
        #print(g)
        #print(gamma)
        dummy=np.ones(yh.shape[0])
        e=1e-8
        b1t=1
        b2t=1
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            
            if(i<len(self.W)-1 or self.l==1):
                
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
            g=np.dot(g,self.W[i].T)
            #print(gamma*wt[i])
            vwt[i]=b2*vwt[i]+(1-b2)*(dw**2)
            vbt[i]=b2*vbt[i]+(1-b2)*(db**2)
            mwt[i]=b1*mwt[i]+(1-b1)*(dw)
            mbt[i]=b1*mbt[i]+(1-b1)*(db)
            #print(mbt[i])
            b1t*=b1
            b2t*=b2
            self.W[i]-=lr*(b1*mwt[i]/(1-b1t)+(1-b1)*dw/(1-b1t))/(np.sqrt(vwt[i]/(1-b2t)+e))
            self.b[i]-=lr*(b1*mbt[i]/(1-b1t)+(1-b1)*db/(1-b1t))/(np.sqrt(vbt[i]/(1-b2t)+e))
    
    def bp_nag(self,X,y,lr,wt,bt,gamma=.9):
        W_temp=self.W
        b_temp=self.b
        for i in range(len(self.W)):
            self.W[i]-=gamma*wt[i]
            self.b[i]-=gamma*bt[i]
            
        yh=self.fp(X)
        g=yh-y
        
        #print(g)
        #print(gamma)
        
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            
            if(i<len(self.W)-1 or self.l==1):
                
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
               
            g=np.dot(g,self.W[i].T)
           
            temp=gamma*wt[i]+lr*dw
            
            W_temp[i]-=temp
            wt[i]=temp
            #print(wt[i])
            temp=gamma*bt[i]+lr*db
            b_temp[i]-=temp
            bt[i]=temp
        self.W=W_temp
        self.b=b_temp
    
    def fit(self,bs,epochs,X,y,lr,t,adaptive=False):
        t_counter=time()
        th=time()
        for i in range(len(self.W)):
            
            
            self.W[i]=np.float64(self.W[i])
            self.b[i]=np.float64(self.b[i])
#         l=[]
        lmin=float('inf')
        t+=time()-th
        for i in range(epochs):
            th=time()
            if(adaptive):
                lr/=(i+1)**.5
            for j in range(X.shape[0]//bs):
                th=time()
#                 if(j==5 and i==0):
#                     for k in range(len(a.W)):
#                         temp=np.concatenate((a.b[k].reshape(1,-1),a.W[k]),axis=0)
#                         np.save('essentials/part_a_and_b/multiclass_dataset/tc_3/w_'+str(k+1)+'_iter.npy',temp)
#                         print(np.max(np.load('essentials/part_a_and_b/multiclass_dataset/tc_3/ac_w_'+str(k+1)+'_iter.npy')-temp))
                    
                self.bp(X[j*bs:(j+1)*bs,:],y[j*bs:(j+1)*bs,:],lr)
                t+=time()-th
                th=time()
                if(4.92*60>t>4.8*60):
                    l=self.loss(y,self.fp(X))
                    if(l<lmin):
                        lmin=l
                        self.final_W=self.W
                        self.final_b=self.b
#                         for k in range(len(self.final_W)):
#                             temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                             np.save(output+'w_'+str(k+1)+'_.npy',temp)
                        print(str(t)+': saved:'+str(lmin))
                        
                t+=time()-th
            
            
            th=time()    
            l=self.loss(y,self.fp(X))
            if(l<lmin):
                lmin=l
                self.final_W=self.W
                self.final_b=self.b
            t+=time()-th
            th=time()
            if(time()-t_counter>30 or t>4.4*60):
                t_counter=time()
#                 for k in range(len(self.final_W)):
#                     temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                     np.save(output+'w_'+str(k+1)+'_.npy',temp)
                print(str(t)+': saved:'+str(lmin))
            t+=time()-th
           
            if(t>300):
                return lmin
#             y_pred=self.fp(X_test)
#             l.append((self.loss(y,self.fp(X)),acc(y,self.fp(X))))
#             print(i,l[-1])
#             print(i,self.loss(y_test,y_pred),acc(y_test,y_pred))
        return lmin    
    def fit_mom(self,bs,epochs,X,y,lr,t,adaptive=False):
        t_counter=time()
        th=time()
        for i in range(len(self.W)):
            
            
            self.W[i]=np.float64(self.W[i])
            self.b[i]=np.float64(self.b[i])
#         l=[]
        lmin=float('inf')
       
        wt=[0]*len(self.W)
        bt=[0]*len(self.W)
        t+=time()-th
        for i in range(epochs):
            th=time()
            if(adaptive):
                lr/=(i+1)**.5
            for j in range(X.shape[0]//bs):
                th=time()
                self.bp_mom(X[j*bs:(j+1)*bs,:],y[j*bs:(j+1)*bs,:],lr,wt,bt)
                t+=time()-th
                th=time()
                if(4.92*60>t>4.8*60):
                    l=self.loss(y,self.fp(X))
                    if(l<lmin):
                        lmin=l
                        self.final_W=self.W
                        self.final_b=self.b
#                         for k in range(len(self.final_W)):
#                             temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                             np.save(output+'w_'+str(k+1)+'_.npy',temp)
                        print(str(t)+': saved:'+str(lmin))
                        
                t+=time()-th
            th=time()    
            l=self.loss(y,self.fp(X))
            if(l<lmin):
                lmin=l
                self.final_W=self.W
                self.final_b=self.b
            t+=time()-th
            th=time()
            if(time()-t_counter>30 or t>4.4*60):
                t_counter=time()
#                 for k in range(len(self.final_W)):
#                     temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                     np.save(output+'w_'+str(k+1)+'_.npy',temp)
                print(str(t)+': saved:'+str(lmin))
            t+=time()-th
            if(t>300):
                return lmin
#             y_pred=self.fp(X_test)
#             l.append((self.loss(y,self.fp(X)),acc(y,self.fp(X))))
#             print(i,l[-1])
#             print(i,self.loss(y_test,y_pred),acc(y_test,y_pred))
        return lmin
    def fit_nag(self,bs,epochs,X,y,lr,t,adaptive=False):
        t_counter=time()
        th=time()
        for i in range(len(self.W)):
            
            
            self.W[i]=np.float64(self.W[i])
            self.b[i]=np.float64(self.b[i])
#         l=[]
        lmin=float('inf')
        wt=[0]*len(self.W)
        bt=[0]*len(self.W)
        t+=time()-th
        for i in range(epochs):
            th=time()
            if(adaptive):
                lr/=(i+1)**.5
            for j in range(X.shape[0]//bs):
                th=time()
                self.bp_nag(X[j*bs:(j+1)*bs,:],y[j*bs:(j+1)*bs,:],lr,wt,bt)
                t+=time()-th
                th=time()
                if(4.92*60>t>4.8*60):
                    l=self.loss(y,self.fp(X))
                    if(l<lmin):
                        lmin=l
                        self.final_W=self.W
                        self.final_b=self.b
#                         for k in range(len(self.final_W)):
#                             temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                             np.save(output+'w_'+str(k+1)+'_.npy',temp)
                        print(str(t)+': saved:'+str(lmin))
                        
                t+=time()-th
            th=time()    
            l=self.loss(y,self.fp(X))
            if(l<lmin):
                lmin=l
                self.final_W=self.W
                self.final_b=self.b
            t+=time()-th
            th=time()
            if(time()-t_counter>30 or t>4.4*60):
                t_counter=time()
#                 for k in range(len(self.final_W)):
#                     temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                     np.save(output+'w_'+str(k+1)+'_.npy',temp)
                print(str(t)+': saved:'+str(lmin))
            t+=time()-th
            if(t>300):
                return lmin
#             y_pred=self.fp(X_test)
#             l.append((self.loss(y,self.fp(X)),acc(y,self.fp(X))))
#             print(i,l[-1])
#             print(i,self.loss(y_test,y_pred),acc(y_test,y_pred))
        return lmin
    def fit_adam(self,bs,epochs,X,y,lr,t,adaptive=False):
        t_counter=time()
        th=time()
        for i in range(len(self.W)):
            
            
            self.W[i]=np.float64(self.W[i])
            self.b[i]=np.float64(self.b[i])
#         l=[]
        lmin=float('inf')
        mwt=[0]*len(self.W)
        mbt=[0]*len(self.W)
        vwt=[0]*len(self.W)
        vbt=[0]*len(self.W)
        t+=time()-th
        for i in range(epochs):
            th=time()
            if(adaptive):
                lr/=(i+1)**.5
            for j in range(X.shape[0]//bs):
                th=time()
                self.bp_adam(X[j*bs:(j+1)*bs,:],y[j*bs:(j+1)*bs,:],lr,vwt,vbt,mwt,mbt)
                t+=time()-th
                th=time()
                if(4.92*60>t>4.8*60):
                    l=self.loss(y,self.fp(X))
                    if(l<lmin):
                        lmin=l
                        self.final_W=self.W
                        self.final_b=self.b
#                         for k in range(len(self.final_W)):
#                             temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                             np.save(output+'w_'+str(k+1)+'_.npy',temp)
                        print(str(t)+': saved:'+str(lmin))
                        
                t+=time()-th
            th=time()    
            l=self.loss(y,self.fp(X))
            if(l<lmin):
                lmin=l
                self.final_W=self.W
                self.final_b=self.b
            t+=time()-th
            th=time()
            if(time()-t_counter>30 or t>4.4*60):
                t_counter=time()
#                 for k in range(len(self.final_W)):
#                     temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                     np.save(output+'w_'+str(k+1)+'_.npy',temp)
                print(str(t)+': saved:'+str(lmin))
            t+=time()-th
            if(t>300):
                return lmin
#             y_pred=self.fp(X_test)
#             l.append((self.loss(y,self.fp(X)),acc(y,self.fp(X))))
#             print(i,l[-1])
#             print(i,self.loss(y_test,y_pred),acc(y_test,y_pred))    
        return lmin
    def fit_nadam(self,bs,epochs,X,y,lr,t,adaptive=False):
        t_counter=time()
        th=time()
        for i in range(len(self.W)):
            
            
            self.W[i]=np.float64(self.W[i])
            self.b[i]=np.float64(self.b[i])
#         l=[]
        lmin=float('inf')
        
        mwt=[0]*len(self.W)
        mbt=[0]*len(self.W)
        vwt=[0]*len(self.W)
        vbt=[0]*len(self.W)
        t+=time()-th
        for i in range(epochs):
            th=time()
            if(adaptive):
                lr/=(i+1)**.5
            for j in range(X.shape[0]//bs):
                th=time()
                self.bp_nadam(X[j*bs:(j+1)*bs,:],y[j*bs:(j+1)*bs,:],lr,vwt,vbt,mwt,mbt)
                t+=time()-th
                th=time()
                if(4.92*60>t>4.8*60):
                    l=self.loss(y,self.fp(X))
                    if(l<lmin):
                        lmin=l
                        self.final_W=self.W
                        self.final_b=self.b
#                         for k in range(len(self.final_W)):
#                             temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                             np.save(output+'w_'+str(k+1)+'_.npy',temp)
                        print(str(t)+': saved:'+str(lmin))
                        
                t+=time()-th
            th=time()    
            l=self.loss(y,self.fp(X))
            if(l<lmin):
                lmin=l
                self.final_W=self.W
                self.final_b=self.b
            t+=time()-th
            th=time()
            if(time()-t_counter>30 or t>4.4*60):
                t_counter=time()
#                 for k in range(len(self.final_W)):
#                     temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                     np.save(output+'w_'+str(k+1)+'_.npy',temp)
                print(str(t)+': saved:'+str(lmin))
            t+=time()-th
            if(t>300):
                return lmin
#             y_pred=self.fp(X_test)
#             l.append((self.loss(y,self.fp(X)),acc(y,self.fp(X))))
#             print(i,l[-1])
#             print(i,self.loss(y_test,y_pred),acc(y_test,y_pred))    
        return lmin
    def fit_rmsprop(self,bs,epochs,X,y,lr,t,adaptive=False):
        t_counter=time()
        th=time()
        for i in range(len(self.W)):
            
            
            self.W[i]=np.float64(self.W[i])
            self.b[i]=np.float64(self.b[i])
#         l=[]
        lmin=float('inf')
        
        wt=[0]*len(self.W)
        bt=[0]*len(self.W)
        t+=time()-th
        for i in range(epochs):
            th=time()
            if(adaptive):
                lr/=(i+1)**.5
            for j in range(X.shape[0]//bs):
                th=time()
                self.bp_rmsprop(X[j*bs:(j+1)*bs,:],y[j*bs:(j+1)*bs,:],lr,wt,bt)
                t+=time()-th
                th=time()
                if(4.92*60>t>4.8*60):
                    l=self.loss(y,self.fp(X))
                    if(l<lmin):
                        lmin=l
                        self.final_W=self.W
                        self.final_b=self.b
#                         for k in range(len(self.final_W)):
#                             temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                             np.save(output+'w_'+str(k+1)+'_.npy',temp)
                        print(str(t)+': saved:'+str(lmin))
                        
                t+=time()-th
            th=time()    
            l=self.loss(y,self.fp(X))
            if(l<lmin):
                lmin=l
                self.final_W=self.W
                self.final_b=self.b
            t+=time()-th
            th=time()
            if(time()-t_counter>30 or t>4.4*60):
                t_counter=time()
#                 for k in range(len(self.final_W)):
#                     temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                     np.save(output+'w_'+str(k+1)+'_.npy',temp)
                print(str(t)+': saved:'+str(lmin))
            t+=time()-th
            if(t>300):
                return lmin
#             y_pred=self.fp(X_test)
#             l.append((self.loss(y,self.fp(X)),acc(y,self.fp(X))))
#             print(i,l[-1])
#             print(i,self.loss(y_test,y_pred),acc(y_test,y_pred))             
        return lmin    
    def pred(X):
        return fp(X)
        

# Architecture 1

In [98]:
a=ANN([256,46],2,1024,0)
t=10
loss=a.fit(50,100,X,y,.4,t)
print(loss)

41.32071661949158: saved:0.12797411423976807
73.32387375831604: saved:0.07027356416544742
103.6833426952362: saved:0.05504735456063214
139.03887939453125: saved:0.021270111887961924
173.8141074180603: saved:0.00017494197113149831
204.0316677093506: saved:0.0001108212985047658
238.1155378818512: saved:8.736161010640652e-05
268.86664485931396: saved:7.584576209434174e-05
274.0070159435272: saved:7.431216657007477e-05
278.96747517585754: saved:7.286542980889305e-05
284.28012108802795: saved:7.149089873203478e-05
288.0011155605316: saved:7.035017011193176e-05
289.2041132450104: saved:7.034872089993565e-05
290.39062309265137: saved:7.034788642504598e-05
290.9436228275299: saved:7.034136261343404e-05
291.50262117385864: saved:7.034045641102987e-05
292.0766203403473: saved:7.034032840463493e-05
296.1106140613556: saved:7.01854951613793e-05
300.4596073627472: saved:6.894863688066795e-05
6.894863688066795e-05


In [ ]:
a=ANN([256,46],2,1024,0)
t=10
loss=a.fit(50,100,X,y,.4,t)
print(loss)

In [100]:
a=ANN([256,46],2,1024,0)
t=10
loss=a.fit(50,100,X,y,.35,t)
print(loss)

42.654956340789795: saved:0.09703423464693788
73.32191133499146: saved:0.037492628910760356
106.49486422538757: saved:0.0021108621700548728
140.47281193733215: saved:0.0006271814128793796
174.98076176643372: saved:0.0004581858223084695
205.0827362537384: saved:0.0003805464109991747
238.90568685531616: saved:0.00032168883124949544
265.6586434841156: saved:0.00028591043445511734
270.14663791656494: saved:0.0002808158818167856
274.70562863349915: saved:0.000276008136203134
279.0646216869354: saved:0.00027125153318918944
283.4306149482727: saved:0.0002666988367700959
287.7686085700989: saved:0.00026236648726842547
288.0016086101532: saved:0.00026149835883794437
288.5986077785492: saved:0.00026149606546131525
289.1876084804535: saved:0.00026112132162253726
289.7596061229706: saved:0.00026110341154071735
290.31560492515564: saved:0.00026092082131281506
290.87560415267944: saved:0.0002609161059662264
291.4456031322479: saved:0.00026087644700339545
292.0086033344269: saved:0.000260868212232297

In [99]:
a=ANN([256,46],2,1024,0)
t=10
loss=a.fit_mom(100,100,X,y,.05,t)
print(loss)

41.36794924736023: saved:0.10391741052848788
73.18089747428894: saved:0.027093435540840507
104.1708471775055: saved:0.003019167796886527
134.9477939605713: saved:0.0017070545284267515
166.51874613761902: saved:0.0012439329661831499
198.00570011138916: saved:0.000985622917816816
229.7176501750946: saved:0.0008171650783001632
263.4245991706848: saved:0.0006840371143262208
267.6345911026001: saved:0.0006704490805206594
271.8125834465027: saved:0.0006577553746658887
276.02157735824585: saved:0.0006447589434031394
280.2115697860718: saved:0.000632713945751054
284.7315630912781: saved:0.0006211128894572326
288.00055718421936: saved:0.0006129897233264396
288.59655714035034: saved:0.0006127368365698264
289.24255657196045: saved:0.0006124623341601627
289.8565535545349: saved:0.000612185301356519
290.46055340766907: saved:0.0006119165136848907
291.04255294799805: saved:0.0006116203451544764
291.63455176353455: saved:0.0006112726232059867
292.30655169487: saved:0.0006108922261968546
292.944549798

In [54]:
a=ANN([256,46],2,1024,0)
t=10
loss=a.fit_adam(1000,100,X,y,8e-3,t)
print(loss)

42.88261604309082: saved:0.11905815183808706
74.70530700683594: saved:0.06015486065135422
106.76371121406555: saved:0.03462704952614278
139.03086924552917: saved:0.021779691090854588
171.53801488876343: saved:0.015705342491201424
204.25313544273376: saved:0.004794366822594597
237.19999861717224: saved:0.0034991839028428724
265.95128655433655: saved:0.0027220486343607774
269.0392801761627: saved:0.002665416017322769
272.05951023101807: saved:0.0026055679386806367
275.0118832588196: saved:0.0025602823329546297
278.0318784713745: saved:0.002513729174474239
281.1850538253784: saved:0.00247078180215711
284.9134247303009: saved:0.002436337359032371
288.0374176502228: saved:0.002403994532707785
289.221657037735: saved:0.002403994532707785
294.52594566345215: saved:0.0024010639082449664
295.16194438934326: saved:0.0023785900389272367
299.0139379501343: saved:0.0023713868870420467
302.9548485279083: saved:0.0023452791722606307
0.0023452791722606307


In [55]:
a=ANN([256,46],0,1024,0)
t=10
loss=a.fit_rmsprop(750,100,X,y,8e-3,t)
print(loss)

41.082013845443726: saved:0.12332795023176055
72.54975748062134: saved:0.02823347383653065
103.60066413879395: saved:0.00369779949627232
134.214679479599: saved:0.0019159525703624583
164.7087323665619: saved:0.001449354468164846
195.45958137512207: saved:0.0012512045293725429
226.23848843574524: saved:0.0011450129717433337
257.25273990631104: saved:0.0010803470852205354
264.22115325927734: saved:0.0010693964864091291
267.6936752796173: saved:0.0010642786470509455
271.0616674423218: saved:0.0010593808166209223
274.49792671203613: saved:0.001054690534860404
277.9023344516754: saved:0.0010501962429505397
281.35093808174133: saved:0.001045887203941671
284.7469308376312: saved:0.0010417534314937002
288.1671586036682: saved:0.001037785625926574
288.2191627025604: saved:0.0010368675260882946
299.0398027896881: saved:0.001033975116715574
302.3440833091736: saved:0.0010303138106814182
0.0010303138106814182


In [56]:
a=ANN([256,46],2,1024,0)
t=10
loss=a.fit_nag(150,100,X,y,2e-2,t)
print(loss)

43.879080057144165: saved:0.10758415115640874
74.01815676689148: saved:0.043209396567702396
104.23666262626648: saved:0.015381860895133615
134.48392486572266: saved:0.007647287038125716
167.35059690475464: saved:0.004869642868010666
200.40246272087097: saved:0.0035740841928279393
233.6527395248413: saved:0.0028018989195474457
266.55492401123047: saved:0.0023040967325510006
270.9352421760559: saved:0.0022473002704949266
275.3676075935364: saved:0.0021923850117627937
279.97220826148987: saved:0.0021414108977161886
284.5765161514282: saved:0.002091324940934573
288.00451254844666: saved:0.002066246915694852
296.5227949619293: saved:0.0020434346269076853
301.0587878227234: saved:0.001998465655169792
0.001998465655169792


In [57]:
a=ANN([256,46],0,1024,0)
t=10
loss=a.fit_nadam(150,100,X,y,8e-3,t)
print(loss)

48.67934823036194: saved:0.20795972976578908
87.0498571395874: saved:0.09825353727164873
117.47231888771057: saved:0.061639313179063376
153.9161021709442: saved:0.03504091762916864
189.98120379447937: saved:0.01931894488943356
226.4648265838623: saved:0.01109597351872415
263.3484139442444: saved:0.006725226908388872
273.23713970184326: saved:0.0059975198086723415
283.18737030029297: saved:0.0052810397817105335
288.0083956718445: saved:0.0038609093531581894
299.92470693588257: saved:0.0038609093531581894
309.35941672325134: saved:0.0038609093531581894
0.0038609093531581894


# Architecture 2

In [14]:
a=ANN([512,256,128,64,46],2,1024,0)
t=10
loss=a.fit(50,100,X,y,.3,t)

52.448058128356934: saved:0.10471136575974462
94.45933961868286: saved:0.022718815072883352
135.11355328559875: saved:0.020879694450728854
175.1303837299347: saved:0.02082787937899941
213.86919569969177: saved:0.020826687344057127
252.7358114719391: saved:0.020826667349356832
265.72941637039185: saved:0.02082666714759808
278.59058237075806: saved:0.020826667102483494
288.00738644599915: saved:0.020826667092779808
289.49539947509766: saved:0.020826667092778448
294.03138184547424: saved:0.020826667092778195
299.2007246017456: saved:0.02082666709263871
312.57374119758606: saved:0.020826667090539822


In [ ]:
a=ANN([512,256,128,64,46],2,1024,0)
t=10
loss=a.fit(50,100,X,y,.3,t,True)

In [22]:
a=ANN([512,256,128,64,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X,y,.01,t)
print(loss)

45.10695552825928: saved:0.31392902907407383
78.85891103744507: saved:0.1522993439375868
112.6389753818512: saved:0.09350404349209757
150.51981902122498: saved:0.05428237077112218
185.24399399757385: saved:0.03978696961527502
220.18794012069702: saved:0.01602000214263224
253.70988988876343: saved:0.004727858148767623
266.56182384490967: saved:0.0022736294231141332
281.415052652359: saved:0.0007337633649778262
288.0108013153076: saved:0.0006673938907146987
289.8357963562012: saved:0.0006664443808572475
292.0337920188904: saved:0.0006656827677705265
294.255788564682: saved:0.000662630966533914
302.86955165863037: saved:0.0006082029050242926
0.0006082029050242926


In [16]:
a=ANN([512,256,128,64,46],2,1024,0)
t=10
loss=a.fit_adam(750,100,X,y,8e-3,t)
print(loss)

33.448949337005615: saved:0.25111272978184007
67.46989727020264: saved:0.1081185689595886
101.12484812736511: saved:0.054789599004107944
134.3647973537445: saved:0.054789599004107944
167.4107472896576: saved:0.03013228660070909
200.3586974143982: saved:0.0260242732341453
233.5976483821869: saved:0.023744490968175623
266.4396002292633: saved:0.023744490968175623
274.6875858306885: saved:0.01363690760512086
283.06759333610535: saved:0.01363690760512086
298.36537861824036: saved:0.01363690760512086
306.6503870487213: saved:0.01363690760512086
0.01363690760512086


In [17]:
a=ANN([512,256,128,64,46],0,1024,0)
t=10
loss=a.fit_rmsprop(450,100,X,y,8e-3,t)
print(loss)

30.356956958770752: saved:1.1361408806583408
60.52393817901611: saved:0.3010104295319888
99.9718508720398: saved:0.10162258497606037
139.39279174804688: saved:0.07712867820591945
169.59074592590332: saved:0.04877729865090138
209.1266839504242: saved:0.04762702472396235
240.40763974189758: saved:0.028826459191550927
270.68659234046936: saved:0.028826459191550927
281.59557461738586: saved:0.028173661834546337
301.22854709625244: saved:0.028173661834546337
0.028173661834546337


In [18]:
a=ANN([512,256,128,64,46],2,1024,0)
t=10
loss=a.fit_nag(250,100,X,y,4e-2,t)
print(loss)

35.64595985412598: saved:0.19448453856199543
73.03190279006958: saved:0.08478366594675804
110.9818480014801: saved:0.07312738459739919
146.22979545593262: saved:0.05469768371758282
182.2627410888672: saved:0.0377501736379212
218.0496699810028: saved:0.036721588888194316
254.15461587905884: saved:0.027445313030928252
272.0875895023346: saved:0.02048549828432247
281.2185871601105: saved:0.0184695873433179
298.7915606498718: saved:0.0184695873433179
307.97154664993286: saved:0.0184695873433179
0.0184695873433179


In [19]:
a=ANN([512,256,128,64,46],0,1024,0)
t=10
loss=a.fit_nadam(150,100,X,y,8e-3,t)
print(loss)

49.158923387527466: saved:0.47719411824933244
96.81585383415222: saved:0.1970304408131215
142.7967836856842: saved:0.12071752417255614
188.6857144832611: saved:0.08986706060715001
240.2697286605835: saved:0.06227296387977846
264.38571071624756: saved:0.05178309337406109
287.9476749897003: saved:0.05118085114559342
288.05769324302673: saved:0.0497114881124312
290.5626699924469: saved:0.047405501152649916
293.1574490070343: saved:0.045785215039322875
320.01240849494934: saved:0.045785215039322875
0.045785215039322875


# PART D

In [1]:
import numpy as np
import pandas as pd
#from scipy.special import expit
import matplotlib
from matplotlib import pyplot as plt
matplotlib.use('Agg')
from time import time



In [2]:
np.random.seed(146)

In [22]:


data=pd.read_csv('data/train_data_shuffled.csv',header=None)
test=pd.read_csv('data/public_test.csv',header=None)
X_test,y_test=test.iloc[:,:-1].to_numpy()/255.,pd.get_dummies(test.iloc[:,-1:],columns=test.columns[-1:]).to_numpy()
X,y=data.iloc[:,:-1].to_numpy()/255.,pd.get_dummies(data.iloc[:,-1:],columns=data.columns[-1:]).to_numpy()
X.shape,y.shape,X_test.shape,y_test.shape



((78200, 1024), (78200, 46), (4600, 1024), (4600, 46))

In [8]:
def softmax(a):
    m=np.max(a,axis=1).reshape(-1,1)
    #gamma=1e-15
    a=np.exp(a-m)
    #return a/(a.sum(axis=a.ndim-1).reshape(-1,1))
    return np.nan_to_num(a/(a.sum(axis=a.ndim-1).reshape(-1,1)),False)
def relu(a):
    return np.where(a>0,a,0)
def drelu(a):
    return np.where(a>0,1,0)
def tanh(a):
    return np.tanh(a)  
def dtanh(a):
    return 1-np.square(a)
def sigmoid(a):
    return 1/(1+np.exp(-a))

    #return expit(a)
def dsig(a):
    #return sigmoid(a)*(1-sigmoid(a))
    return a*(1-a)
    #return (1-expit(a))*expit(a)
def CE_loss(y_true,y_preds):
    gamma=1e-15
    return -np.sum(np.multiply(y_true,np.log(np.clip(y_preds,gamma,1-gamma))))/y_true.shape[0]

def mse(y_true,y_preds):  
    return np.sum(np.square(y_true.reshape(-1,1)-y_preds.reshape(-1,1)))/y_preds.shape[0]


def acc(y_true,y_preds):
    y_preds=np.argmax(y_preds,axis=1).squeeze()
    y_true=np.argmax(y_true,axis=1).squeeze()
    return np.count_nonzero(y_true-y_preds==0)/len(y_true)

In [19]:
class ANN:
    hidden_units=daf=af=l=W=Z=0
    
    def __init__(self,hidden_units,af,num_features,loss):
        
        self.hidden_units=hidden_units
        self.l=loss
        if(af==0):
            self.af=sigmoid
            self.daf=dsig
        elif(af==1):
            self.af=tanh
            self.daf=dtanh
        else:
            self.af=relu
            self.daf=drelu
        if(loss==0):
            self.loss=CE_loss
        else:
            self.loss=mse 
        self.W=[]
        self.b=[]
        
        
        temp=np.float32(np.random.normal(0,1,(num_features+1,hidden_units[0])))*(np.sqrt(2/(num_features+1+hidden_units[0])))
        self.W.append(temp[1:,:])
        self.b.append(temp[0])
        for i in range(1,len(hidden_units)):
            temp=np.float32(np.random.normal(0,1,(hidden_units[i-1]+1,hidden_units[i])))*(np.sqrt(2/(hidden_units[i-1]+1+hidden_units[i])))
            self.W.append(temp[1:,:])
            self.b.append(temp[0])
        self.final_W=self.W
        self.final_b=self.b
        self.Z=[]    
    def fp(self,X):
        self.Z=[]
        
        self.Z.append(X)
        
#         for i in range(len(self.W)):
#             self.Z.append(self.af(np.matmul(self.Z[i],self.W[i])+self.b[i]))
            
#         if(self.l==0):
#             return softmax(self.Z[-1]) 
        for i in range(len(self.W)-1):
            self.Z.append(self.af(np.matmul(self.Z[i],self.W[i])+self.b[i]))
            
        if(self.l==0):
            #print(self.Z[-1].shape,self.W[-1].shape,self.b[-1].shape)
            self.Z.append(np.matmul(self.Z[-1],self.W[-1])+self.b[-1])
            return softmax(self.Z[-1])
            
        else:
            self.Z.append(self.af(np.matmul(self.Z[-1],self.W[-1])+self.b[-1]))
        return self.Z[-1]
    def bp(self,X,y,lr):
        yh=self.fp(X)
        g=yh-y
        #print(g)
        dummy=np.ones(yh.shape[0])
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            
            if(i<len(self.W)-1 or self.l==1):
                
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #print(dw[0])
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
            
            g=np.dot(g,self.W[i].T)
            self.W[i]-=dw*lr
            self.b[i]-=db*lr
#     def bp(self,X,y,lr):
#         yh=self.fp(X)
#         g=yh-y
#         #print(g)
#         dummy=np.ones(yh.shape[0])
#         for i in reversed(range(len(self.W))):
#             deriv_af=self.daf(self.Z[i+1])
#             g=np.multiply(deriv_af,g)
#             dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
#             #print(dw[0])
#             #db=np.matmul(dummy,g)/yh.shape[0]
#             db=np.sum(g,axis=0)/yh.shape[0]
            
#             g=np.dot(g,self.W[i].T)
#             self.W[i]-=dw*lr
#             self.b[i]-=db*lr
    def bp_mom(self,X,y,lr,wt,bt,gamma=.9):
        yh=self.fp(X)
        g=yh-y
        #print(g)
        #print(gamma)
        dummy=np.ones(yh.shape[0])
        
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            if(i<len(self.W)-1 or self.l==1):
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
            g=np.dot(g,self.W[i].T)
            #print(gamma*wt[i])
            change=gamma*wt[i]+dw*lr
            self.W[i]-=change
            wt[i]=change
            #print(wt[i])
            change=gamma*bt[i]+db*lr
            self.b[i]-=change
            bt[i]=change
            
    
    def bp_rmsprop(self,X,y,lr,wt,bt,gamma=.9):
        
        yh=self.fp(X)
        g=yh-y
        
        #print(g)
        #print(gamma)
        dummy=np.ones(yh.shape[0])
        e=1e-8
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            
            if(i<len(self.W)-1 or self.l==1):
                
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
            g=np.dot(g,self.W[i].T)
            #print(gamma*wt[i])
            temp=gamma*wt[i]+0.1*(dw**2)
            self.W[i]-=lr*dw/(np.sqrt(temp+e))
            wt[i]=temp
            #print(wt[i])
            temp=gamma*bt[i]+.1*(db**2)
            self.b[i]-=lr*db/(np.sqrt(temp+e))
            bt[i]=temp
    def bp_adam(self,X,y,lr,vwt,vbt,mwt,mbt,b1=.9,b2=.999):
        yh=self.fp(X)
        g=yh-y
        #print(g)
        #print(gamma)
        dummy=np.ones(yh.shape[0])
        e=1e-8
        b1t=1
        b2t=1
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            
            if(i<len(self.W)-1 or self.l==1):
                
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
            g=np.dot(g,self.W[i].T)
            #print(gamma*wt[i])
            vwt[i]=b2*vwt[i]+(1-b2)*(dw**2)
            vbt[i]=b2*vbt[i]+(1-b2)*(db**2)
            mwt[i]=b1*mwt[i]+(1-b1)*(dw)
            mbt[i]=b1*mbt[i]+(1-b1)*(db)
            #print(mbt[i])
            b1t*=b1
            b2t*=b2
            self.W[i]-=lr*(mwt[i]/(1-b1t))/(np.sqrt(vwt[i]/(1-b2t)+e))
            self.b[i]-=lr*(mbt[i]/(1-b1t))/(np.sqrt(vbt[i]/(1-b2t)+e))
            
            
    def bp_nadam(self,X,y,lr,vwt,vbt,mwt,mbt,b1=.9,b2=.999):
        yh=self.fp(X)
        g=yh-y
        #print(g)
        #print(gamma)
        dummy=np.ones(yh.shape[0])
        e=1e-8
        b1t=1
        b2t=1
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            
            if(i<len(self.W)-1 or self.l==1):
                
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
            g=np.dot(g,self.W[i].T)
            #print(gamma*wt[i])
            vwt[i]=b2*vwt[i]+(1-b2)*(dw**2)
            vbt[i]=b2*vbt[i]+(1-b2)*(db**2)
            mwt[i]=b1*mwt[i]+(1-b1)*(dw)
            mbt[i]=b1*mbt[i]+(1-b1)*(db)
            #print(mbt[i])
            b1t*=b1
            b2t*=b2
            self.W[i]-=lr*(b1*mwt[i]/(1-b1t)+(1-b1)*dw/(1-b1t))/(np.sqrt(vwt[i]/(1-b2t)+e))
            self.b[i]-=lr*(b1*mbt[i]/(1-b1t)+(1-b1)*db/(1-b1t))/(np.sqrt(vbt[i]/(1-b2t)+e))
    
    def bp_nag(self,X,y,lr,wt,bt,gamma=.9):
        W_temp=self.W
        b_temp=self.b
        for i in range(len(self.W)):
            self.W[i]-=gamma*wt[i]
            self.b[i]-=gamma*bt[i]
            
        yh=self.fp(X)
        g=yh-y
        
        #print(g)
        #print(gamma)
        
        for i in reversed(range(len(self.W))):
            
            deriv_af=self.daf(self.Z[i+1])
            
            if(i<len(self.W)-1 or self.l==1):
                
                g=np.multiply(g,deriv_af)
            dw=np.matmul(self.Z[i].T,g)/yh.shape[0]
            #db=np.matmul(dummy,g)/yh.shape[0]
            db=np.sum(g,axis=0)/yh.shape[0]
               
            g=np.dot(g,self.W[i].T)
           
            temp=gamma*wt[i]+lr*dw
            
            W_temp[i]-=temp
            wt[i]=temp
            #print(wt[i])
            temp=gamma*bt[i]+lr*db
            b_temp[i]-=temp
            bt[i]=temp
        self.W=W_temp
        self.b=b_temp
    
    def fit(self,bs,epochs,X,y,x_t,y_t,lr,t,adaptive=False):
        t_counter=time()
        th=time()
        for i in range(len(self.W)):
            
            
            self.W[i]=np.float64(self.W[i])
            self.b[i]=np.float64(self.b[i])
#         l=[]
        lmin=float('inf')
        amax=float('-inf')
        t+=time()-th
        for i in range(epochs):
            th=time()
            if(adaptive):
                lr/=(i+1)**.5
            for j in range(X.shape[0]//bs):
                th=time()
#                 if(j==5 and i==0):
#                     for k in range(len(a.W)):
#                         temp=np.concatenate((a.b[k].reshape(1,-1),a.W[k]),axis=0)
#                         np.save('essentials/part_a_and_b/multiclass_dataset/tc_3/w_'+str(k+1)+'_iter.npy',temp)
#                         print(np.max(np.load('essentials/part_a_and_b/multiclass_dataset/tc_3/ac_w_'+str(k+1)+'_iter.npy')-temp))
                    
                self.bp(X[j*bs:(j+1)*bs,:],y[j*bs:(j+1)*bs,:],lr)
                t+=time()-th
                th=time()
                if(4.92*60>t>4.8*60):
                    l=self.loss(y_t,self.fp(x_t))
                    a=acc(y_t,self.fp(x_t))
                    if(a>amax):
                        amax=a
                        lmin=l
                        self.final_W=self.W
                        self.final_b=self.b
#                         for k in range(len(self.final_W)):
#                             temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                             np.save(output+'w_'+str(k+1)+'_.npy',temp)
                        yh=self.fp(X_test)
                        print(str(t)+': train_loss:'+ str(self.loss(y,self.fp(X))) +' test_acc:'+str(amax)+" val loss:" +str(self.loss(y_test,yh))+" acc:"+str(acc(y_test,yh)))
    
                t+=time()-th
            th=time()    
            l=self.loss(y_t,self.fp(x_t))
            a=acc(y_t,self.fp(x_t))
            if(a>amax):
                amax=a
                lmin=l
                self.final_W=self.W
                self.final_b=self.b
            t+=time()-th
            th=time()
            if(time()-t_counter>30 or t>4.4*60):
                t_counter=time()
#                 for k in range(len(self.final_W)):
#                     temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                     np.save(output+'w_'+str(k+1)+'_.npy',temp)
                yh=self.fp(X_test)
                print(str(t)+': train_loss:' +str(self.loss(y,self.fp(X))) +' test_acc:'+str(amax)+" val loss:" +str(self.loss(y_test,yh))+" acc:"+str(acc(y_test,yh)))
    
            t+=time()-th
            if(t>300):
                return lmin
                
#             y_pred=self.fp(X_test)
#             l.append((self.loss(y,self.fp(X)),acc(y,self.fp(X))))
#             print(i,l[-1])
#             print(i,self.loss(y_test,y_pred),acc(y_test,y_pred))
        return lmin    
    def fit_mom(self,bs,epochs,X,y,x_t,y_t,lr,t,adaptive=False):
        t_counter=time()
        th=time()
        for i in range(len(self.W)):
            
            
            self.W[i]=np.float64(self.W[i])
            self.b[i]=np.float64(self.b[i])
#         l=[]
        lmin=float('inf')
        amax=float('-inf')
       
        wt=[0]*len(self.W)
        bt=[0]*len(self.W)
        t+=time()-th
        for i in range(epochs):
            th=time()
            if(adaptive):
                lr/=(i+1)**.5
            for j in range(X.shape[0]//bs):
                th=time()
                self.bp_mom(X[j*bs:(j+1)*bs,:],y[j*bs:(j+1)*bs,:],lr,wt,bt)
                t+=time()-th
                th=time()
                if(4.92*60>t>4.8*60):
                    l=self.loss(y_t,self.fp(x_t))
                    a=acc(y_t,self.fp(x_t))
                    if(a>amax):
                        amax=a
                        lmin=l
                        self.final_W=self.W
                        self.final_b=self.b
#                         for k in range(len(self.final_W)):
#                             temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                             np.save(output+'w_'+str(k+1)+'_.npy',temp)
                        yh=self.fp(X_test)
                        print(str(t)+': train_loss:'+ str(self.loss(y,self.fp(X))) +' test_acc:'+str(amax)+" val loss:" +str(self.loss(y_test,yh))+" acc:"+str(acc(y_test,yh)))
    
                t+=time()-th
            th=time()    
            l=self.loss(y_t,self.fp(x_t))
            a=acc(y_t,self.fp(x_t))
            if(a>amax):
                amax=a
                lmin=l
                self.final_W=self.W
                self.final_b=self.b
            t+=time()-th
            th=time()
            if(time()-t_counter>30 or t>4.4*60):
                t_counter=time()
#                 for k in range(len(self.final_W)):
#                     temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                     np.save(output+'w_'+str(k+1)+'_.npy',temp)
                yh=self.fp(X_test)
                print(str(t)+': train_loss:' +str(self.loss(y,self.fp(X))) +' test_acc:'+str(amax)+" val loss:" +str(self.loss(y_test,yh))+" acc:"+str(acc(y_test,yh)))
    
            t+=time()-th
            if(t>300):
                return lmin
            
#             y_pred=self.fp(X_test)
#             l.append((self.loss(y,self.fp(X)),acc(y,self.fp(X))))
#             print(i,l[-1])
#             print(i,self.loss(y_test,y_pred),acc(y_test,y_pred))
        return lmin
    def fit_nag(self,bs,epochs,X,y,x_t,y_t,lr,t,adaptive=False):
        t_counter=time()
        th=time()
        for i in range(len(self.W)):
            
            
            self.W[i]=np.float64(self.W[i])
            self.b[i]=np.float64(self.b[i])
#         l=[]
        lmin=float('inf')
        amax=float('-inf')
        wt=[0]*len(self.W)
        bt=[0]*len(self.W)
        t+=time()-th
        for i in range(epochs):
            th=time()
            if(adaptive):
                lr/=(i+1)**.5
            for j in range(X.shape[0]//bs):
                th=time()
                self.bp_nag(X[j*bs:(j+1)*bs,:],y[j*bs:(j+1)*bs,:],lr,wt,bt)
                t+=time()-th
                th=time()
                if(4.92*60>t>4.8*60):
                    l=self.loss(y_t,self.fp(x_t))
                    a=acc(y_t,self.fp(x_t))
                    if(a>amax):
                        amax=a
                        lmin=l
                        self.final_W=self.W
                        self.final_b=self.b
#                         for k in range(len(self.final_W)):
#                             temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                             np.save(output+'w_'+str(k+1)+'_.npy',temp)
                        yh=self.fp(X_test)
                        print(str(t)+': train_loss:'+ str(self.loss(y,self.fp(X))) +' test_acc:'+str(amax)+" val loss:" +str(self.loss(y_test,yh))+" acc:"+str(acc(y_test,yh)))
    
                t+=time()-th
            th=time()    
            l=self.loss(y_t,self.fp(x_t))
            a=acc(y_t,self.fp(x_t))
            if(a>amax):
                amax=a
                lmin=l
                self.final_W=self.W
                self.final_b=self.b
            t+=time()-th
            th=time()
            if(time()-t_counter>30 or t>4.4*60):
                t_counter=time()
#                 for k in range(len(self.final_W)):
#                     temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                     np.save(output+'w_'+str(k+1)+'_.npy',temp)
                yh=self.fp(X_test)
                print(str(t)+': train_loss:' +str(self.loss(y,self.fp(X))) +' test_acc:'+str(amax)+" val loss:" +str(self.loss(y_test,yh))+" acc:"+str(acc(y_test,yh)))
    
            t+=time()-th
            if(t>300):
                return lmin
        
#             y_pred=self.fp(X_test)
#             l.append((self.loss(y,self.fp(X)),acc(y,self.fp(X))))
#             print(i,l[-1])
#             print(i,self.loss(y_test,y_pred),acc(y_test,y_pred))
        return lmin
    def fit_adam(self,bs,epochs,X,y,x_t,y_t,lr,t,adaptive=False):
        t_counter=time()
        th=time()
        for i in range(len(self.W)):
            
            
            self.W[i]=np.float64(self.W[i])
            self.b[i]=np.float64(self.b[i])
#         l=[]
        lmin=float('inf')
        amax=float('-inf')
    
        mwt=[0]*len(self.W)
        mbt=[0]*len(self.W)
        vwt=[0]*len(self.W)
        vbt=[0]*len(self.W)
        t+=time()-th
        for i in range(epochs):
            th=time()
            if(adaptive):
                lr/=(i+1)**.5
            for j in range(X.shape[0]//bs):
                th=time()
                self.bp_adam(X[j*bs:(j+1)*bs,:],y[j*bs:(j+1)*bs,:],lr,vwt,vbt,mwt,mbt)
                t+=time()-th
                th=time()
                if(4.92*60>t>4.8*60):
                    l=self.loss(y_t,self.fp(x_t))
                    a=acc(y_t,self.fp(x_t))
                    if(a>amax):
                        amax=a
                        lmin=l
                        self.final_W=self.W
                        self.final_b=self.b
#                         for k in range(len(self.final_W)):
#                             temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                             np.save(output+'w_'+str(k+1)+'_.npy',temp)
                        yh=self.fp(X_test)
                        print(str(t)+': train_loss:'+ str(self.loss(y,self.fp(X))) +' test_acc:'+str(amax)+" val loss:" +str(self.loss(y_test,yh))+" acc:"+str(acc(y_test,yh)))
    
                t+=time()-th
            th=time()    
            l=self.loss(y_t,self.fp(x_t))
            a=acc(y_t,self.fp(x_t))
            if(a>amax):
                amax=a
                lmin=l
                self.final_W=self.W
                self.final_b=self.b
            t+=time()-th
            th=time()
            if(time()-t_counter>30 or t>4.4*60):
                t_counter=time()
#                 for k in range(len(self.final_W)):
#                     temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                     np.save(output+'w_'+str(k+1)+'_.npy',temp)
                yh=self.fp(X_test)
                print(str(t)+': train_loss:' +str(self.loss(y,self.fp(X))) +' test_acc:'+str(amax)+" val loss:" +str(self.loss(y_test,yh))+" acc:"+str(acc(y_test,yh)))
    
            t+=time()-th
            if(t>300):
                return lmin
            
#             y_pred=self.fp(X_test)
#             l.append((self.loss(y,self.fp(X)),acc(y,self.fp(X))))
#             print(i,l[-1])
#             print(i,self.loss(y_test,y_pred),acc(y_test,y_pred))    
        return lmin
    def fit_nadam(self,bs,epochs,X,y,x_t,y_t,lr,t,adaptive=False):
        t_counter=time()
        th=time()
        for i in range(len(self.W)):
            
            
            self.W[i]=np.float64(self.W[i])
            self.b[i]=np.float64(self.b[i])
#         l=[]
        lmin=float('inf')
        amax=float('-inf')
        
        mwt=[0]*len(self.W)
        mbt=[0]*len(self.W)
        vwt=[0]*len(self.W)
        vbt=[0]*len(self.W)
        t+=time()-th
        for i in range(epochs):
            th=time()
            if(adaptive):
                lr/=(i+1)**.5
            for j in range(X.shape[0]//bs):
                th=time()
                self.bp_nadam(X[j*bs:(j+1)*bs,:],y[j*bs:(j+1)*bs,:],lr,vwt,vbt,mwt,mbt)
                t+=time()-th
                th=time()
                if(4.92*60>t>4.8*60):
                    l=self.loss(y_t,self.fp(x_t))
                    a=acc(y_t,self.fp(x_t))
                    if(a>amax):
                        amax=a
                        lmin=l
                        self.final_W=self.W
                        self.final_b=self.b
#                         for k in range(len(self.final_W)):
#                             temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                             np.save(output+'w_'+str(k+1)+'_.npy',temp)
                        yh=self.fp(X_test)
                        print(str(t)+': train_loss:'+ str(self.loss(y,self.fp(X))) +' test_acc:'+str(amax)+" val loss:" +str(self.loss(y_test,yh))+" acc:"+str(acc(y_test,yh)))
    
                t+=time()-th
            th=time()    
            l=self.loss(y_t,self.fp(x_t))
            a=acc(y_t,self.fp(x_t))
            if(a>amax):
                amax=a
                lmin=l
                self.final_W=self.W
                self.final_b=self.b
            t+=time()-th
            th=time()
            if(time()-t_counter>30 or t>4.4*60):
                t_counter=time()
#                 for k in range(len(self.final_W)):
#                     temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                     np.save(output+'w_'+str(k+1)+'_.npy',temp)
                yh=self.fp(X_test)
                print(str(t)+': train_loss:' +str(self.loss(y,self.fp(X))) +' test_acc:'+str(amax)+" val loss:" +str(self.loss(y_test,yh))+" acc:"+str(acc(y_test,yh)))
    
            t+=time()-th
            if(t>300):
                return lmin
            
#             y_pred=self.fp(X_test)
#             l.append((self.loss(y,self.fp(X)),acc(y,self.fp(X))))
#             print(i,l[-1])
#             print(i,self.loss(y_test,y_pred),acc(y_test,y_pred))


        return lmin
    def fit_rmsprop(self,bs,epochs,X,y,x_t,y_t,lr,t,adaptive=False):
        t_counter=time()
        th=time()
        for i in range(len(self.W)):
            
            
            self.W[i]=np.float64(self.W[i])
            self.b[i]=np.float64(self.b[i])
#         l=[]
        lmin=float('inf')
        amax=float('-inf')
        
        
        wt=[0]*len(self.W)
        bt=[0]*len(self.W)
        t+=time()-th
        for i in range(epochs):
            th=time()
            if(adaptive):
                lr/=(i+1)**.5
            for j in range(X.shape[0]//bs):
                th=time()
                self.bp_rmsprop(X[j*bs:(j+1)*bs,:],y[j*bs:(j+1)*bs,:],lr,wt,bt)
                t+=time()-th
                th=time()
                if(4.92*60>t>4.8*60):
                    l=self.loss(y_t,self.fp(x_t))
                    a=acc(y_t,self.fp(x_t))
                    if(a>amax):
                        amax=a
                        lmin=l
                        self.final_W=self.W
                        self.final_b=self.b
#                         for k in range(len(self.final_W)):
#                             temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                             np.save(output+'w_'+str(k+1)+'_.npy',temp)
                        yh=self.fp(X_test)
                        print(str(t)+': train_loss:'+ str(self.loss(y,self.fp(X))) +' test_acc:'+str(amax)+" val loss:" +str(self.loss(y_test,yh))+" acc:"+str(acc(y_test,yh)))
    
                t+=time()-th
            th=time()    
            l=self.loss(y_t,self.fp(x_t))
            a=acc(y_t,self.fp(x_t))
            if(a>amax):
                amax=a
                lmin=l
                self.final_W=self.W
                self.final_b=self.b
            t+=time()-th
            th=time()
            if(time()-t_counter>30 or t>4.4*60):
                t_counter=time()
#                 for k in range(len(self.final_W)):
#                     temp=np.concatenate((self.final_b[k].reshape(1,-1),self.final_W[k]),axis=0)
#                     np.save(output+'w_'+str(k+1)+'_.npy',temp)
                yh=self.fp(X_test)
                print(str(t)+': train_loss:' +str(self.loss(y,self.fp(X))) +' test_acc:'+str(amax)+" val loss:" +str(self.loss(y_test,yh))+" acc:"+str(acc(y_test,yh)))
    
            t+=time()-th
            if(t>300):
                return lmin
#             y_pred=self.fp(X_test)
#             l.append((self.loss(y,self.fp(X)),acc(y,self.fp(X))))
#             print(i,l[-1])
            
        return lmin    
    def pred(X):
        return fp(X)
        

# Experiments With Architecture 1

In [26]:
a=ANN([256,46],2,1024,0)
t=10
loss=a.fit(50,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.4,t)
print(loss)

41.262192249298096: train_loss:0.08266702214499312 test_acc:0.9149616368286445 val loss:0.38792946983895893 acc:0.9145652173913044
72.63228297233582: train_loss:0.02552921895982873 test_acc:0.9254475703324808 val loss:0.4165266288488191 acc:0.927608695652174
104.06613659858704: train_loss:0.0010182816194912537 test_acc:0.9414322250639386 val loss:0.37159762358082354 acc:0.941304347826087
135.56419205665588: train_loss:0.0006106780468631648 test_acc:0.9423273657289003 val loss:0.3723652484839842 acc:0.9423913043478261
167.34012484550476: train_loss:0.0005571264521559632 test_acc:0.9423273657289003 val loss:0.37446969389294493 acc:0.9419565217391305
198.6105751991272: train_loss:0.0005292271927941024 test_acc:0.9425831202046036 val loss:0.37608795919496163 acc:0.9421739130434783
230.08026099205017: train_loss:0.0005122029369257819 test_acc:0.9427109974424552 val loss:0.37745694377502587 acc:0.9423913043478261
261.8625705242157: train_loss:0.0005008625643571648 test_acc:0.9427109974424552

In [30]:
a=ANN([256,46],2,1024,0)
t=10
loss=a.fit_mom(100,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.05,t)
print(loss)

40.09081435203552: train_loss:0.09553805454174076 test_acc:0.9163682864450128 val loss:0.3434852961308703 acc:0.9119565217391304
70.81776857376099: train_loss:0.011066131520207668 test_acc:0.9378516624040921 val loss:0.2889095003097324 acc:0.9332608695652174
101.29762959480286: train_loss:0.0027267216056061066 test_acc:0.9414322250639386 val loss:0.28538812858878065 acc:0.941304347826087
132.89635944366455: train_loss:0.0018710243952697382 test_acc:0.9419437340153453 val loss:0.2908431752469638 acc:0.9419565217391305
163.4471263885498: train_loss:0.0014927759410966646 test_acc:0.9419437340153453 val loss:0.2954722209251515 acc:0.9419565217391305
193.98150753974915: train_loss:0.0012763307823573835 test_acc:0.9423273657289003 val loss:0.299405674524126 acc:0.9419565217391305
226.16499304771423: train_loss:0.0011349965877748827 test_acc:0.9423273657289003 val loss:0.30299046855250367 acc:0.9423913043478261
258.57956290245056: train_loss:0.0010353454428433703 test_acc:0.9427109974424552 v

In [35]:
a=ANN([128,46],2,1024,0)
t=10
loss=a.fit_mom(100,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.05,t)
print(loss)

40.347622871398926: train_loss:0.15520961572526473 test_acc:0.8904092071611254 val loss:0.5322703651949836 acc:0.8897826086956522
71.10976314544678: train_loss:0.06673060024408951 test_acc:0.8994884910485933 val loss:0.6782378064601419 acc:0.8978260869565218
103.04412889480591: train_loss:0.01652035610465319 test_acc:0.9090792838874681 val loss:0.7125660450932827 acc:0.9056521739130434
133.18637251853943: train_loss:0.0011109855978471317 test_acc:0.9135549872122762 val loss:0.650013827449448 acc:0.9158695652173913
164.24121165275574: train_loss:0.0010318196563880219 test_acc:0.9138107416879795 val loss:0.6511235507636801 acc:0.9173913043478261
196.11633849143982: train_loss:0.000998657048981445 test_acc:0.9141943734015345 val loss:0.6528985867385492 acc:0.917608695652174
0.6725228741577973


In [45]:
a=ANN([512,46],2,1024,0)
t=10
loss=a.fit_mom(100,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.05,t)
print(loss)

5:46.99563479423523: train_loss:0.08884899296369662 test_acc:0.9271099744245525 val loss:0.2647423444649119 acc:0.9243478260869565
9:80.354576587677: train_loss:0.02517171927953997 test_acc:0.9392583120204604 val loss:0.23091899873189387 acc:0.9410869565217391
13:114.04790234565735: train_loss:0.006379836430520092 test_acc:0.9465473145780051 val loss:0.22095986712183718 acc:0.9458695652173913
17:147.79002165794373: train_loss:0.003671411449451637 test_acc:0.9482097186700768 val loss:0.22094406689348012 acc:0.9460869565217391
21:179.59834694862366: train_loss:0.002724783554403161 test_acc:0.9482097186700768 val loss:0.2225635417780868 acc:0.9476086956521739
25:210.26033091545105: train_loss:0.0022055723908933817 test_acc:0.9491048593350384 val loss:0.22425573581812006 acc:0.9469565217391305
30:247.6072859764099: train_loss:0.001799531807372963 test_acc:0.9491048593350384 val loss:0.2262532332914022 acc:0.9484782608695652
33:269.0992648601532: train_loss:0.0016249512225627396 test_acc:0.

In [47]:
a=ANN([512,46],2,1024,0)
t=10
loss=a.fit_mom(100,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.06,t)
print(loss)

47.30288052558899: train_loss:0.06963760580129334 test_acc:0.9296675191815856 val loss:0.254137256981233 acc:0.9321739130434783
82.55191707611084: train_loss:0.01358105849361978 test_acc:0.9445012787723785 val loss:0.22806044770409728 acc:0.9434782608695652
113.24575591087341: train_loss:0.003785048138990098 test_acc:0.9485933503836317 val loss:0.22832210684999657 acc:0.9480434782608695
145.40468549728394: train_loss:0.002382815203149153 test_acc:0.9485933503836317 val loss:0.23227050400137242 acc:0.9482608695652174
181.18163895606995: train_loss:0.0017275648658705276 test_acc:0.948849104859335 val loss:0.2355662916311726 acc:0.9478260869565217
211.87357664108276: train_loss:0.0014406515159333287 test_acc:0.94923273657289 val loss:0.23760388478676625 acc:0.9489130434782609
247.94447779655457: train_loss:0.0012077578518339344 test_acc:0.9494884910485933 val loss:0.23980710149189052 acc:0.9491304347826087
264.07745337486267: train_loss:0.0011383412781555762 test_acc:0.949616368286445 val

In [48]:
a=ANN([512,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.08,t)
print(loss)

45.51695275306702: train_loss:0.07608854046887875 test_acc:0.9248081841432225 val loss:0.27165022827778573 acc:0.9252173913043479
76.87988519668579: train_loss:0.01134816823154632 test_acc:0.9457800511508951 val loss:0.24485457747817987 acc:0.9421739130434783
108.53381705284119: train_loss:0.002924155124502741 test_acc:0.950383631713555 val loss:0.23560230959822004 acc:0.947391304347826
139.1097981929779: train_loss:0.0019930090080198183 test_acc:0.950383631713555 val loss:0.23832971933136926 acc:0.9491304347826087
169.19277095794678: train_loss:0.0015288191628450968 test_acc:0.950383631713555 val loss:0.2407579939811907 acc:0.9489130434782609
205.0137231349945: train_loss:0.0011997086113290083 test_acc:0.9507672634271099 val loss:0.24334854317898638 acc:0.9491304347826087
235.73558592796326: train_loss:0.0010193290183852712 test_acc:0.951150895140665 val loss:0.24520035113197983 acc:0.9493478260869566
267.1865301132202: train_loss:0.0008875172390670605 test_acc:0.9516624040920716 val 

In [40]:
a=ANN([1024,46],2,1024,0)
t=10
loss=a.fit_mom(100,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.05,t)
print(loss)

44.46650981903076: train_loss:0.20816794744472017 test_acc:0.9099744245524297 val loss:0.32418478652049604 acc:0.9063043478260869
77.50869083404541: train_loss:0.08940816995695972 test_acc:0.9313299232736573 val loss:0.25677534359627785 acc:0.9289130434782609
110.24464154243469: train_loss:0.043227982005507704 test_acc:0.9396419437340153 val loss:0.2264443572779894 acc:0.9373913043478261
144.71792554855347: train_loss:0.016360817431878328 test_acc:0.9497442455242967 val loss:0.20510072344360375 acc:0.9458695652173913
177.17969465255737: train_loss:0.008275799309103922 test_acc:0.9497442455242967 val loss:0.1989772763261806 acc:0.9495652173913044
209.79056549072266: train_loss:0.005462310000141867 test_acc:0.9501278772378516 val loss:0.19810480697777186 acc:0.9510869565217391
242.65252804756165: train_loss:0.004127277694436684 test_acc:0.9502557544757033 val loss:0.19800656719537005 acc:0.9515217391304348
275.52540159225464: train_loss:0.003362364377982156 test_acc:0.9507672634271099 va

In [49]:
a=ANN([1024,46],2,1024,0)
t=10
loss=a.fit_mom(100,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.09,t)
print(loss)

54.12390112876892: train_loss:0.10652899983206786 test_acc:0.9239130434782609 val loss:0.26683997341595017 acc:0.9239130434782609
98.61893105506897: train_loss:0.03735469564772316 test_acc:0.9363171355498722 val loss:0.25654920037340995 acc:0.938695652173913
130.57963228225708: train_loss:0.012030378035568412 test_acc:0.9489769820971867 val loss:0.21772665490029575 acc:0.9471739130434783
166.2162139415741: train_loss:0.002819844201322849 test_acc:0.9510230179028133 val loss:0.21638401451213224 acc:0.9545652173913044
200.3132610321045: train_loss:0.001337575497853241 test_acc:0.9526854219948849 val loss:0.21257135855642548 acc:0.9543478260869566
234.62621688842773: train_loss:0.0010017348529206913 test_acc:0.9526854219948849 val loss:0.21638408546410837 acc:0.9547826086956521
270.1179029941559: train_loss:0.00088133759547869 test_acc:0.9526854219948849 val loss:0.21779983069441688 acc:0.9547826086956521
288.0098659992218: train_loss:0.0008588181586851214 test_acc:0.9528132992327366 val 

In [83]:
a=ANN([1024,46],2,1024,0)
t=10
loss=a.fit_mom(100,100,X[:-3000],y[:-3000],X[-3000:],y[-3000:],.09,t)
print(loss)

42.2140839099884: train_loss:0.17499722053703234 test_acc:0.9176666666666666 val loss:0.3299338812855967 acc:0.9047826086956522
72.74903774261475: train_loss:0.07310760319936425 test_acc:0.9316666666666666 val loss:0.26916663247348727 acc:0.9339130434782609
103.0559914112091: train_loss:0.033452664225160335 test_acc:0.9423333333333334 val loss:0.25764685149821537 acc:0.9415217391304348
134.18470191955566: train_loss:0.012637947859854656 test_acc:0.9526666666666667 val loss:0.2245291946895543 acc:0.9497826086956521
164.35965657234192: train_loss:0.0035428793382821016 test_acc:0.956 val loss:0.21546437563962043 acc:0.9530434782608695
195.87660789489746: train_loss:0.0012670203319164084 test_acc:0.957 val loss:0.20892065714698313 acc:0.9571739130434782
226.8465621471405: train_loss:0.000740983410923291 test_acc:0.957 val loss:0.2098803162188857 acc:0.9576086956521739
257.32551741600037: train_loss:0.0006163912826427537 test_acc:0.957 val loss:0.21107518639911094 acc:0.957391304347826
273.

In [87]:
a=ANN([1024,46],2,1024,0)
t=10
loss=a.fit_mom(100,100,X[:-4600],y[:-4600],X[-4600:],y[-4600:],.09,t)
print(loss)

52.013225078582764: train_loss:0.09653931051393023 test_acc:0.9319565217391305 val loss:0.24443105178907118 acc:0.933695652173913
84.03978967666626: train_loss:0.04726175231346494 test_acc:0.9376086956521739 val loss:0.24207150586876197 acc:0.9402173913043478
116.80743312835693: train_loss:0.023331535634837967 test_acc:0.9447826086956522 val loss:0.228863264220836 acc:0.945
149.162029504776: train_loss:0.005594485313718897 test_acc:0.952391304347826 val loss:0.20496404984660338 acc:0.9526086956521739
181.17680978775024: train_loss:0.0017309473000619653 test_acc:0.953695652173913 val loss:0.20088699408326904 acc:0.9543478260869566
214.41254377365112: train_loss:0.0008475438372312761 test_acc:0.955 val loss:0.20042922045437822 acc:0.9567391304347826
247.7152373790741: train_loss:0.0006823866768170152 test_acc:0.9554347826086956 val loss:0.20272389381834865 acc:0.9569565217391305
265.71294593811035: train_loss:0.0006284797120027301 test_acc:0.9554347826086956 val loss:0.20354322243736606 

In [50]:
a=ANN([4112,46],2,1024,0)
t=10
loss=a.fit_mom(100,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.09,t)
print(loss)

73.48195576667786: train_loss:0.2793754464604033 test_acc:0.8929667519181586 val loss:0.3670539588475433 acc:0.8958695652173913
131.96083736419678: train_loss:0.14320226450624285 test_acc:0.9236572890025575 val loss:0.27602814691581373 acc:0.92
190.4877471923828: train_loss:0.07849507093362737 test_acc:0.9382352941176471 val loss:0.2321326617972155 acc:0.9328260869565217
249.70969796180725: train_loss:0.05569279133561433 test_acc:0.9388746803069054 val loss:0.23117351738542055 acc:0.9378260869565217
288.0546040534973: train_loss:0.04489325997350582 test_acc:0.9416879795396419 val loss:0.21585801503348812 acc:0.938695652173913
314.7215449810028: train_loss:0.031897960492760356 test_acc:0.9441176470588235 val loss:0.2044953705263588 acc:0.948695652173913
0.21121879281437775


In [41]:
a=ANN([2056,46],2,1024,0)
t=10
loss=a.fit_mom(100,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.05,t)
print(loss)

43.16076397895813: train_loss:0.3610211596900865 test_acc:0.8772378516624041 val loss:0.4380539930832572 acc:0.8756521739130435
76.46585297584534: train_loss:0.18818353181637534 test_acc:0.9164961636828645 val loss:0.3029515827340974 acc:0.9160869565217391
109.5958948135376: train_loss:0.11245989134679582 test_acc:0.931457800511509 val loss:0.24983375247720652 acc:0.928695652173913
142.86073684692383: train_loss:0.07419904303116276 test_acc:0.939002557544757 val loss:0.2243831285375406 acc:0.9389130434782609
176.07668805122375: train_loss:0.05451811945852207 test_acc:0.9410485933503836 val loss:0.2175851731233941 acc:0.9408695652173913
209.36673212051392: train_loss:0.03992116208457885 test_acc:0.9429667519181586 val loss:0.2115691445373178 acc:0.9423913043478261
242.7466893196106: train_loss:0.022492660041832344 test_acc:0.948849104859335 val loss:0.19329625449756058 acc:0.9480434782608695
275.8405261039734: train_loss:0.01291798225743894 test_acc:0.9514066496163683 val loss:0.1869871

In [84]:
a=ANN([2056,46],2,1024,0)
t=10
loss=a.fit_mom(100,100,X[:-3000],y[:-3000],X[-3000:],y[-3000:],.05,t)
print(loss)

40.475953578948975: train_loss:0.32567096806633383 test_acc:0.8873333333333333 val loss:0.40099215435908636 acc:0.8943478260869565
76.9208984375: train_loss:0.16286261272132174 test_acc:0.926 val loss:0.2793116249187157 acc:0.9202173913043479
108.5098512172699: train_loss:0.10218394900407163 test_acc:0.933 val loss:0.24081555569653681 acc:0.9339130434782609
139.27780413627625: train_loss:0.06680309442398752 test_acc:0.9416666666666667 val loss:0.21826236954122769 acc:0.9421739130434783
169.76875734329224: train_loss:0.04370842773337474 test_acc:0.9446666666666667 val loss:0.2043726374793081 acc:0.9447826086956522
200.807710647583: train_loss:0.0294209301029503 test_acc:0.946 val loss:0.19485465370959154 acc:0.9508695652173913
231.48666429519653: train_loss:0.01989515202269364 test_acc:0.9486666666666667 val loss:0.18887721734882904 acc:0.9528260869565217
262.635617017746: train_loss:0.011477335484571699 test_acc:0.9503333333333334 val loss:0.17652412525267816 acc:0.9565217391304348
288

In [88]:
a=ANN([2056,46],2,1024,0)
t=10
loss=a.fit_mom(100,100,X[:-4600],y[:-4600],X[-4600:],y[-4600:],.06,t)
print(loss)

40.36271381378174: train_loss:0.31638916282488627 test_acc:0.8913043478260869 val loss:0.3957563851023779 acc:0.8930434782608696
75.48057961463928: train_loss:0.15834353943793072 test_acc:0.923695652173913 val loss:0.27437673218523884 acc:0.9243478260869565
110.17982697486877: train_loss:0.08963922121451945 test_acc:0.9360869565217391 val loss:0.22309822844051658 acc:0.9380434782608695
144.69655299186707: train_loss:0.055856968845533476 test_acc:0.9415217391304348 val loss:0.20648595315026752 acc:0.9432608695652174
178.37724113464355: train_loss:0.03491401148547181 test_acc:0.9467391304347826 val loss:0.19456294792526263 acc:0.9458695652173913
213.31767463684082: train_loss:0.021340736971827794 test_acc:0.9476086956521739 val loss:0.18378407604409575 acc:0.9519565217391305
248.07698965072632: train_loss:0.015977084402071916 test_acc:0.9476086956521739 val loss:0.186540600960453 acc:0.9497826086956521
283.40035700798035: train_loss:0.011308817680099977 test_acc:0.9493478260869566 val lo

In [44]:
a=ANN([2056,1024,46],2,1024,0)
t=10
loss=a.fit_mom(100,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.05,t)
print(loss)

1:69.24138593673706: train_loss:0.28507074199132143 test_acc:0.8881074168797954 val loss:0.3699993964614538 acc:0.8878260869565218
2:135.0524618625641: train_loss:0.14745248405465233 test_acc:0.9251918158567775 val loss:0.2767719780751033 acc:0.9158695652173913
3:203.11429905891418: train_loss:0.09038522043145687 test_acc:0.9347826086956522 val loss:0.23647526629260607 acc:0.9332608695652174
4:269.9599668979645: train_loss:0.0476625784216795 test_acc:0.9415601023017903 val loss:0.2073131974355972 acc:0.9443478260869566
5:342.4302337169647: train_loss:0.03522232466073427 test_acc:0.9470588235294117 val loss:0.21396611583366912 acc:0.9469565217391305
0.21792264972152994


In [33]:
a=ANN([256,46],2,1024,0)
t=10
loss=a.fit_adam(1000,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],8e-3,t)
print(loss)

41.89926290512085: train_loss:0.10431818219809841 test_acc:0.9080562659846547 val loss:0.340794729671566 acc:0.9108695652173913
72.3189685344696: train_loss:0.05966634921443277 test_acc:0.9139386189258312 val loss:0.379033161079445 acc:0.9095652173913044
103.4020745754242: train_loss:0.02090885009035843 test_acc:0.9153452685421994 val loss:0.4022986507892415 acc:0.9189130434782609
133.53558468818665: train_loss:0.008837360884718719 test_acc:0.9163682864450128 val loss:0.4159753248869036 acc:0.923695652173913
163.64719605445862: train_loss:0.005724785595400709 test_acc:0.9187979539641944 val loss:0.43393230039009667 acc:0.9234782608695652
194.06693625450134: train_loss:0.004565727839021777 test_acc:0.919309462915601 val loss:0.45636666850320207 acc:0.9247826086956522
225.0540895462036: train_loss:0.0038983693465492225 test_acc:0.919309462915601 val loss:0.47677987267917477 acc:0.9254347826086956
0.4672875783036417


In [93]:
a=ANN([256,46],0,1024,0)
t=10
loss=a.fit_rmsprop(750,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],8e-3,t)
print(loss)

40.546563148498535: train_loss:0.11087464040553803 test_acc:0.9156010230179028 val loss:0.35497194032422613 acc:0.8958695652173913
72.26303672790527: train_loss:0.007188024978275241 test_acc:0.9289002557544757 val loss:0.3153499131649393 acc:0.9247826086956522
103.91778349876404: train_loss:0.0035108789652393015 test_acc:0.9289002557544757 val loss:0.333819259536786 acc:0.9291304347826087
135.19662141799927: train_loss:0.0029852528640763173 test_acc:0.9289002557544757 val loss:0.34509091985214474 acc:0.93
165.9431426525116: train_loss:0.002814976174917824 test_acc:0.9289002557544757 val loss:0.352702007028565 acc:0.9295652173913044
197.35965251922607: train_loss:0.0027275233830706376 test_acc:0.9289002557544757 val loss:0.3592447546908421 acc:0.9282608695652174
228.5703160762787: train_loss:0.002682261283034257 test_acc:0.9289002557544757 val loss:0.3646032115091804 acc:0.928695652173913
259.92398262023926: train_loss:0.0026572408152392996 test_acc:0.9289002557544757 val loss:0.3691543

In [91]:
a=ANN([256,46],2,1024,0)
t=10
loss=a.fit_nag(150,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],2e-2,t)
print(loss)

43.07457256317139: train_loss:0.1153549374133409 test_acc:0.9132992327365729 val loss:0.312848888155484 acc:0.9134782608695652
75.18223237991333: train_loss:0.030654748434270143 test_acc:0.9269820971867008 val loss:0.2750560930870429 acc:0.9308695652173913
107.52529430389404: train_loss:0.011544496406402793 test_acc:0.931841432225064 val loss:0.2726731542645378 acc:0.9317391304347826
140.97376656532288: train_loss:0.006276327198359369 test_acc:0.9341432225063938 val loss:0.27408413289256484 acc:0.9358695652173913
171.93313908576965: train_loss:0.004571346583124602 test_acc:0.9346547314578005 val loss:0.2794649125594886 acc:0.9371739130434783
203.663227558136: train_loss:0.0035295594736820955 test_acc:0.9355498721227622 val loss:0.283557200475558 acc:0.9371739130434783
235.1708209514618: train_loss:0.002826980791930811 test_acc:0.9355498721227622 val loss:0.28691427556483584 acc:0.9371739130434783
264.51430201530457: train_loss:0.002357970785752178 test_acc:0.9355498721227622 val loss:0

In [38]:
a=ANN([256,46],2,1024,0)
t=10
loss=a.fit_nadam(150,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],8e-3,t)
print(loss)

43.34117817878723: train_loss:0.27468045039991384 test_acc:0.8778772378516624 val loss:0.42536898729900546 acc:0.8823913043478261
77.58846282958984: train_loss:0.19384948157524687 test_acc:0.8850383631713555 val loss:0.41309831713108053 acc:0.8910869565217391
111.50620007514954: train_loss:0.15401172876076638 test_acc:0.8850383631713555 val loss:0.4503490034487381 acc:0.8915217391304348
142.7453498840332: train_loss:0.13462461897046923 test_acc:0.8860613810741688 val loss:0.4879785365173288 acc:0.8917391304347826
185.29615378379822: train_loss:0.12524161069998702 test_acc:0.8860613810741688 val loss:0.5330855258116973 acc:0.8871739130434783
228.05610418319702: train_loss:0.11589378248594402 test_acc:0.8860613810741688 val loss:0.5823250053498805 acc:0.8869565217391304
271.5781600475311: train_loss:0.11463531304662661 test_acc:0.8860613810741688 val loss:0.6322816296192193 acc:0.8865217391304347
285.81313824653625: train_loss:0.11694737196695879 test_acc:0.8860613810741688 val loss:0.64

# Experiments With Architecture 2

In [52]:
a=ANN([512,256,128,64,46],2,1024,0)
t=10
loss=a.fit(50,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.3,t)

49.47291088104248: train_loss:0.19176146738862815 test_acc:0.9044757033248082 val loss:0.34973870890445047 acc:0.9073913043478261
79.65886425971985: train_loss:0.14193743008725376 test_acc:0.9108695652173913 val loss:0.32819198065333594 acc:0.9154347826086957
119.16980171203613: train_loss:0.08495978606322958 test_acc:0.9290281329923273 val loss:0.29294482445536785 acc:0.9334782608695652
158.70673823356628: train_loss:0.06862308152143834 test_acc:0.9336317135549872 val loss:0.317702261442539 acc:0.9330434782608695
199.32466983795166: train_loss:0.054169859014960416 test_acc:0.9372122762148337 val loss:0.36656742336366627 acc:0.9360869565217391
230.41061878204346: train_loss:0.07712629852436655 test_acc:0.9387468030690537 val loss:0.3902841925781242 acc:0.9280434782608695
269.04755902290344: train_loss:0.09399990750990728 test_acc:0.9387468030690537 val loss:0.4006824232803993 acc:0.9269565217391305
279.66954231262207: train_loss:0.04609923752822146 test_acc:0.9387468030690537 val loss:

In [56]:
a=ANN([512,256,128,64,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.01,t)
print(loss)

43.82794761657715: train_loss:0.19084719356580024 test_acc:0.9012787723785166 val loss:0.3450837435342641 acc:0.9054347826086957
79.42789125442505: train_loss:0.06746372833268331 test_acc:0.9258312020460358 val loss:0.2837520133426393 acc:0.9278260869565217
113.96383833885193: train_loss:0.02276970697391312 test_acc:0.9351662404092072 val loss:0.2903643661459689 acc:0.9391304347826087
144.35479354858398: train_loss:0.004509106102090241 test_acc:0.9432225063938618 val loss:0.2742124044266747 acc:0.9471739130434783
180.81573867797852: train_loss:0.0007430282572061838 test_acc:0.94769820971867 val loss:0.2751374498636 acc:0.9480434782608695
216.92668223381042: train_loss:0.0005684626871931719 test_acc:0.94846547314578 val loss:0.2840306531984164 acc:0.9478260869565217
252.12063121795654: train_loss:0.0004901584534738832 test_acc:0.94846547314578 val loss:0.2897596348081359 acc:0.9478260869565217
266.9156084060669: train_loss:0.00046934723552896976 test_acc:0.94846547314578 val loss:0.2916

In [62]:
a=ANN([1024,256,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.01,t)
print(loss)

54.917006731033325: train_loss:0.41127489231853065 test_acc:0.8687979539641943 val loss:0.4765130495301958 acc:0.8758695652173913
88.18719172477722: train_loss:0.2355767709852649 test_acc:0.9099744245524297 val loss:0.3366383204362781 acc:0.9121739130434783
120.20163059234619: train_loss:0.15599971345825708 test_acc:0.9210997442455243 val loss:0.2840180438044925 acc:0.9243478260869565
150.724769115448: train_loss:0.1141485955558958 test_acc:0.9255754475703325 val loss:0.2637566555286072 acc:0.9280434782608695
181.8798725605011: train_loss:0.08325465735359694 test_acc:0.9265984654731457 val loss:0.2490940379719092 acc:0.9315217391304348
213.17160868644714: train_loss:0.05340766480489517 test_acc:0.9351662404092072 val loss:0.22841590448521554 acc:0.9397826086956522
246.12781238555908: train_loss:0.03556430548184356 test_acc:0.939769820971867 val loss:0.2185281422904139 acc:0.9423913043478261
264.1588439941406: train_loss:0.03024596505297454 test_acc:0.9415601023017903 val loss:0.2172765

In [63]:
a=ANN([512,256,512,256,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.01,t)
print(loss)

43.49921631813049: train_loss:0.3105380245777005 test_acc:0.8831202046035805 val loss:0.40883912221665575 acc:0.8804347826086957
80.31079959869385: train_loss:0.1657357182208851 test_acc:0.9104859335038363 val loss:0.3186552269744593 acc:0.9108695652173913
114.08378386497498: train_loss:0.0805198854007733 test_acc:0.9286445012787724 val loss:0.270501190336825 acc:0.9282608695652174
150.0311357975006: train_loss:0.05564888767637494 test_acc:0.9294117647058824 val loss:0.26318511307164916 acc:0.9321739130434783
184.1056945323944: train_loss:0.046672214982333364 test_acc:0.9355498721227622 val loss:0.2998708789752177 acc:0.9343478260869565
217.47676825523376: train_loss:0.006215849141663356 test_acc:0.948849104859335 val loss:0.24178839649717226 acc:0.9493478260869566
250.28018021583557: train_loss:0.0019036663818992486 test_acc:0.9506393861892584 val loss:0.2443670044433574 acc:0.9506521739130435
272.96229219436646: train_loss:0.0013526572814701672 test_acc:0.9510230179028133 val loss:0.

In [64]:
a=ANN([256,128,256,128,64,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.01,t)
print(loss)

43.92864441871643: train_loss:0.18263682245140028 test_acc:0.9021739130434783 val loss:0.39328123364959644 acc:0.8895652173913043
74.5399100780487: train_loss:0.09335717390313365 test_acc:0.9171355498721228 val loss:0.39099295040064375 acc:0.9139130434782609
105.13591432571411: train_loss:0.0355176794096262 test_acc:0.9292838874680307 val loss:0.3821457737530466 acc:0.926304347826087
135.6523745059967: train_loss:0.017741777011358106 test_acc:0.9332480818414323 val loss:0.3812161186395822 acc:0.9356521739130435
166.34570217132568: train_loss:0.00978720916073081 test_acc:0.9365728900255754 val loss:0.4200164081230529 acc:0.9360869565217391
197.0086634159088: train_loss:0.017678937207059663 test_acc:0.9383631713554987 val loss:0.4531469524172703 acc:0.9302173913043478
227.70532321929932: train_loss:0.023839650410977326 test_acc:0.9383631713554987 val loss:0.4844976224217421 acc:0.9271739130434783
258.1768400669098: train_loss:0.0004290991357458417 test_acc:0.9452685421994885 val loss:0.4

In [65]:
a=ANN([256,128,256,128,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.01,t)
print(loss)

43.05595326423645: train_loss:0.13969006541728313 test_acc:0.9132992327365729 val loss:0.3289942777009638 acc:0.9110869565217391
77.03307914733887: train_loss:0.059864478080335466 test_acc:0.9257033248081842 val loss:0.35930738821854163 acc:0.9232608695652174
111.03222751617432: train_loss:0.017096907049738184 test_acc:0.9364450127877237 val loss:0.3716435099196049 acc:0.9330434782608695
141.0550172328949: train_loss:0.0014214886576775576 test_acc:0.9479539641943734 val loss:0.3495997731567689 acc:0.9441304347826087
175.05461883544922: train_loss:0.0005394945310253137 test_acc:0.949616368286445 val loss:0.3665529463592259 acc:0.9454347826086956
209.14695191383362: train_loss:0.0004644095913548144 test_acc:0.949616368286445 val loss:0.37794177267730406 acc:0.9445652173913044
239.34978795051575: train_loss:0.00043136535143561456 test_acc:0.949616368286445 val loss:0.38469256668612145 acc:0.9456521739130435
264.97272658348083: train_loss:0.00041353033575351614 test_acc:0.949616368286445 v

In [61]:
a=ANN([1024,512,256,128,64,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.01,t)
print(loss)

47.40791320800781: train_loss:0.42277894174166764 test_acc:0.8480818414322251 val loss:0.5126006176354457 acc:0.8552173913043478
87.7118992805481: train_loss:0.21842451859309764 test_acc:0.8879795396419438 val loss:0.3852807350303759 acc:0.8893478260869565
127.9761803150177: train_loss:0.11591946399753598 test_acc:0.9135549872122762 val loss:0.3283117312584686 acc:0.9130434782608695
168.32944703102112: train_loss:0.07875942694631981 test_acc:0.9254475703324808 val loss:0.2995744550833555 acc:0.9228260869565217
208.55870413780212: train_loss:0.0357228781143441 test_acc:0.9315856777493606 val loss:0.2893856147819876 acc:0.935
248.77161264419556: train_loss:0.036473020024979375 test_acc:0.9327365728900255 val loss:0.31374912124639853 acc:0.931304347826087
270.50874853134155: train_loss:0.019076261476553846 test_acc:0.9361892583120205 val loss:0.2962700136854112 acc:0.936304347826087
290.0964834690094: train_loss:0.01716341519958125 test_acc:0.9369565217391305 val loss:0.2889385890612051 a

In [58]:
a=ANN([512,256,128,128,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.01,t)
print(loss)

44.57294821739197: train_loss:0.1796797787483442 test_acc:0.9058823529411765 val loss:0.3298823051760812 acc:0.9082608695652173
81.39089274406433: train_loss:0.11414935408156233 test_acc:0.9202046035805627 val loss:0.38292789840970964 acc:0.9119565217391304
116.37184143066406: train_loss:0.028298479282018802 test_acc:0.9367007672634271 val loss:0.3100015717671789 acc:0.9321739130434783
152.8537871837616: train_loss:0.003323510502790771 test_acc:0.9441176470588235 val loss:0.29066796765553754 acc:0.9458695652173913
189.12273335456848: train_loss:0.0005840333661155511 test_acc:0.94769820971867 val loss:0.2997047249594942 acc:0.9458695652173913
219.82368803024292: train_loss:0.0004457476740783466 test_acc:0.9483375959079284 val loss:0.30680855420125946 acc:0.945
254.90063643455505: train_loss:0.0003639169793008708 test_acc:0.9483375959079284 val loss:0.3130846877794576 acc:0.9460869565217391
269.5056140422821: train_loss:0.0003420105473684642 test_acc:0.9483375959079284 val loss:0.3151375

In [66]:
a=ANN([512,256,128,128,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.02,t)
print(loss)

40.99072051048279: train_loss:0.16868984484848681 test_acc:0.9097186700767264 val loss:0.31995765494570316 acc:0.9063043478260869
73.42544388771057: train_loss:0.08669732154331952 test_acc:0.9250639386189259 val loss:0.3094399223323513 acc:0.9252173913043479
105.76535606384277: train_loss:0.03919317596558535 test_acc:0.9356777493606138 val loss:0.29352202123856674 acc:0.9341304347826087
138.20045733451843: train_loss:0.025982386948331805 test_acc:0.9379795396419437 val loss:0.27412895753816147 acc:0.9441304347826087
170.50446677207947: train_loss:0.018375166848529235 test_acc:0.9414322250639386 val loss:0.3330124252655081 acc:0.9415217391304348
202.84794902801514: train_loss:0.013394854506498159 test_acc:0.9437340153452686 val loss:0.29935172204676 acc:0.9430434782608695
235.13612270355225: train_loss:0.006049070970172094 test_acc:0.9460358056265985 val loss:0.3031988643970789 acc:0.9480434782608695
267.47882175445557: train_loss:0.0069529734301575505 test_acc:0.9478260869565217 val lo

In [81]:
a=ANN([512,256,128,128,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X[:-3000],y[:-3000],X[-3000:],y[-3000:],.02,t)
print(loss)

40.081953287124634: train_loss:0.16496862947134014 test_acc:0.9093333333333333 val loss:0.32871099466151243 acc:0.9130434782608695
71.8319034576416: train_loss:0.07303015477463039 test_acc:0.9276666666666666 val loss:0.318901439689024 acc:0.9271739130434783
104.20485544204712: train_loss:0.036603412523440924 test_acc:0.9363333333333334 val loss:0.30283446131944014 acc:0.9393478260869565
137.40031790733337: train_loss:0.021100356305079084 test_acc:0.9443333333333334 val loss:0.2872399775875575 acc:0.9404347826086956
170.298269033432: train_loss:0.016533412284951064 test_acc:0.9443333333333334 val loss:0.34056767860403003 acc:0.9421739130434783
201.7262225151062: train_loss:0.014208477862478862 test_acc:0.9443333333333334 val loss:0.2953918597318869 acc:0.9445652173913044
233.9071786403656: train_loss:0.0030428821456885686 test_acc:0.9493333333333334 val loss:0.31919569727331754 acc:0.9491304347826087
265.66612911224365: train_loss:0.00014132492437420724 test_acc:0.9513333333333334 val l

In [76]:
a=ANN([512,256,128,128,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.03,t)
print(loss)

43.95992565155029: train_loss:0.12412107190841198 test_acc:0.9186700767263427 val loss:0.31156829886527043 acc:0.9121739130434783
78.79885077476501: train_loss:0.051540890734467285 test_acc:0.931457800511509 val loss:0.28756910812543807 acc:0.9304347826086956
113.53880167007446: train_loss:0.026691653250763926 test_acc:0.9429667519181586 val loss:0.3140425681035258 acc:0.9389130434782609
148.61378574371338: train_loss:0.02795752607744958 test_acc:0.9451406649616368 val loss:0.36482713614487283 acc:0.9371739130434783
183.63577914237976: train_loss:0.019284409513587644 test_acc:0.9451406649616368 val loss:0.33162678047980854 acc:0.9441304347826087
220.62057757377625: train_loss:0.0063192580817191535 test_acc:0.94769820971867 val loss:0.331605088757638 acc:0.9489130434782609
256.33756589889526: train_loss:0.011063150505148132 test_acc:0.94769820971867 val loss:0.37016355108915716 acc:0.946304347826087
264.35750365257263: train_loss:0.012924507835683833 test_acc:0.94769820971867 val loss:0

In [69]:
a=ANN([512,256,256,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.02,t)
print(loss)

41.4821982383728: train_loss:0.1953169856007383 test_acc:0.9058823529411765 val loss:0.3456525543027837 acc:0.9034782608695652
74.49162292480469: train_loss:0.07334036958449709 test_acc:0.9285166240409207 val loss:0.29144766359792873 acc:0.9256521739130434
107.27178192138672: train_loss:0.020507256421982548 test_acc:0.9420716112531969 val loss:0.2587719333704231 acc:0.9397826086956522
140.25158262252808: train_loss:0.003589813808538389 test_acc:0.9510230179028133 val loss:0.2521024248866194 acc:0.9465217391304348
173.20758271217346: train_loss:0.0008542498565637495 test_acc:0.9540920716112532 val loss:0.25428812256040106 acc:0.9508695652173913
206.17899990081787: train_loss:0.0006454728529170661 test_acc:0.9542199488491049 val loss:0.2605393134547744 acc:0.9515217391304348
238.92146372795105: train_loss:0.000550063604174286 test_acc:0.9542199488491049 val loss:0.2650973469355215 acc:0.9517391304347826
271.92225098609924: train_loss:0.0004910141034548838 test_acc:0.9542199488491049 val 

In [72]:
a=ANN([512,256,256,46],2,1024,0)
t=10
loss=a.fit_mom(200,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.03,t)
print(loss)

41.64327359199524: train_loss:0.2332620987634109 test_acc:0.8965473145780051 val loss:0.3628071144208353 acc:0.8932608695652174
75.69082045555115: train_loss:0.11916599067660678 test_acc:0.9251918158567775 val loss:0.2955916100084705 acc:0.9132608695652173
111.38012528419495: train_loss:0.05563229694561047 test_acc:0.9301790281329924 val loss:0.26450241726910917 acc:0.933695652173913
147.5569453239441: train_loss:0.015360137216163658 test_acc:0.9439897698209718 val loss:0.24421590168182805 acc:0.9430434782608695
182.98660707473755: train_loss:0.003794724508541884 test_acc:0.950383631713555 val loss:0.2288504531667525 acc:0.951304347826087
218.03302311897278: train_loss:0.001638222420652297 test_acc:0.9535805626598466 val loss:0.2325209066461335 acc:0.9545652173913044
252.7469732761383: train_loss:0.0010821558923690396 test_acc:0.9535805626598466 val loss:0.24056577092746767 acc:0.9543478260869566
265.7082440853119: train_loss:0.0010473493032529878 test_acc:0.9535805626598466 val loss:0

In [75]:
a=ANN([512,256,256,46],2,1024,0)
t=10
loss=a.fit_mom(200,100,X[:-4600],y[:-4600],X[-4600:],y[-4600:],.03,t)
print(loss)

42.4039409160614: train_loss:0.11436793984406049 test_acc:0.9267391304347826 val loss:0.2799860269975428 acc:0.9232608695652174
75.92689061164856: train_loss:0.04034812916379725 test_acc:0.9410869565217391 val loss:0.2624130386392817 acc:0.9410869565217391
109.57883882522583: train_loss:0.00475942354466963 test_acc:0.953695652173913 val loss:0.23688297804160405 acc:0.9515217391304348
143.54989075660706: train_loss:0.0004123258930839416 test_acc:0.9580434782608696 val loss:0.2371463244625777 acc:0.9578260869565217
179.52984189987183: train_loss:0.0002647787548149624 test_acc:0.9580434782608696 val loss:0.24566424987487825 acc:0.9576086956521739
210.45980978012085: train_loss:0.0002134846977379771 test_acc:0.9586956521739131 val loss:0.2497513500542939 acc:0.9580434782608696
245.50075721740723: train_loss:0.00017440113572679905 test_acc:0.9591304347826087 val loss:0.2535047286708873 acc:0.9584782608695652
265.75472831726074: train_loss:0.00015777360948021263 test_acc:0.9591304347826087 v

In [95]:
a=ANN([512,256,256,46],2,1024,0)
t=10
loss=a.fit_mom(200,100,X[:-4600],y[:-4600],X[-4600:],y[-4600:],.03,t)
print(loss)

40.409884214401245: train_loss:0.12829757955840818 test_acc:0.9293478260869565 val loss:0.26738867291073515 acc:0.9228260869565217
71.41286134719849: train_loss:0.06916113035050046 test_acc:0.936304347826087 val loss:0.2754044208840598 acc:0.927608695652174
102.57964372634888: train_loss:0.03755930245413452 test_acc:0.9389130434782609 val loss:0.28874733440401545 acc:0.9330434782608695
133.9258165359497: train_loss:0.00402313680442555 test_acc:0.9508695652173913 val loss:0.23318355972404364 acc:0.9517391304347826
165.23503160476685: train_loss:0.00042005149832913594 test_acc:0.9571739130434782 val loss:0.23372000658881564 acc:0.9556521739130435
195.97104024887085: train_loss:0.0002878701687416236 test_acc:0.9576086956521739 val loss:0.24118928137138096 acc:0.9571739130434782
227.56770610809326: train_loss:0.0002271951593885984 test_acc:0.9576086956521739 val loss:0.24623369689866176 acc:0.9578260869565217
258.68665409088135: train_loss:0.00019031215249189925 test_acc:0.9576086956521739

In [80]:
a=ANN([512,256,256,46],2,1024,0)
t=10
loss=a.fit_mom(200,100,X[:-3000],y[:-3000],X[-3000:],y[-3000:],.03,t)
print(loss)

42.56695079803467: train_loss:0.09554930801234618 test_acc:0.9323333333333333 val loss:0.25019959801270847 acc:0.9282608695652174
76.48290014266968: train_loss:0.03781283831516697 test_acc:0.9403333333333334 val loss:0.251778954655288 acc:0.9380434782608695
110.31585049629211: train_loss:0.0093439896179738 test_acc:0.948 val loss:0.24439797618052828 acc:0.9454347826086956
146.4455258846283: train_loss:0.00039804466274266713 test_acc:0.9533333333333334 val loss:0.22383631567390813 acc:0.9565217391304348
179.3161907196045: train_loss:0.0002810947921482433 test_acc:0.9533333333333334 val loss:0.22903411618645064 acc:0.9569565217391305
210.7321436405182: train_loss:0.00022427390987310417 test_acc:0.9533333333333334 val loss:0.23318318524628065 acc:0.9569565217391305
246.1370975971222: train_loss:0.0001815996507514041 test_acc:0.954 val loss:0.2372024738914938 acc:0.9565217391304348
269.48506116867065: train_loss:0.00016369263350073018 test_acc:0.954 val loss:0.2392531959448399 acc:0.956521

In [71]:
a=ANN([512,512,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.02,t)
print(loss)

41.875457763671875: train_loss:0.2379354841598592 test_acc:0.9080562659846547 val loss:0.3378465176933245 acc:0.9043478260869565
80.06733798980713: train_loss:0.09936203662660663 test_acc:0.9350383631713555 val loss:0.24445117441031208 acc:0.9308695652173913
119.85888504981995: train_loss:0.06488108469035558 test_acc:0.9364450127877237 val loss:0.24668046734821675 acc:0.9302173913043478
160.36369061470032: train_loss:0.024026858619412058 test_acc:0.9428388746803069 val loss:0.20999714349337614 acc:0.9423913043478261
201.01279067993164: train_loss:0.011735644381758954 test_acc:0.9474424552429668 val loss:0.20586895089746501 acc:0.9484782608695652
242.05000591278076: train_loss:0.005909021511908479 test_acc:0.9480818414322251 val loss:0.203259152399204 acc:0.9508695652173913
270.1389191150665: train_loss:0.004505887988859828 test_acc:0.94846547314578 val loss:0.2049297373277337 acc:0.952391304347826
285.41789722442627: train_loss:0.0040850082748103604 test_acc:0.9491048593350384 val loss

In [85]:
a=ANN([512,512,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X[:-4600],y[:-4600],X[-4600:],y[-4600:],.02,t)
print(loss)

42.38194966316223: train_loss:0.16128149638774195 test_acc:0.9239130434782609 val loss:0.2784238868741365 acc:0.917608695652174
75.81388783454895: train_loss:0.06838508760089507 test_acc:0.9360869565217391 val loss:0.24055884399359273 acc:0.9339130434782609
109.38483381271362: train_loss:0.028103206940178524 test_acc:0.9460869565217391 val loss:0.2244332071871174 acc:0.9417391304347826
143.23877549171448: train_loss:0.008926009013080211 test_acc:0.9480434782608695 val loss:0.2149175423163724 acc:0.9489130434782609
178.23272228240967: train_loss:0.004389468409952323 test_acc:0.9508695652173913 val loss:0.2150540959374948 acc:0.9508695652173913
211.24367237091064: train_loss:0.0030148425132350857 test_acc:0.9521739130434783 val loss:0.21644434458692147 acc:0.9519565217391305
244.462628364563: train_loss:0.0023737799015236326 test_acc:0.9532608695652174 val loss:0.21870611625968836 acc:0.9521739130434783
270.02058959007263: train_loss:0.002078452337899832 test_acc:0.9532608695652174 val l

In [70]:
a=ANN([512,512,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.02,t)
print(loss)

44.36293387413025: train_loss:0.18073683461139037 test_acc:0.9158567774936062 val loss:0.29994761000352543 acc:0.917608695652174
80.08662033081055: train_loss:0.07343919031209581 test_acc:0.9319693094629156 val loss:0.24487209205059868 acc:0.9330434782608695
115.94376254081726: train_loss:0.030021680969168033 test_acc:0.9383631713554987 val loss:0.23602869616431443 acc:0.9426086956521739
151.9185085296631: train_loss:0.009145063295632763 test_acc:0.9464194373401534 val loss:0.2209876589947629 acc:0.9491304347826087
187.9098629951477: train_loss:0.004126240144564174 test_acc:0.94693094629156 val loss:0.21699673457490398 acc:0.9530434782608695
223.6702573299408: train_loss:0.002921495562653962 test_acc:0.94769820971867 val loss:0.2210010098329331 acc:0.9530434782608695
259.717764377594: train_loss:0.0022997021478676685 test_acc:0.94769820971867 val loss:0.22487996078369504 acc:0.953695652173913
269.83374857902527: train_loss:0.002181011450072151 test_acc:0.9478260869565217 val loss:0.225

In [74]:
a=ANN([1024,1024,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.03,t)
print(loss)

40.56820869445801: train_loss:0.46839543025487 test_acc:0.8521739130434782 val loss:0.5321649925856171 acc:0.8586956521739131
75.40152406692505: train_loss:0.24763896760263113 test_acc:0.9057544757033248 val loss:0.3472491458647032 acc:0.9010869565217391
107.80459690093994: train_loss:0.15808120247284785 test_acc:0.9239130434782609 val loss:0.2806855102580965 acc:0.9147826086956522
139.05158138275146: train_loss:0.11348945709924026 test_acc:0.9301790281329924 val loss:0.25549075899830925 acc:0.9230434782608695
187.43898558616638: train_loss:0.05976339194172702 test_acc:0.9401534526854219 val loss:0.22430726250199096 acc:0.9373913043478261
232.6249144077301: train_loss:0.030762385360820942 test_acc:0.9427109974424552 val loss:0.21847761290936232 acc:0.9410869565217391
276.3908483982086: train_loss:0.010790640977519039 test_acc:0.9516624040920716 val loss:0.1951807697899671 acc:0.9521739130434783
307.03080201148987: train_loss:0.006996356884959013 test_acc:0.9526854219948849 val loss:0.1

In [59]:
a=ANN([512,256,256,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.01,t)
print(loss)

41.38517928123474: train_loss:0.27111940133239576 test_acc:0.8936061381074168 val loss:0.3676677291522402 acc:0.8917391304347826
75.18897867202759: train_loss:0.1089062633880117 test_acc:0.9274936061381074 val loss:0.26076186067548496 acc:0.9260869565217391
108.69157195091248: train_loss:0.05978080037016671 test_acc:0.931457800511509 val loss:0.25997599953813105 acc:0.9328260869565217
141.51609706878662: train_loss:0.04069353150692223 test_acc:0.9337595907928389 val loss:0.2787972916490774 acc:0.9347826086956522
174.39516353607178: train_loss:0.011477495106889554 test_acc:0.9416879795396419 val loss:0.2571441511393191 acc:0.9432608695652174
207.15118718147278: train_loss:0.0034230325472409382 test_acc:0.9456521739130435 val loss:0.25246862800514397 acc:0.9471739130434783
239.87031841278076: train_loss:0.002221499662185178 test_acc:0.9464194373401534 val loss:0.2563379630784779 acc:0.9476086956521739
264.97727704048157: train_loss:0.0018342210042716002 test_acc:0.94693094629156 val loss

In [67]:
a=ANN([512,256,256,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.02,t)
print(loss)

41.629854917526245: train_loss:0.14973734327446533 test_acc:0.917774936061381 val loss:0.2866384447369292 acc:0.9182608695652174
77.19422960281372: train_loss:0.06410900827497469 test_acc:0.9309462915601023 val loss:0.26436951588728297 acc:0.9295652173913044
112.67437815666199: train_loss:0.02381636184129077 test_acc:0.9346547314578005 val loss:0.263800962846603 acc:0.9408695652173913
149.66728401184082: train_loss:0.0038204634161079646 test_acc:0.949616368286445 val loss:0.24482140556015136 acc:0.95
185.07041835784912: train_loss:0.0008468743716388611 test_acc:0.9523017902813299 val loss:0.23689611706756003 acc:0.9552173913043478
218.83436393737793: train_loss:0.0006521076395288145 test_acc:0.9528132992327366 val loss:0.24406591513150008 acc:0.9530434782608695
252.06135845184326: train_loss:0.0005647578563364955 test_acc:0.9535805626598466 val loss:0.24873094664510212 acc:0.9534782608695652
269.20358657836914: train_loss:0.0005367949917437183 test_acc:0.9537084398976982 val loss:0.250

In [60]:
a=ANN([512,256,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.01,t)
print(loss)

46.25514054298401: train_loss:0.26547285027025863 test_acc:0.9019181585677749 val loss:0.3676401966293095 acc:0.8995652173913044
76.42666268348694: train_loss:0.13583169584825758 test_acc:0.9216112531969309 val loss:0.2870738112762597 acc:0.9217391304347826
106.54511022567749: train_loss:0.07839401196628214 test_acc:0.9312020460358056 val loss:0.26385320926774375 acc:0.9297826086956522
136.76750683784485: train_loss:0.04172269813032992 test_acc:0.9352941176470588 val loss:0.24596626678208153 acc:0.9360869565217391
166.99532628059387: train_loss:0.02335890719801453 test_acc:0.939386189258312 val loss:0.2416835019141257 acc:0.9369565217391305
199.2639935016632: train_loss:0.015903610137241335 test_acc:0.9410485933503836 val loss:0.2477999308542595 acc:0.9391304347826087
230.70724987983704: train_loss:0.01053396650979548 test_acc:0.9427109974424552 val loss:0.2484263353787227 acc:0.9391304347826087
260.8614149093628: train_loss:0.007443752343327421 test_acc:0.9437340153452686 val loss:0.2

In [68]:
a=ANN([512,256,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.03,t)
print(loss)

46.25674080848694: train_loss:0.11490916211742856 test_acc:0.9226342710997443 val loss:0.2760795054470436 acc:0.9202173913043479
76.42368483543396: train_loss:0.04651299090993473 test_acc:0.9332480818414323 val loss:0.26113418242471415 acc:0.933695652173913
106.49452376365662: train_loss:0.009411723296352568 test_acc:0.9480818414322251 val loss:0.220510712200291 acc:0.9506521739130435
136.65159440040588: train_loss:0.002260232293768041 test_acc:0.9524296675191816 val loss:0.21149507305992787 acc:0.9539130434782609
166.75990414619446: train_loss:0.0015522928052144724 test_acc:0.9530690537084399 val loss:0.21565025150355815 acc:0.955
196.99534225463867: train_loss:0.0012359357578498394 test_acc:0.9535805626598466 val loss:0.21851838821090122 acc:0.9554347826086956
234.11841750144958: train_loss:0.0010096598671228204 test_acc:0.9539641943734015 val loss:0.22129908457477376 acc:0.9552173913043478
264.6541676521301: train_loss:0.000892391122512749 test_acc:0.9542199488491049 val loss:0.2232

In [79]:
a=ANN([512,256,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X[:-3000],y[:-3000],X[-3000:],y[-3000:],.03,t)
print(loss)

43.18994998931885: train_loss:0.09048313945513242 test_acc:0.934 val loss:0.2540797840470409 acc:0.9291304347826087
78.24106574058533: train_loss:0.024677674065927375 test_acc:0.9433333333333334 val loss:0.24465243997647765 acc:0.9391304347826087
109.93901586532593: train_loss:0.00547697772914037 test_acc:0.9463333333333334 val loss:0.2266004035058861 acc:0.9482608695652174
140.68196845054626: train_loss:0.0017388076438267417 test_acc:0.952 val loss:0.22978980232542034 acc:0.9510869565217391
171.91992330551147: train_loss:0.0012290047436839479 test_acc:0.9526666666666667 val loss:0.23463363819529306 acc:0.9515217391304348
206.27187156677246: train_loss:0.000912465192029155 test_acc:0.9526666666666667 val loss:0.23907420479929006 acc:0.952391304347826
240.61181926727295: train_loss:0.0007247172287255706 test_acc:0.9526666666666667 val loss:0.24244173023852109 acc:0.9530434782608695
268.3977711200714: train_loss:0.0006213314944791805 test_acc:0.9526666666666667 val loss:0.244690042105278

In [86]:
a=ANN([512,256,46],2,1024,0)
t=10
loss=a.fit_mom(150,100,X[:-4600],y[:-4600],X[-4600:],y[-4600:],.03,t)
print(loss)

44.46494698524475: train_loss:0.0936535790818405 test_acc:0.9310869565217391 val loss:0.24240935069963065 acc:0.9302173913043478
79.53789353370667: train_loss:0.031804517378189186 test_acc:0.9410869565217391 val loss:0.24706425107917895 acc:0.9395652173913044
114.30884146690369: train_loss:0.00366805510867973 test_acc:0.9532608695652174 val loss:0.21660787235456638 acc:0.9502173913043478
150.3997881412506: train_loss:0.0017304294491060823 test_acc:0.9532608695652174 val loss:0.21924960090084714 acc:0.9528260869565217
185.64973521232605: train_loss:0.0013308191785239526 test_acc:0.9534782608695652 val loss:0.22268984374903747 acc:0.9530434782608695
221.66668057441711: train_loss:0.00110922137339981 test_acc:0.9534782608695652 val loss:0.2253790800430979 acc:0.9541304347826087
256.9666426181793: train_loss:0.0009670380205599357 test_acc:0.953695652173913 val loss:0.22761275025360103 acc:0.9543478260869566
264.9406313896179: train_loss:0.0009448331965669297 test_acc:0.953695652173913 val 

In [73]:
a=ANN([512,256,46],2,1024,0)
t=10
loss=a.fit_mom(200,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],.04,t)
print(loss)

46.90794014930725: train_loss:0.13978623000508275 test_acc:0.9176470588235294 val loss:0.28038170627961456 acc:0.9189130434782609
83.65839409828186: train_loss:0.06153740115158231 test_acc:0.9272378516624041 val loss:0.27053321571445393 acc:0.9273913043478261
122.00581622123718: train_loss:0.014293695332916584 test_acc:0.9436061381074169 val loss:0.23843051103149102 acc:0.9443478260869566
157.99660897254944: train_loss:0.0028899494741573503 test_acc:0.9489769820971867 val loss:0.22424910316043442 acc:0.9502173913043478
194.18672633171082: train_loss:0.0019944121847373397 test_acc:0.9501278772378516 val loss:0.22818197668602133 acc:0.9515217391304348
230.40885376930237: train_loss:0.0015934476623618314 test_acc:0.9515345268542199 val loss:0.23099691466644054 acc:0.951304347826087
266.48199367523193: train_loss:0.0013599478811423577 test_acc:0.9516624040920716 val loss:0.23352664060274433 acc:0.9519565217391305
276.772500038147: train_loss:0.0013173173477007611 test_acc:0.951662404092071

In [77]:
a=ANN([512,256,46],2,1024,0)
t=10
loss=a.fit_mom(200,100,X[:-4600],y[:-4600],X[-4600:],y[-4600:],.04,t)
print(loss)

45.97894597053528: train_loss:0.07881469784647963 test_acc:0.9317391304347826 val loss:0.24895252373275512 acc:0.936304347826087
77.25290060043335: train_loss:0.02602879182921264 test_acc:0.94 val loss:0.23231868609024267 acc:0.9421739130434783
108.09285426139832: train_loss:0.0022034076930860656 test_acc:0.9532608695652174 val loss:0.22113495823981605 acc:0.9517391304347826
138.74881505966187: train_loss:0.001239952464399942 test_acc:0.9534782608695652 val loss:0.22506388120688875 acc:0.952391304347826
169.7127673625946: train_loss:0.0009042555249615498 test_acc:0.953695652173913 val loss:0.22912478048074356 acc:0.9515217391304348
202.76022505760193: train_loss:0.0007100053947015633 test_acc:0.9539130434782609 val loss:0.232358121322313 acc:0.9517391304347826
234.23717713356018: train_loss:0.0005839904652394818 test_acc:0.9539130434782609 val loss:0.23503607439235324 acc:0.951304347826087
264.72213196754456: train_loss:0.0004958700878914641 test_acc:0.9539130434782609 val loss:0.23741

In [78]:
a=ANN([512,256,46],2,1024,0)
t=10
loss=a.fit_mom(200,100,X[:-3000],y[:-3000],X[-3000:],y[-3000:],.04,t)
print(loss)

40.643953800201416: train_loss:0.09789081658740091 test_acc:0.93 val loss:0.2615154527542008 acc:0.9304347826086956
73.53290700912476: train_loss:0.023500941994804743 test_acc:0.9453333333333334 val loss:0.23349992886322637 acc:0.9408695652173913
104.28686046600342: train_loss:0.0030725040454500615 test_acc:0.952 val loss:0.22754231605120012 acc:0.9497826086956521
137.11081266403198: train_loss:0.0012639650140734537 test_acc:0.956 val loss:0.23152896976108503 acc:0.9530434782608695
170.2477638721466: train_loss:0.000895309136175661 test_acc:0.956 val loss:0.23555572157428165 acc:0.9528260869565217
204.72471117973328: train_loss:0.0006968105408350091 test_acc:0.9566666666666667 val loss:0.2387884069567739 acc:0.9530434782608695
237.69266200065613: train_loss:0.0005702906052649592 test_acc:0.9566666666666667 val loss:0.24164022107513272 acc:0.9532608695652174
265.50461983680725: train_loss:0.0004978818664100202 test_acc:0.9566666666666667 val loss:0.24364145113550298 acc:0.95347826086956

In [53]:
a=ANN([512,256,128,64,46],2,1024,0)
t=10
loss=a.fit_adam(750,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],8e-3,t)
print(loss)

43.175949811935425: train_loss:0.14233548078646738 test_acc:0.9062659846547314 val loss:0.35983846701693517 acc:0.9069565217391304
76.70089888572693: train_loss:0.05031158287521853 test_acc:0.9228900255754475 val loss:0.37630741341667023 acc:0.9215217391304348
110.11784934997559: train_loss:0.027229892134822874 test_acc:0.9258312020460358 val loss:0.4123399667708028 acc:0.925
143.53679943084717: train_loss:0.030734953829462452 test_acc:0.9271099744245525 val loss:0.4857495686806458 acc:0.9252173913043479
177.34074759483337: train_loss:0.028594697174064612 test_acc:0.9287723785166241 val loss:0.4723198054002998 acc:0.9315217391304348
211.3806962966919: train_loss:0.028956470546354314 test_acc:0.9287723785166241 val loss:0.520832257892302 acc:0.9284782608695652
245.4826431274414: train_loss:0.02569672724725539 test_acc:0.9287723785166241 val loss:0.5256079428386186 acc:0.9295652173913044
269.1996080875397: train_loss:0.018520051187218618 test_acc:0.9287723785166241 val loss:0.55126960461

In [54]:
a=ANN([512,256,128,64,46],0,1024,0)
t=10
loss=a.fit_rmsprop(450,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],8e-3,t)
print(loss)

42.28795075416565: train_loss:0.5052172422759501 test_acc:0.8204603580562659 val loss:0.6541471669687453 acc:0.8145652173913044
77.4858992099762: train_loss:0.13428203408888717 test_acc:0.9076726342710998 val loss:0.37395202043494924 acc:0.9067391304347826
107.70585370063782: train_loss:0.06396408015746953 test_acc:0.921227621483376 val loss:0.3493682818516102 acc:0.9241304347826087
143.25080394744873: train_loss:0.05940466803692268 test_acc:0.9230179028132992 val loss:0.4003668890340231 acc:0.9208695652173913
176.7117531299591: train_loss:0.037368625338639376 test_acc:0.9276214833759591 val loss:0.4254416753691534 acc:0.9191304347826087
211.70470142364502: train_loss:0.04016745921203442 test_acc:0.928388746803069 val loss:0.4306455568936109 acc:0.9223913043478261
246.09364819526672: train_loss:0.02211348724304685 test_acc:0.9327365728900255 val loss:0.4332278720987436 acc:0.9284782608695652
267.38361620903015: train_loss:0.01880379354618827 test_acc:0.9327365728900255 val loss:0.42972

In [57]:
a=ANN([256,46],2,1024,0)
t=10
loss=a.fit_nag(150,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],2e-2,t)
print(loss)

40.73895454406738: train_loss:0.0871873273039065 test_acc:0.9226342710997443 val loss:0.2910229475363546 acc:0.9208695652173913
72.50990581512451: train_loss:0.019679890787943383 test_acc:0.9340153452685422 val loss:0.27231522293632543 acc:0.9391304347826087
103.9488570690155: train_loss:0.007694698455458168 test_acc:0.9359335038363171 val loss:0.27132242237580456 acc:0.941304347826087
134.9597990512848: train_loss:0.004581719227916861 test_acc:0.9375959079283888 val loss:0.2769035205505894 acc:0.9419565217391305
165.06175589561462: train_loss:0.0033308313801123065 test_acc:0.9377237851662404 val loss:0.28275555805666885 acc:0.9404347826086956
196.3687093257904: train_loss:0.002516797746965468 test_acc:0.9382352941176471 val loss:0.28826842820254023 acc:0.941304347826087
227.49765539169312: train_loss:0.0020188798768587327 test_acc:0.9383631713554987 val loss:0.29289913906878445 acc:0.9410869565217391
257.86660957336426: train_loss:0.001717757028334523 test_acc:0.9387468030690537 val l

In [55]:
a=ANN([512,256,128,64,46],0,1024,0)
t=10
loss=a.fit_nadam(150,100,X[:-7820],y[:-7820],X[-7820:],y[-7820:],8e-3,t)
print(loss)

42.03195142745972: train_loss:0.7472613574130109 test_acc:0.7631713554987213 val loss:0.8293718830511755 acc:0.7697826086956522
78.78189516067505: train_loss:0.21710062208852707 test_acc:0.890153452685422 val loss:0.39558821765781665 acc:0.8913043478260869
112.49584436416626: train_loss:0.11209192308764163 test_acc:0.9006393861892583 val loss:0.3508727309950269 acc:0.9060869565217391
146.3507936000824: train_loss:0.08211634992038495 test_acc:0.9028132992327366 val loss:0.3562742009247172 acc:0.9102173913043479
180.12274289131165: train_loss:0.07296569045168307 test_acc:0.9049872122762148 val loss:0.39074377108223535 acc:0.9082608695652173
214.00669121742249: train_loss:0.04646988334280291 test_acc:0.9126598465473146 val loss:0.37387951983782064 acc:0.9156521739130434
247.88063979148865: train_loss:0.04010982625979221 test_acc:0.9164961636828645 val loss:0.37997418393328264 acc:0.917608695652174
265.5986144542694: train_loss:0.04416628729279405 test_acc:0.9164961636828645 val loss:0.403